In [1]:
import pandas as pd
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
import numpy as np
import warnings
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import joblib
import gc
from scipy.optimize import brentq
from bisect import bisect_right
from scipy.stats import norm
from functools import partial
import dask
from dask import delayed, compute
import dask.multiprocessing
from dask.diagnostics import ProgressBar 

from scipy.optimize import root_scalar

import boto3
import pyarrow.dataset as ds
from datetime import datetime

import re
import glob
import pytz
from datetime import datetime, timedelta
from bisect import bisect_right
from scipy.stats import norm
from scipy.optimize import brentq
from io import BytesIO  
from pyarrow import parquet as pq
import pyarrow.dataset as ds
import pandas as pd
import shutil 
import os

import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import gc


ModuleNotFoundError: No module named 'optuna'

In [2]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeable
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.7 MB/s eta 0:00:00
Using cached mako-1.3.10-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"
START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 15)

# Expiry Dates (Thursdays)
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

OUTPUT_FILE = "master_fut_df.csv"

# ==========================================
# UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    """Determines the active futures contract based on date."""
    if current_date <= EXPIRY_SEP:
        return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT:
        return "NIFTY25OCTFUT"
    else:
        return "NIFTY25NOVFUT"

def download_and_process_day(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    
    fut_ts = get_trading_symbol(date_obj)
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{fut_ts}.parquet"
    
    print(f"Processing {date_obj} | Symbol: {fut_ts}...", end=" ")
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        # 1. Standardize Date/Time
        # Ensure Date and Time columns exist for easy filtering later
        if 'Date' not in df.columns or 'Time' not in df.columns:
            # Fallback if DateTime is a single column
            if 'DateTime' in df.columns:
                dt = pd.to_datetime(df['DateTime'])
                df['Date'] = dt.dt.date
                df['Time'] = dt.dt.time
        
        # 2. Add Ticker
        df['Ticker'] = fut_ts
        
        # 3. Return As-Is (No renaming)
        print(f"Found {len(df)} rows.")
        return df
        
    except s3.exceptions.NoSuchKey:
        print("Data not found (Holiday/Weekend).")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# ==========================================
# MAIN LOOP
# ==========================================
def main():
    all_dfs = []
    current_date = START_DATE
    
    while current_date <= END_DATE:
        df = download_and_process_day(current_date)
        if df is not None and not df.empty:
            all_dfs.append(df)
        
        current_date += datetime.timedelta(days=1)
        
    if all_dfs:
        print("\nConcatenating master dataframe...")
        master_df = pd.concat(all_dfs, ignore_index=True)
        
        # Final Format Check
        print(f"Saving {len(master_df)} rows to {OUTPUT_FILE}...")
        master_df.to_csv(OUTPUT_FILE, index=False)
        print("Done.")
    else:
        print("No data downloaded.")

if __name__ == "__main__":
    main()

Processing 2025-09-01 | Symbol: NIFTY25SEPFUT... Found 33433 rows.
Processing 2025-09-02 | Symbol: NIFTY25SEPFUT... Found 35933 rows.
Processing 2025-09-03 | Symbol: NIFTY25SEPFUT... Found 25346 rows.
Processing 2025-09-04 | Symbol: NIFTY25SEPFUT... Found 36497 rows.
Processing 2025-09-05 | Symbol: NIFTY25SEPFUT... Found 35720 rows.
Processing 2025-09-06 | Symbol: NIFTY25SEPFUT... Data not found (Holiday/Weekend).
Processing 2025-09-07 | Symbol: NIFTY25SEPFUT... Data not found (Holiday/Weekend).
Processing 2025-09-08 | Symbol: NIFTY25SEPFUT... Found 35001 rows.
Processing 2025-09-09 | Symbol: NIFTY25SEPFUT... Found 30184 rows.
Processing 2025-09-10 | Symbol: NIFTY25SEPFUT... Found 38016 rows.
Processing 2025-09-11 | Symbol: NIFTY25SEPFUT... Found 30489 rows.
Processing 2025-09-12 | Symbol: NIFTY25SEPFUT... Found 33590 rows.
Processing 2025-09-13 | Symbol: NIFTY25SEPFUT... Data not found (Holiday/Weekend).
Processing 2025-09-14 | Symbol: NIFTY25SEPFUT... Data not found (Holiday/Weekend)

In [27]:
df_start = pd.read_csv('master_fut_df.csv')
df_start

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,01/09/2025,08:46:42.278,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,01/09/2025,09:10:01.994,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,01/09/2025,09:15:01.722,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,01/09/2025,09:15:02.471,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,01/09/2025,09:15:02.975,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,13/11/2025,15:29:33.077,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,13/11/2025,15:29:34.659,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,13/11/2025,15:29:35.142,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,13/11/2025,15:29:35.834,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [28]:
# define all the functions here

from datetime import datetime as dt

def clean_time_format(time_str):
    # Check if milliseconds are missing
    if '.' not in time_str:
        return f"{time_str}.000000"  # Append milliseconds
    else:
        # Ensure milliseconds are in the format `000000`
        time_parts = time_str.split('.')
        milliseconds = time_parts[1][:6].ljust(6, '0')  # Pad or truncate to 6 digits
        return f"{time_parts[0]}.{milliseconds}"  

def additional_columns(df):
    df = df.copy()
    df['Session'] = np.where((df['Date'] != df['Date'].shift(1)) & (df['Time'] != pd.to_datetime('00:00:00').time()), 0, np.nan)
    df['Session'] = np.where((df['Session'].shift(-1) == 0), 1, df['Session'])  
    df.at[df.index[-1], 'Session'] = 1
    
    return df

def add_opening_and_closing_trades(df, indicator_base, trade_indicator_base, num_indicators):
    df = df.copy()
    
    for i in range(num_indicators):
        indicator = f'{indicator_base}{i+1}'
        trade_indicator = f'{trade_indicator_base}{i+1}'
        
        df[f'Trade{trade_indicator}'] = np.where(
            (df['Session'] == 0) & (~np.isnan(df[f'Trend{trade_indicator}'])), 
            df['Open'] * df[f'Trend{trade_indicator}'], 
            df[f'Trade{trade_indicator}']
        )
        
        df[f'Trade{trade_indicator}'] = np.where(
            df[f'Trade{trade_indicator}'] == 0, 
            np.nan, 
            df[f'Trade{trade_indicator}']
        )
        
        df['Cum_non_NaN'] = df[f'Trade{trade_indicator}'].notna().cumsum()
        df['Has_non_NaN'] = np.where(
            (df['Session'] == 1) & (df['Cum_non_NaN'] > 0), 
            True, 
            False
        )
        
        df[f'Trend{trade_indicator}'] = np.where(
            (df['Session'] == 1) & (df['Has_non_NaN']) & 
            (df[f'Trend{trade_indicator}'] == df[f'Trend{trade_indicator}'].shift(1)), 
            df[f'Trend{trade_indicator}'] * -1, 
            df[f'Trend{trade_indicator}']
        )
        
        df[f'Trade{trade_indicator}'] = np.where(
            (df['Session'] == 1) & (df['Has_non_NaN']), 
            df['Close'] * df[f'Trend{trade_indicator}'], 
            df[f'Trade{trade_indicator}']
        )
        
        df = df.drop(columns=['Cum_non_NaN', 'Has_non_NaN'])
    
    return df

def resampled_ohlc(df):
    df = df.copy()
    df['Time'] = df['Time'].astype(str)
    df['Time'] = df['Time'].apply(clean_time_format) # Only when time contains milliseconds
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')

    df.set_index('datetime', inplace=True)

    #intervals = ['1s', '15s', '30s', '45s', '1min', '5min', '15min', '30min', '60min']
    intervals = ['15s']
    resampled_dfs = {}

    for interval in intervals:
        start_time = time.time()
        
        interval_start = df.index.floor(interval)
        df['group'] = interval_start
        df['GroupTime'] = interval_start.time
        df['TimePassed'] = ((pd.to_datetime(df['Time'], format='%H:%M:%S.%f') - pd.to_datetime(df['GroupTime'], format='%H:%M:%S')).dt.total_seconds())

        df['Open'] = df.groupby('group')['LTP'].transform('first')
        df['High'] = df.groupby('group')['LTP'].transform('cummax')
        df['Low'] = df.groupby('group')['LTP'].transform('cummin')
        df['Close'] = df['LTP'] 
        df['BestBid'] = df['BuyPrice']
        df['BestAsk'] = df['SellPrice']
        df['BidSize'] = df['BuyQty']
        df['AskSize'] = df['SellQty']
        df['Volume'] = df.groupby('group')['LTQ'].transform('cumsum')
                
        df['GroupNum'] = (df['group'] != df['group'].shift(1)).cumsum()

        df['GroupTimeOnly'] = df['group'].dt.time
        intervals_list = df['GroupTimeOnly'].unique()
        intervals_list_sorted = sorted(intervals_list)
        intervals_list_sorted = sorted(df['GroupTimeOnly'].unique())
        time_to_groupnum_mapping = {time: idx + 1 for idx, time in enumerate(intervals_list_sorted)}
        df['GroupNumTime'] = df['GroupTimeOnly'].map(time_to_groupnum_mapping)
        
        ohlc = df.copy()

        ohlc.reset_index(inplace=True)

        ohlc['Date'] = ohlc['datetime'].dt.strftime('%d/%m/%Y')
     #   ohlc['Time'] = ohlc['datetime'].dt.strftime('%H:%M:%S')

        ohlc.drop(columns=['datetime'], inplace=True)
        
        bar_change_mask = ohlc['GroupNum'] != ohlc['GroupNum'].shift(1)
        ohlc['PrevBar'] = np.nan
        ohlc['PrevPrevBar'] = np.nan

        ohlc['PrevBar'] = np.where(bar_change_mask, ohlc.index - 1, np.nan)
        ohlc['PrevBar'] = np.where(ohlc['PrevBar'] == -1, np.nan, ohlc['PrevBar'])
        ohlc['PrevBar'] = ohlc['PrevBar'].ffill()

        ohlc['PrevPrevBar'] = np.where(bar_change_mask, ohlc['PrevBar'].shift(1), np.nan)
        ohlc['PrevPrevBar'] = ohlc['PrevPrevBar'].ffill()

        ohlc.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'group': 'Group'}, inplace=True)
        ohlc.dropna(subset=['Open', 'High', 'Low', 'Close'], inplace=True)
        ohlc = ohlc[['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk', 'BidSize', 'AskSize', 'Volume', 'Group', 'GroupNumTime', 'TimePassed', 'GroupNum', 'PrevBar', 'PrevPrevBar']]
        
        ohlc = additional_columns(ohlc)
        ohlc['SessionBar'] = ohlc.groupby('Group')['Session'].transform(lambda x: (x == 0).any())
        
        ohlc = ohlc.reset_index(drop=True)
        
        resampled_dfs[interval] = [ohlc]

    return resampled_dfs



pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

from hurst import compute_Hc
from scipy.signal import hilbert

def rolling_hilbert(series):
    if len(series) < 10:  # Ensure enough data points
        return np.nan
    return np.angle(hilbert(series)[-1])  # Compute Hilbert phase for last value
    
def forecast_next(x):
        if len(x) < 2:
            return x[-1]
        X = np.arange(len(x))
        coef = np.polyfit(X, x, 1)   
        return np.polyval(coef, len(x))  

def hurst_exp(x):
        if len(x) > 10:
            H, _, _ = compute_Hc(x, kind='price', simplified=True)
            return H
        return np.nan

def generate_basic_variables(df):
    df = df.copy()

    open_dict, high_dict, low_dict, close_dict, volume_dict = df['Open'].to_dict(), df['High'].to_dict(), df['Low'].to_dict(), df['Close'].to_dict(), df['Volume'].to_dict()
    df['PrevOpen'] = df['PrevBar'].map(lambda x: open_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevHigh'] = df['PrevBar'].map(lambda x: high_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevLow'] = df['PrevBar'].map(lambda x: low_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevClose'] = df['PrevBar'].map(lambda x: close_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevVolume'] = df['PrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SecondInterval'] = np.floor(df['TimePassed']).where(df['TimePassed'] < 15, np.nan) + 1

    df['Spread'] = (df['BestAsk'] - df['BestBid']).round(2)
    df['SpreadSize'] = (df['AskSize'] - df['BidSize']).round(2)

    df['LTQ'] = np.where(df['GroupNum'] == df['GroupNum'].shift(1), df['Volume'] - df['Volume'].shift(1), df['Volume'])
    LTQ_dict = df['LTQ'].to_dict()
    df['PrevLTQ'] = df['PrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevLTQ'] = df['PrevPrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['MeanLTQ'] = df['LTQ'].expanding().mean().round(2)
    df['StdLTQ'] = df['LTQ'].expanding().std().round(2)
    df['SumPrevLTQ'] = df['PrevLTQ'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLTQ'] = df['SumPrevLTQ'].ffill()
    df['PrevMeanLTQ'] = (df['SumPrevLTQ']/(df['GroupNum']-1)).round(2)
    df['ZScoreLTQ'] = ((df['LTQ'] - df['MeanLTQ'])/df['StdLTQ']).round(2)
    df['SkewLTQVariance'] = ((df['LTQ'] - df['MeanLTQ'])**3).cumsum()
    df['SkewLTQ'] = (df['SkewLTQVariance']/((df['StdLTQ']**3)*(df.index))).round(2)
    df['LTQ_10th_Percentile'] = df['LTQ'].expanding().quantile(0.10)
    df['LTQ_25th_Percentile'] = df['LTQ'].expanding().quantile(0.25)
    df['LTQ_50th_Percentile'] = df['LTQ'].expanding().quantile(0.50)
    df['LTQ_75th_Percentile'] = df['LTQ'].expanding().quantile(0.75)
    df['LTQ_90th_Percentile'] = df['LTQ'].expanding().quantile(0.90)
    df['LTQIQR'] = (df['LTQ_75th_Percentile'] - df['LTQ_25th_Percentile']).round(2)
    df['PrevAutoCorLTQNumerator'] = ((df['LTQ'] - df['MeanLTQ']) * (df['LTQ'].shift(1) - df['MeanLTQ'])).cumsum()
    df['PrevAutoCorLTQDenominator'] = np.square(df['LTQ']- df['MeanLTQ']).cumsum()
    df['AutoCorrelationLTQ'] = (df['PrevAutoCorLTQNumerator']/df['PrevAutoCorLTQDenominator']).round(2)
    df['DeviationFlag'] = np.where(df['LTQ'] > (df['MeanLTQ'] + df['StdLTQ']), 1, 0)
    df['AbsoluteDeviationLTQ'] = df['DeviationFlag'].cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['LTQ'].where(df['AbsoluteDeviationLTQ'] != df['AbsoluteDeviationLTQ'].shift(1)).cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['SumAbsoluteDeviationLTQ'].ffill()
    df['MeanAbsoluteDeviationLTQ'] = np.where(df['AbsoluteDeviationLTQ'] != 0, (df['SumAbsoluteDeviationLTQ']/df['AbsoluteDeviationLTQ']).round(2), np.nan)
    
    volume_dict = df['Volume'].to_dict()
    df['PrevPrevVolume'] = df['PrevPrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevVolume'] = df['PrevVolume'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevVolume'] = df['SumPrevVolume'].ffill()
    df['PrevMeanVolume'] = (df['SumPrevVolume']/(df['GroupNum']-1)).round(2)
    df['MeanVolume'] = ((df['SumPrevVolume'] + df['Volume'])/df['GroupNum']).round(2)
    df['ResidualsVolume'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsVolume'] = df['ResidualsVolume'].ffill()
    df['PrevStdVolume'] = np.sqrt(df['ResidualsVolume']/df['GroupNum']-1)
    df['ResidualsVolume'] = df['ResidualsVolume'] + np.square(df['Volume'] - df['MeanVolume'])
    df['StdVolume'] = (np.sqrt(df['ResidualsVolume']/df['GroupNum'])).round(2)
    df['PrevZScoreVolume'] = ((df['PrevVolume']-df['PrevMeanVolume'])/df['PrevStdVolume']).round(2)
    df['ZScoreVolume'] = ((df['Volume']-df['MeanVolume'])/df['StdVolume']).round(2)
    df['SkewNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewVolume'] = df['PrevSkewVariance']/(df['PrevStdVolume']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Volume'] - df['MeanVolume'])**3)/df['GroupNum']
    df['SkewVolume'] = (df['SkewVariance']/(df['StdVolume']**3)).round(2)
    df['CVVolume'] = (df['StdVolume']/df['MeanVolume']).round(2)
    df['KurtosisNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisVolume'] = df['PrevKurtosisVariance']/(df['PrevStdVolume']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Volume'] - df['MeanVolume'])**4)/df['GroupNum']
    df['KurtosisVolume'] = (df['KurtosisVariance']/(df['StdVolume']**4)-3).round(2)
    df['Volume_10th_Percentile'] = df['Volume'].expanding().quantile(0.10)
    df['Volume_25th_Percentile'] = df['Volume'].expanding().quantile(0.25)
    df['Volume_50th_Percentile'] = df['Volume'].expanding().quantile(0.50)
    df['Volume_75th_Percentile'] = df['Volume'].expanding().quantile(0.75)
    df['Volume_90th_Percentile'] = df['Volume'].expanding().quantile(0.90)
    df['VolumeIQR'] = (df['Volume_75th_Percentile'] - df['Volume_25th_Percentile']).round(2)
    df['PrevAutoCorVolumeNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume']) * (df['PrevPrevVolume'] - df['PrevMeanVolume'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorVolumeDenominator'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationVolume'] = (df['PrevAutoCorVolumeNumerator']/df['PrevAutoCorVolumeDenominator']).round(2)
    df['AutoCorrelationVolume'] = df['AutoCorrelationVolume'].ffill()
    mask = df['GroupNum'] != df['GroupNum'].shift(1)
    df['TimeSumPrevVolume'] = (df.groupby('GroupNumTime')['PrevVolume'].apply(lambda group: group.where(mask).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevVolume'] = df['TimeSumPrevVolume'].ffill()
    df['TimeSumGroupNumTime'] = (df.groupby('GroupNumTime').apply(lambda group: group['GroupNum'].ne(group['GroupNum'].shift()).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevVolume'] = np.where(df['GroupNumTime'] == 1, 0, df['TimeSumPrevVolume'])
    df['TimeMeanVolume'] = (df['TimeSumPrevVolume']/df['TimeSumGroupNumTime']).round(2)
    df['VolumeOutlierSum1'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 1)).cumsum()
    df['VolumeOutlierSum1'] = df['VolumeOutlierSum1'].ffill()
    df['VolumeOutlierSum2'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 2)).cumsum()
    df['VolumeOutlierSum2'] = df['VolumeOutlierSum2'].ffill()
    df['VolumeOutlierSum3'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 3)).cumsum()
    df['VolumeOutlierSum3'] = df['VolumeOutlierSum3'].ffill()
    df['VolumeOutlierImpact1'] = (df['VolumeOutlierSum1']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact2'] = (df['VolumeOutlierSum2']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact3'] = (df['VolumeOutlierSum3']/df['SumPrevVolume']).round(2)
    df['ForwardVolume'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Volume'].shift(1), np.nan)
    df['ForwardVolume'] = df['ForwardVolume'].shift(-1)
    df['ForwardVolume'] = df['ForwardVolume'].bfill()
    df['ProportionVolume'] = (df['Volume']/df['ForwardVolume']).round(2)
    df['SpeedVolume'] = ((df['Volume']*15)/df['TimePassed']).round(2)
    df['SumIntervalMeanVolume'] = (df.groupby('SecondInterval')['ProportionVolume'].apply(lambda group: group.where(df['SecondInterval'] != df['SecondInterval'].shift(1)).cumsum()).reset_index(level=0, drop=True))
    df['SumIntervalMeanVolume'] = df['SumIntervalMeanVolume'].ffill()
    df['IntervalMeanVolume'] = (df['SumIntervalMeanVolume']/df['GroupNum']).round(2)

    df['Range'] = df['High'] - df['Low']
    df['ShiftedRange'] = df['Range'].shift(1).where(df['GroupNum'] != df['GroupNum'].shift(1))
    df['MinRange'] = df['ShiftedRange'].cummin()
    df['MinRange'] = df['MinRange'].ffill()
    range_dict = df['Range'].to_dict()
    df['PrevRange'] = df['PrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevRange'] = df['PrevPrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevRange'] = df['PrevRange'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevRange'] = df['SumPrevRange'].ffill()
    df['PrevMeanRange'] = (df['SumPrevRange']/(df['GroupNum']-1)).round(2)
    df['MeanRange'] = ((df['SumPrevRange'] + df['Range'])/df['GroupNum']).round(2)
    df['ResidualsRange'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsRange'] = df['ResidualsRange'].ffill()
    df['PrevStdRange'] = np.sqrt(df['ResidualsRange']/df['GroupNum']-1)
    df['ResidualsRange'] = df['ResidualsRange'] + np.square(df['Range'] - df['MeanRange'])
    df['StdRange'] = (np.sqrt(df['ResidualsRange']/df['GroupNum'])).round(2)
    df['PrevZScoreRange'] = ((df['PrevRange']-df['PrevMeanRange'])/df['PrevStdRange']).round(2)
    df['ZScoreRange'] = ((df['Range']-df['MeanRange'])/df['StdRange']).round(2)
    df['MaxRange'] = df['Range'].where(df['ZScoreRange'] <= 3).cummax()
    df['MaxRange'] = df['MaxRange'].ffill()
    df['SkewNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewRange'] = df['PrevSkewVariance']/(df['PrevStdRange']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Range'] - df['MeanRange'])**3)/df['GroupNum']
    df['SkewRange'] = (df['SkewVariance']/(df['StdRange']**3)).round(2)
    df['CVRange'] = (df['StdRange']/df['MeanRange']).round(2)
    df['KurtosisNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisRange'] = df['PrevKurtosisVariance']/(df['PrevStdRange']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Range'] - df['MeanRange'])**4)/df['GroupNum']
    df['KurtosisRange'] = (df['KurtosisVariance']/(df['StdRange']**4)-3).round(2)
    df['Range_10th_Percentile'] = df['Range'].expanding().quantile(0.10)
    df['Range_25th_Percentile'] = df['Range'].expanding().quantile(0.25)
    df['Range_50th_Percentile'] = df['Range'].expanding().quantile(0.50)
    df['Range_75th_Percentile'] = df['Range'].expanding().quantile(0.75)
    df['Range_90th_Percentile'] = df['Range'].expanding().quantile(0.90)
    df['RangeIQR'] = (df['Range_75th_Percentile'] - df['Range_25th_Percentile']).round(2)
    df['PrevAutoCorRangeNumerator'] = ((df['PrevRange'] - df['PrevMeanRange']) * (df['PrevPrevRange'] - df['PrevMeanRange'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorRangeDenominator'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationRange'] = (df['PrevAutoCorRangeNumerator']/df['PrevAutoCorRangeDenominator']).round(2)
    df['AutoCorrelationRange'] = df['AutoCorrelationRange'].ffill()
    df['NormalizedRange'] = np.minimum(((df['Range'] - df['MinRange'])/(df['MaxRange'] - df['MinRange'])).round(2), 1)
    mask = df['GroupNum'] != df['GroupNum'].shift(1)
    df['TimeSumPrevRange'] = (df.groupby('GroupNumTime')['PrevRange'].apply(lambda group: group.where(mask).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevRange'] = df['TimeSumPrevRange'].ffill()
    df['TimeSumGroupNumTime'] = (df.groupby('GroupNumTime').apply(lambda group: group['GroupNum'].ne(group['GroupNum'].shift()).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevRange'] = np.where(df['GroupNumTime'] == 1, 0, df['TimeSumPrevRange'])
    df['TimeMeanRange'] = (df['TimeSumPrevRange']/df['TimeSumGroupNumTime']).round(2)
    df['RangeOutlierSum1'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 1)).cumsum()
    df['RangeOutlierSum1'] = df['RangeOutlierSum1'].ffill()
    df['RangeOutlierSum2'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 2)).cumsum()
    df['RangeOutlierSum2'] = df['RangeOutlierSum2'].ffill()
    df['RangeOutlierSum3'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 3)).cumsum()
    df['RangeOutlierSum3'] = df['RangeOutlierSum3'].ffill()
    df['RangeOutlierImpact1'] = (df['RangeOutlierSum1']/df['SumPrevRange']).round(2)
    df['RangeOutlierImpact2'] = (df['RangeOutlierSum2']/df['SumPrevRange']).round(2)
    df['RangeOutlierImpact3'] = (df['RangeOutlierSum3']/df['SumPrevRange']).round(2)
    df['ForwardRange'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Range'].shift(1), np.nan)
    df['ForwardRange'] = df['ForwardRange'].shift(-1)
    df['ForwardRange'] = df['ForwardRange'].bfill()
    df['ProportionRange'] = (df['Range']/df['ForwardRange']).round(2)
    df['SpeedRange'] = ((df['Range']*15)/df['TimePassed']).round(2)
    df['SumIntervalMeanRange'] = (df.groupby('SecondInterval')['ProportionRange'].apply(lambda group: group.where(df['SecondInterval'] != df['SecondInterval'].shift(1)).cumsum()).reset_index(level=0, drop=True))
    df['SumIntervalMeanRange'] = df['SumIntervalMeanRange'].ffill()
    df['IntervalMeanRange'] = (df['SumIntervalMeanRange']/df['GroupNum']).round(2)

    df['Lows'] = (df['Low'] - df['PrevLow']).round(2)
    df['Lows'] = np.where(df['SessionBar'] == True, 0, df['Lows'])
    df['Lows'] = np.where(df['Lows'] < 0, 0, df['Lows'])
    df['LastLows'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Lows'].shift(1), np.nan)
    df['LastLows'] = df['LastLows'].shift(1)
    df['LastLows'] = df['LastLows'].ffill()
    df['LastLows'] = np.where(df['LastLows'] < 0, 0, df['LastLows'])
    lows_dict = df['Lows'].to_dict() 
    df['PrevLows'] = df['PrevBar'].map(lambda x: lows_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfLows'] = ((df['PrevLows'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfLows'] = np.where(df['Lows'] > 0, df['NumberOfLows']+1, df['NumberOfLows'])
    df['SumPrevLows'] = df['PrevLows'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLows'] = df['SumPrevLows'].ffill()
    df['PrevMeanLows'] = (df['SumPrevLows']/df['NumberOfLows']).round(2)
    df['MeanLows'] = ((df['SumPrevLows'] + df['Lows'])/df['CurrentNoOfLows']).round(2)
    df['ResidualsLows'] = (np.square(df['PrevLows'] - df['PrevMeanLows']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()).round(2)
    df['ResidualsLows'] = df['ResidualsLows'].ffill()
    df['PrevStdLows'] = np.sqrt(df['ResidualsLows']/df['NumberOfLows'])
    df['ResidualsLows'] = df['ResidualsLows'] + np.square(df['Lows'] - df['MeanLows'])
    df['StdLows'] = (np.sqrt(df['ResidualsLows']/df['CurrentNoOfLows'])).round(2)
    df['PrevZScoreLows'] = ((df['PrevLows']-df['PrevMeanLows'])/df['PrevStdLows']).round(2)
    df['ZScoreLows'] = ((df['Lows']-df['MeanLows'])/df['StdLows']).round(2)
    df['SkewLowsNumerator'] = ((df['PrevLows'] - df['PrevMeanLows'])**3).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()
    df['SkewLowsNumerator'] = df['SkewLowsNumerator'].ffill()
    df['PrevSkewLowsVariance'] = df['SkewLowsNumerator']/(df['NumberOfLows'])
    df['PrevSkewLows'] = df['PrevSkewLowsVariance']/(df['PrevStdLows']**3)
    df['SkewLowsVariance'] = (df['SkewLowsNumerator'] + (df['Lows'] - df['MeanLows'])**3)/df['CurrentNoOfLows']
    df['SkewLows'] = (df['SkewLowsVariance']/(df['StdLows']**3)).round(2)
    df['CVLows'] = (df['StdLows']/df['MeanLows']).round(2)
    df['KurtosisLowsNumerator'] = ((df['PrevLows'] - df['PrevMeanLows'])**4).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()
    df['KurtosisLowsNumerator'] = df['KurtosisLowsNumerator'].ffill()
    df['PrevKurtosisLowsVariance'] = df['KurtosisLowsNumerator']/(df['NumberOfLows'])
    df['PrevKurtosisLows'] = df['PrevKurtosisLowsVariance']/(df['PrevStdLows']**4)-3
    df['KurtosisLowsVariance'] = (df['KurtosisLowsNumerator'] + (df['Lows'] - df['MeanLows'])**4)/df['CurrentNoOfLows']
    df['KurtosisLows'] = (df['KurtosisLowsVariance']/(df['StdLows']**4)-3).round(2)
    df['Lows_10th_Percentile'] = df['Lows'].expanding().quantile(0.10)
    df['Lows_25th_Percentile'] = df['Lows'].expanding().quantile(0.25)
    df['Lows_50th_Percentile'] = df['Lows'].expanding().quantile(0.50)
    df['Lows_75th_Percentile'] = df['Lows'].expanding().quantile(0.75)
    df['Lows_90th_Percentile'] = df['Lows'].expanding().quantile(0.90)
    df['LowsIQR'] = (df['Lows_75th_Percentile'] - df['Lows_25th_Percentile']).round(2)

    df['Highs'] = (df['PrevHigh'] - df['High']).round(2)
    df['Highs'] = np.where(df['SessionBar'] == True, 0, df['Highs'])
    df['Highs'] = np.where(df['Highs'] < 0, 0, df['Highs'])
    df['LastHighs'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Highs'].shift(1), np.nan)
    df['LastHighs'] = df['LastHighs'].shift(1)
    df['LastHighs'] = df['LastHighs'].ffill()
    df['LastHighs'] = np.where(df['LastHighs'] < 0, 0, df['LastHighs'])
    highs_dict = df['Highs'].to_dict()
    df['PrevHighs'] = df['PrevBar'].map(lambda x: highs_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfHighs'] = ((df['PrevHighs'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfHighs'] = np.where(df['Lows'] > 0, df['NumberOfHighs']+1, df['NumberOfHighs'])
    df['SumPrevHighs'] = df['PrevHighs'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevHighs'] = df['SumPrevHighs'].ffill()
    df['PrevMeanHighs'] = (df['SumPrevHighs']/df['NumberOfHighs']).round(2)
    df['MeanHighs'] = ((df['SumPrevHighs'] + df['Highs'])/df['CurrentNoOfHighs']).round(2)
    df['ResidualsHighs'] = (np.square(df['PrevHighs'] - df['PrevMeanHighs']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()).round(2)
    df['ResidualsHighs'] = df['ResidualsHighs'].ffill()
    df['PrevStdHighs'] = np.sqrt(df['ResidualsHighs']/df['NumberOfHighs'])
    df['ResidualsHighs'] = df['ResidualsHighs'] + np.square(df['Highs'] - df['MeanHighs'])
    df['StdHighs'] = (np.sqrt(df['ResidualsHighs']/df['CurrentNoOfHighs'])).round(2)
    df['PrevZScoreHighs'] = ((df['PrevHighs']-df['PrevMeanHighs'])/df['PrevStdHighs']).round(2)
    df['ZScoreHighs'] = ((df['Highs']-df['MeanHighs'])/df['StdHighs']).round(2)
    df['SkewHighsNumerator'] = ((df['PrevHighs'] - df['PrevMeanHighs'])**3).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()
    df['SkewHighsNumerator'] = df['SkewHighsNumerator'].ffill()
    df['PrevSkewHighsVariance'] = df['SkewHighsNumerator']/(df['NumberOfHighs'])
    df['PrevSkewHighs'] = df['PrevSkewHighsVariance']/(df['PrevStdHighs']**3)
    df['SkewHighsVariance'] = (df['SkewHighsNumerator'] + (df['Highs'] - df['MeanHighs'])**3)/df['CurrentNoOfHighs']
    df['SkewHighs'] = (df['SkewHighsVariance']/(df['StdHighs']**3)).round(2)
    df['CVHighs'] = (df['StdHighs']/df['MeanHighs']).round(2)
    df['KurtosisHighsNumerator'] = ((df['PrevHighs'] - df['PrevMeanHighs'])**4).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()
    df['KurtosisHighsNumerator'] = df['KurtosisHighsNumerator'].ffill()
    df['PrevKurtosisHighsVariance'] = df['KurtosisHighsNumerator']/(df['NumberOfHighs'])
    df['PrevKurtosisHighs'] = df['PrevKurtosisHighsVariance']/(df['PrevStdHighs']**4)-3
    df['KurtosisHighsVariance'] = (df['KurtosisHighsNumerator'] + (df['Highs'] - df['MeanHighs'])**4)/df['CurrentNoOfHighs']
    df['KurtosisHighs'] = (df['KurtosisHighsVariance']/(df['StdHighs']**4)-3).round(2)
    df['Highs_10th_Percentile'] = df['Highs'].expanding().quantile(0.10)
    df['Highs_25th_Percentile'] = df['Highs'].expanding().quantile(0.25)
    df['Highs_50th_Percentile'] = df['Highs'].expanding().quantile(0.50)
    df['Highs_75th_Percentile'] = df['Highs'].expanding().quantile(0.75)
    df['Highs_90th_Percentile'] = df['Highs'].expanding().quantile(0.90)
    df['HighsIQR'] = (df['Highs_75th_Percentile'] - df['Highs_25th_Percentile']).round(2)




    print('Phase 4 Started')
    df['le_next_low_pred'] = 2 * df['Low'] - df['Low'].shift(1)
    df['le_next_high_pred'] = 2 * df['High'] - df['High'].shift(1) 
    df['atr'] = df['Range'].rolling(window=14, min_periods=1).mean()  
    df['atr_next_low_pred'] = df['Low'] - 0.5 * df['atr']
    df['atr_next_high_pred'] = df['High'] + 0.5 * df['atr'] 
    df['reg_next_low_pred'] = df['Low'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['reg_next_high_pred'] = df['High'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['ema_low'] = df['Low'].ewm(span=10, adjust=False).mean()
    df['ema_high'] = df['High'].ewm(span=10, adjust=False).mean()
    df['low_std'] = df['Low'].rolling(window=10, min_periods=1).std()
    df['high_std'] = df['High'].rolling(window=10, min_periods=1).std()
    df['ema_next_low_pred'] = df['ema_low'] - df['low_std']
    df['ema_next_high_pred'] = df['ema_high'] + df['high_std']
    #ensemble of em all
    df['predicted_next_low'] = (df['le_next_low_pred'] + df['atr_next_low_pred'] + df['reg_next_low_pred']) / 3
    df['predicted_next_high'] = (df['le_next_high_pred'] + df['atr_next_high_pred'] + df['reg_next_high_pred']) / 3
    df['feature_imbalance'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    df['realtive_spread'] = df['Spread'] / ((df['BestBid'] + df['BestAsk']) / 2)
    print("ensemble done")
    #rapdi changed in bid ask prices - quote stuffing
    df['quote_stuffing'] = df['BestBid'].diff().abs() + df['BestAsk'].diff().abs()
    #liq based    
    df['depth_slope'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    # Hidden Liquidity Detection (large orders appearing and disappearing)
    print('depth slope done')
    df['hidden_liquidity'] = ((df['BidSize'] > df['BidSize'].shift(1) * 2) & (df['BidSize'].shift(-1) < df['BidSize'] / 2)).astype(int)
    ## 4. Price Action & Trend Features
    df['price_momentum'] = df['Close'].diff()
    df['price_momentum_ratio'] = df['Close'] / df['Close'].shift(1)
    print('prr done')
    #df['hurst_exponent'] = df['Close'].rolling(120).apply(lambda x: compute_Hc(x[:-1], kind='price', simplified=True)[0] if len(x) >= 100 else np.nan,raw=True).shift(1)

    # Hilbert Transform (Measures Market Cycles)
    df['hilbert_phase'] = df['Close'].rolling(50).apply(rolling_hilbert, raw=True).shift(1)
    print('hilbert phase done')
    # Fraction of Large Orders (Relative to Volume)
    df['large_order_fraction'] = (df['BidSize'] + df['AskSize']) / (df['Volume'] + 1e-9)
    print('lof done')
    #micorstucture volatlity
    df['mvr'] = df['Close'].pct_change().rolling(5).std().shift(1) / df['Close'].pct_change().rolling(30).std().shift(1)

                                                         


    df = df.drop(columns = ['PrevRange', 'GroupNumTime', 'PrevMeanRange', 'PrevStdRange', 'PrevZScoreRange', 'PrevSkewRange', 'PrevKurtosisVariance', 
                            'PrevKurtosisRange', 'PrevMeanLows', 'PrevLows', 'PrevMeanHighs', 'PrevLows', 'PrevHighs', 'PrevStdLows', 
                            'PrevStdHighs', 'PrevZScoreLows', 'PrevZScoreHighs', 'PrevSkewLows', 'PrevKurtosisLows', 'PrevZScoreHighs',
                            'PrevSkewHighs', 'PrevKurtosisHighs', 'MinRange', 'MaxRange'])
    df = df.drop(columns = ['Group', 'Session', 'SessionBar', 'PrevBar', 'PrevPrevBar', 'PrevOpen', 'PrevHigh', 'PrevLow', 'PrevClose', 
                            'ShiftedRange', 'PrevPrevRange', 'SumPrevRange', 'ResidualsRange', 'SkewNumerator', 'PrevSkewVariance', 
                            'SkewVariance', 'KurtosisNumerator', 'KurtosisVariance', 'PrevAutoCorRangeNumerator', 'Range_10th_Percentile', 
                            'Range_25th_Percentile', 'Range_50th_Percentile', 'Range_75th_Percentile', 'Range_90th_Percentile', 'RangeOutlierSum2',
                            'PrevAutoCorRangeDenominator', 'SumPrevLows', 'SumPrevHighs', 'NumberOfLows', 'NumberOfHighs', 'CurrentNoOfLows', 
                            'CurrentNoOfHighs', 'ResidualsLows', 'Lows_10th_Percentile', 'Lows_25th_Percentile', 'Lows_50th_Percentile',
                            'Lows_75th_Percentile', 'Lows_90th_Percentile', 'KurtosisLowsVariance', 'PrevKurtosisLowsVariance', 'RangeOutlierSum3',
                            'KurtosisLowsNumerator', 'SkewLowsVariance', 'PrevSkewLowsVariance', 'SkewLowsNumerator', 'ResidualsHighs', 
                            'SkewHighsNumerator', 'PrevSkewHighsVariance', 'SkewHighsVariance', 'KurtosisHighsNumerator', 'RangeOutlierSum1',
                            'PrevKurtosisHighsVariance', 'KurtosisHighsVariance', 'Highs_10th_Percentile', 'Highs_25th_Percentile', 
                            'Highs_50th_Percentile', 'Highs_75th_Percentile', 'Highs_90th_Percentile', 'TimeSumPrevRange','TimeSumGroupNumTime',
                            'RangeOutlierImpact2', 'RangeOutlierImpact3', 'SumIntervalMeanRange', 'SkewLTQVariance', 'SumPrevLTQ',
                            'PrevLTQ', 'PrevMeanLTQ', 'PrevAutoCorLTQDenominator', 'PrevAutoCorLTQNumerator', 'PrevPrevLTQ', 'LTQ_10th_Percentile',
                            'LTQ_25th_Percentile', 'LTQ_50th_Percentile', 'LTQ_75th_Percentile', 'LTQ_90th_Percentile', 'DeviationFlag', 
                            'SumAbsoluteDeviationLTQ', 'AbsoluteDeviationLTQ', 'SumPrevVolume', 'ResidualsVolume', 'PrevStdVolume', 'PrevMeanVolume',
                            'PrevZScoreVolume', 'PrevSkewVolume', 'PrevKurtosisVolume', 'PrevAutoCorVolumeNumerator', 'PrevAutoCorVolumeDenominator', 
                            'TimeSumPrevVolume', 'TimeSumGroupNumTime', 'VolumeOutlierSum1', 'VolumeOutlierSum2', 'VolumeOutlierSum3', 'ForwardVolume', 
                            'SumIntervalMeanVolume', 'PrevPrevVolume'])
    df = df.drop(columns=[
                            'ForwardRange', 'ProportionVolume', 'ProportionRange', 'SpeedVolume', 'SpeedRange', 
                            'MeanLTQ', 'StdLTQ', 'MeanVolume', 'StdVolume', 'MeanRange', 'StdRange', 
                            'Volume_10th_Percentile', 'Volume_25th_Percentile', 'Volume_50th_Percentile', 
                            'Volume_75th_Percentile', 'Volume_90th_Percentile', 'TimeMeanVolume', 
                            'IntervalMeanVolume', 'TimeMeanRange', 'IntervalMeanRange', 'RangeOutlierImpact1'
                    ])

    return df

import numpy as np
from numba import njit

@njit
def std_numba(arr):
    n = arr.shape[0]
    s = 0.0
    mean = 0.0
    for i in range(n):
        mean += arr[i]
    mean /= n
    for i in range(n):
        diff = arr[i] - mean
        s += diff * diff
    return np.sqrt(s / n)

@njit
def hurst_exponent_numba(ts, max_lag=100):
    n = ts.shape[0]
    if n < max_lag:
        return np.nan
    m = max_lag - 2
    lags = np.empty(m)
    tau = np.empty(m)
    for i in range(m):
        lag = i + 2
        lags[i] = lag
        diff_sum = 0.0
        count = n - lag
        s = 0.0
        for j in range(count):
            s += ts[j + lag] - ts[j]
        mean_diff = s / count
        var = 0.0
        for j in range(count):
            d = (ts[j + lag] - ts[j]) - mean_diff
            var += d * d
        tau[i] = np.sqrt(var / count)
    sum_log_lags = 0.0
    sum_log_tau = 0.0
    for i in range(m):
        sum_log_lags += np.log(lags[i])
        sum_log_tau += np.log(tau[i])
    mean_log_lags = sum_log_lags / m
    mean_log_tau = sum_log_tau / m
    cov = 0.0
    var_ll = 0.0
    for i in range(m):
        diff_ll = np.log(lags[i]) - mean_log_lags
        diff_tau = np.log(tau[i]) - mean_log_tau
        cov += diff_ll * diff_tau
        var_ll += diff_ll * diff_ll
    slope = cov / var_ll
    return slope
    
@njit
def fractal_dimension_numba(ts, max_scale=20):
    n = ts.shape[0]
    if n < max_scale:
        return np.nan
    m = max_scale - 2
    scales = np.empty(m)
    variances = np.empty(m)
    for i in range(m):
        scale = i + 2
        scales[i] = scale
        npart = n // scale
        var_sum = 0.0
        for j in range(npart):
            seg_mean = 0.0
            for k in range(scale):
                seg_mean += ts[j * scale + k]
            seg_mean /= scale
            seg_var = 0.0
            for k in range(scale):
                diff = ts[j * scale + k] - seg_mean
                seg_var += diff * diff
            seg_std = np.sqrt(seg_var / scale)
            var_sum += seg_std
        variances[i] = var_sum / npart
    sum_log_scales = 0.0
    sum_log_vars = 0.0
    for i in range(m):
        sum_log_scales += np.log(scales[i])
        sum_log_vars += np.log(variances[i])
    mean_log_scales = sum_log_scales / m
    mean_log_vars = sum_log_vars / m
    cov = 0.0
    var_ll = 0.0
    for i in range(m):
        diff_sc = np.log(scales[i]) - mean_log_scales
        diff_v = np.log(variances[i]) - mean_log_vars
        cov += diff_sc * diff_v
        var_ll += diff_sc * diff_sc
    slope = cov / var_ll
    return 2 - slope

@njit
def lyapunov_exponent_numba(time_series, epsilon=1e-4, steps=100):
    N = time_series.shape[0]
    if N <= steps:
        return np.nan
    s = 0.0
    count = N - steps
    base = 0.0
    diff0 = np.abs(time_series[steps] - time_series[0])
    if diff0 < epsilon:
        diff0 = epsilon
    for i in range(count):
        diff = np.abs(time_series[i + steps] - time_series[i])
        if diff < epsilon:
            diff = epsilon
        s += np.log(diff / diff0)
    return s / (count * steps)



In [29]:
import pandas as pd
import gc


df_start['Date'] = pd.to_datetime(df_start['Date'], dayfirst=True, errors='coerce')
df_start['Date'] = df_start['Date'].dt.floor('D')

df_start['Time'] = pd.to_datetime(
    df_start['Time'],
    format="%H:%M:%S.%f",
    errors='coerce'
).fillna(pd.to_datetime(df_start['Time'], format="%H:%M:%S", errors='coerce')).dt.time

unique_dates = sorted(df_start['Date'].unique())
window_size = 15

start_time = pd.to_datetime("09:15:00").time()
end_time = pd.to_datetime("15:30:00").time()

for i in range(len(unique_dates) - window_size + 1):    
    start_date = unique_dates[i]
    print('starting backtester data creation for ,', start_date)
    end_date = unique_dates[i + window_size - 1]

    df_current = df_start[
        (df_start['Date'] >= start_date) &
        (df_start['Date'] <= end_date) &
        (df_start['Time'] >= start_time) &
        (df_start['Time'] < end_time)
    ].copy()
    
    Nifty_FUT_resampled = resampled_ohlc(df_current)
    df = Nifty_FUT_resampled['15s'][0]
    del Nifty_FUT_resampled
    del df_current
    gc.collect()

    #aggregation

    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    df.set_index('DateTime', inplace = True)

    

    #resampling

    resampled_df = df.resample('15s').agg({      
                    'Date': 'last',         
                    'Time': 'last',        
                    'Open': 'first',
                    'High':'max',
                    'Low':'min',
                    'Close':'last',
                    'BestBid':'last',
                    'BestAsk':'last',
                    'BidSize':'last',
                    'AskSize':'last',
                    'Volume':'sum',
                    'Group':'last',
                    'GroupNumTime':'last',
                    'TimePassed':'last', 
                    'GroupNum':'last', 
                    'PrevBar':'last',
                    'PrevPrevBar':'last', 
                    'Session':'last', 
                    'SessionBar':'last'	
        }).reset_index()
    resampled_df.drop(columns = 'DateTime', inplace = True , axis =1, errors = 'ignore')

    print('Agg Done')
    
    df = resampled_df.copy()
    gc.collect()

    #adding variables
    
    df_after_adding_variables = generate_basic_variables(df)

    print('Variables Added')

    #adding more

    df_after_adding_variables['Date_Time'] = pd.to_datetime(df_after_adding_variables['Date'] + ' ' + df_after_adding_variables['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    df_after_adding_variables.set_index('Date_Time', inplace=True)
    df_after_adding_variables['Close'] = pd.to_numeric(df_after_adding_variables['Close'], errors='coerce')
    df_after_adding_variables.dropna(subset=['Close'], inplace=True)
    
    window_sizes = [200, 500, 1000, 1500]
    
    for window in window_sizes:
        df_after_adding_variables[f'Lyapunov_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
            lambda x: lyapunov_exponent_numba(x), raw=True)
        df_after_adding_variables[f'Hurst_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
            lambda x: hurst_exponent_numba(x), raw=True)
        df_after_adding_variables[f'FDI_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
            lambda x: fractal_dimension_numba(x), raw=True)
    
    df_after_adding_variables.reset_index(drop=True, inplace=True)
    df_after_adding_variables[['Lyapunov_200','Hurst_200','FDI_200','Lyapunov_500','Hurst_500','FDI_500','Lyapunov_1000','Hurst_1000','FDI_1000','Lyapunov_1500','Hurst_1500','FDI_1500']].tail()

    print(f'New Training data for {start_date} saved to temp var')
    print('Starting Training')

    df = df_after_adding_variables.copy()
    del df_after_adding_variables
    gc.collect()

    #untuned xgboost model with old weight, for day to day rolling 15 test
    '''
    df['Pct_Change'] = df['Close'].pct_change() * 100

    df['Target'] = df['Pct_Change'].shift(-1)
    
    
    df = df.dropna(subset=['Pct_Change', 'Target'])
    
    feature_columns = ['Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk',
                       'BidSize', 'AskSize', 'Volume', 'TimePassed', 'GroupNum', 'PrevVolume',
                       'SecondInterval', 'Spread', 'SpreadSize', 'LTQ', 'ZScoreLTQ', 'SkewLTQ',
                       'AutoCorrelationLTQ', 'MeanAbsoluteDeviationLTQ', 'ZScoreVolume',
                       'SkewVolume', 'CVVolume', 'KurtosisVolume', 'AutoCorrelationVolume',
                       'VolumeOutlierImpact1', 'VolumeOutlierImpact2', 'VolumeOutlierImpact3', 'Range',
                       'ZScoreRange', 'SkewRange', 'CVRange', 'KurtosisRange',
                       'AutoCorrelationRange', 'NormalizedRange', 'Lows', 'LastLows', 'MeanLows',
                       'StdLows', 'ZScoreLows', 'SkewLows', 'CVLows', 'KurtosisLows',
                       'Highs', 'LastHighs', 'MeanHighs', 'StdHighs', 'ZScoreHighs', 'SkewHighs',
                       'CVHighs', 'KurtosisHighs', 'le_next_low_pred', 'le_next_high_pred',
                       'atr', 'atr_next_low_pred', 'atr_next_high_pred', 'reg_next_low_pred',
                       'reg_next_high_pred', 'ema_low', 'ema_high', 'low_std', 'high_std',
                       'ema_next_low_pred', 'ema_next_high_pred', 'predicted_next_low',
                       'predicted_next_high', 'feature_imbalance', 'realtive_spread', 'quote_stuffing',
                       'depth_slope', 'price_momentum', 'price_momentum_ratio',
                       'hilbert_phase', 'large_order_fraction', 'mvr']
    
    X = df[feature_columns]
    y = df['Target']
    
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    
    
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.median())
    X_test  = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.median())
    
    # Standardize the features.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    import joblib
    
    # Save the fitted scaler
    joblib.dump(scaler, f"{start_date}-{end_date}-scaler_untuned.pkl")
    
    
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators= 142,
        max_depth= 6,
        learning_rate= 0.017409934489017977,
        subsample= 0.8469721090951134,
        colsample_bytree= 0.7404203262606515,
        random_state=42
    )
    
    model.fit(X_train_scaled, y_train)
    
    joblib.dump(model, f"{start_date}-{end_date}-model-untuned.pkl") 
    
    y_pred = model.predict(X_test_scaled)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f'For {start_date}')
    print("Test R^2 Score:", r2)
    print("Test RMSE:", rmse)

    '''
    # -Saving weight with tuner
    df['Pct_Change'] = df['Close'].pct_change() * 100
    df['Target'] = df['Pct_Change'].shift(-1)
    df = df.dropna(subset=['Pct_Change', 'Target'])
    
    feature_columns = ['Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk',
                       'BidSize', 'AskSize', 'Volume', 'TimePassed', 'GroupNum', 'PrevVolume',
                       'SecondInterval', 'Spread', 'SpreadSize', 'LTQ', 'ZScoreLTQ', 'SkewLTQ',
                       'AutoCorrelationLTQ', 'MeanAbsoluteDeviationLTQ', 'ZScoreVolume',
                       'SkewVolume', 'CVVolume', 'KurtosisVolume', 'AutoCorrelationVolume',
                       'VolumeOutlierImpact1', 'VolumeOutlierImpact2', 'VolumeOutlierImpact3', 'Range',
                       'ZScoreRange', 'SkewRange', 'CVRange', 'KurtosisRange',
                       'AutoCorrelationRange', 'NormalizedRange', 'Lows', 'LastLows', 'MeanLows',
                       'StdLows', 'ZScoreLows', 'SkewLows', 'CVLows', 'KurtosisLows',
                       'Highs', 'LastHighs', 'MeanHighs', 'StdHighs', 'ZScoreHighs', 'SkewHighs',
                       'CVHighs', 'KurtosisHighs', 'le_next_low_pred', 'le_next_high_pred',
                       'atr', 'atr_next_low_pred', 'atr_next_high_pred', 'reg_next_low_pred',
                       'reg_next_high_pred', 'ema_low', 'ema_high', 'low_std', 'high_std',
                       'ema_next_low_pred', 'ema_next_high_pred', 'predicted_next_low',
                       'predicted_next_high', 'feature_imbalance', 'realtive_spread', 'quote_stuffing',
                       'depth_slope', 'price_momentum', 'price_momentum_ratio',
                       'hilbert_phase', 'large_order_fraction', 'mvr']
    
    X = df[feature_columns]
    y = df['Target']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    
    # Clean and scale data
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.median())
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.median())
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    
    joblib.dump(scaler, f"{start_date}-{end_date}-scaler_tuned.pkl")
    
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 4, 16),
            "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.15),
            "subsample": trial.suggest_float("subsample", 0.7, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 0.8),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
            "alpha": trial.suggest_float("alpha", 0, 1),
            "objective": "reg:squarederror",
            "random_state": 42,
            "n_jobs": -1,
        }
    
        model = xgb.XGBRegressor(**params)
        tscv = TimeSeriesSplit(n_splits=5)
        scores = []
    
        for train_idx, valid_idx in tscv.split(X_train_scaled):
            X_t, X_v = X_train_scaled[train_idx], X_train_scaled[valid_idx]
            y_t, y_v = y_train.values[train_idx], y_train.values[valid_idx]
    
            model.fit(X_t, y_t,
                      eval_set=[(X_v, y_v)],
                      verbose=False)
            
            y_pred = model.predict(X_v)
            rmse = mean_squared_error(y_v, y_pred)
            scores.append(rmse)
    
        return np.mean(scores)
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)
    
    print("Best parameters:", study.best_trial.params)
    
    best_params = study.best_trial.params
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_train_scaled, y_train)
    
    joblib.dump(model, f"{start_date}-{end_date}-model-tuned.pkl")
    
    # ---- 5. Evaluate on Test Set ----
    y_pred = final_model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print("Test R² Score:", r2)
    print("Test RMSE:", rmse)
    



starting backtester data creation for , 2025-09-01 00:00:00


KeyError: 'BuyPrice'

In [30]:
df_start = pd.read_csv('master_fut_df.csv')

In [34]:
import pandas as pd
import numpy as np
import gc
import warnings
import time
from datetime import datetime as dt
from scipy.signal import hilbert
from numba import njit
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import joblib
import optuna
import os

# Use 'pip install hurst' if needed
try:
    from hurst import compute_Hc
except ImportError:
    pass 

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

# ==========================================
# 1. NUMBA OPTIMIZED FUNCTIONS
# ==========================================

@njit
def std_numba(arr):
    n = arr.shape[0]
    s = 0.0
    mean = 0.0
    for i in range(n):
        mean += arr[i]
    mean /= n
    for i in range(n):
        diff = arr[i] - mean
        s += diff * diff
    return np.sqrt(s / n)

@njit
def hurst_exponent_numba(ts, max_lag=100):
    n = ts.shape[0]
    if n < max_lag:
        return np.nan
    m = max_lag - 2
    lags = np.empty(m)
    tau = np.empty(m)
    for i in range(m):
        lag = i + 2
        lags[i] = lag
        diff_sum = 0.0
        count = n - lag
        s = 0.0
        for j in range(count):
            s += ts[j + lag] - ts[j]
        mean_diff = s / count
        var = 0.0
        for j in range(count):
            d = (ts[j + lag] - ts[j]) - mean_diff
            var += d * d
        tau[i] = np.sqrt(var / count)
    sum_log_lags = 0.0
    sum_log_tau = 0.0
    for i in range(m):
        sum_log_lags += np.log(lags[i])
        sum_log_tau += np.log(tau[i])
    mean_log_lags = sum_log_lags / m
    mean_log_tau = sum_log_tau / m
    cov = 0.0
    var_ll = 0.0
    for i in range(m):
        diff_ll = np.log(lags[i]) - mean_log_lags
        diff_tau = np.log(tau[i]) - mean_log_tau
        cov += diff_ll * diff_tau
        var_ll += diff_ll * diff_ll
    slope = cov / var_ll
    return slope
    
@njit
def fractal_dimension_numba(ts, max_scale=20):
    n = ts.shape[0]
    if n < max_scale:
        return np.nan
    m = max_scale - 2
    scales = np.empty(m)
    variances = np.empty(m)
    for i in range(m):
        scale = i + 2
        scales[i] = scale
        npart = n // scale
        var_sum = 0.0
        for j in range(npart):
            seg_mean = 0.0
            for k in range(scale):
                seg_mean += ts[j * scale + k]
            seg_mean /= scale
            seg_var = 0.0
            for k in range(scale):
                diff = ts[j * scale + k] - seg_mean
                seg_var += diff * diff
            seg_std = np.sqrt(seg_var / scale)
            var_sum += seg_std
        variances[i] = var_sum / npart
    sum_log_scales = 0.0
    sum_log_vars = 0.0
    for i in range(m):
        sum_log_scales += np.log(scales[i])
        sum_log_vars += np.log(variances[i])
    mean_log_scales = sum_log_scales / m
    mean_log_vars = sum_log_vars / m
    cov = 0.0
    var_ll = 0.0
    for i in range(m):
        diff_sc = np.log(scales[i]) - mean_log_scales
        diff_v = np.log(variances[i]) - mean_log_vars
        cov += diff_sc * diff_v
        var_ll += diff_sc * diff_sc
    slope = cov / var_ll
    return 2 - slope

@njit
def lyapunov_exponent_numba(time_series, epsilon=1e-4, steps=100):
    N = time_series.shape[0]
    if N <= steps:
        return np.nan
    s = 0.0
    count = N - steps
    diff0 = np.abs(time_series[steps] - time_series[0])
    if diff0 < epsilon:
        diff0 = epsilon
    for i in range(count):
        diff = np.abs(time_series[i + steps] - time_series[i])
        if diff < epsilon:
            diff = epsilon
        s += np.log(diff / diff0)
    return s / (count * steps)

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def clean_time_format(time_str):
    if '.' not in time_str:
        return f"{time_str}.000000"
    else:
        time_parts = time_str.split('.')
        milliseconds = time_parts[1][:6].ljust(6, '0')
        return f"{time_parts[0]}.{milliseconds}"  

def additional_columns(df):
    df = df.copy()
    df['Session'] = np.where((df['Date'] != df['Date'].shift(1)) & (df['Time'] != pd.to_datetime('00:00:00').time()), 0, np.nan)
    df['Session'] = np.where((df['Session'].shift(-1) == 0), 1, df['Session'])  
    df.at[df.index[-1], 'Session'] = 1
    return df

def rolling_hilbert(series):
    if len(series) < 10:
        return np.nan
    return np.angle(hilbert(series)[-1])
    
def forecast_next(x):
        if len(x) < 2:
            return x[-1]
        X = np.arange(len(x))
        coef = np.polyfit(X, x, 1)   
        return np.polyval(coef, len(x))  

# ==========================================
# 3. FEATURE GENERATION (The Large Function)
# ==========================================

def generate_basic_variables(df):
    df = df.copy()

    open_dict, high_dict, low_dict, close_dict, volume_dict = df['Open'].to_dict(), df['High'].to_dict(), df['Low'].to_dict(), df['Close'].to_dict(), df['Volume'].to_dict()
    df['PrevOpen'] = df['PrevBar'].map(lambda x: open_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevHigh'] = df['PrevBar'].map(lambda x: high_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevLow'] = df['PrevBar'].map(lambda x: low_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevClose'] = df['PrevBar'].map(lambda x: close_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevVolume'] = df['PrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SecondInterval'] = np.floor(df['TimePassed']).where(df['TimePassed'] < 15, np.nan) + 1

    df['Spread'] = (df['BestAsk'] - df['BestBid']).round(2)
    df['SpreadSize'] = (df['AskSize'] - df['BidSize']).round(2)

    df['LTQ'] = np.where(df['GroupNum'] == df['GroupNum'].shift(1), df['Volume'] - df['Volume'].shift(1), df['Volume'])
    LTQ_dict = df['LTQ'].to_dict()
    df['PrevLTQ'] = df['PrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevLTQ'] = df['PrevPrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['MeanLTQ'] = df['LTQ'].expanding().mean().round(2)
    df['StdLTQ'] = df['LTQ'].expanding().std().round(2)
    df['SumPrevLTQ'] = df['PrevLTQ'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLTQ'] = df['SumPrevLTQ'].ffill()
    df['PrevMeanLTQ'] = (df['SumPrevLTQ']/(df['GroupNum']-1)).round(2)
    df['ZScoreLTQ'] = ((df['LTQ'] - df['MeanLTQ'])/df['StdLTQ']).round(2)
    df['SkewLTQVariance'] = ((df['LTQ'] - df['MeanLTQ'])**3).cumsum()
    df['SkewLTQ'] = (df['SkewLTQVariance']/((df['StdLTQ']**3)*(df.index))).round(2)
    df['LTQ_10th_Percentile'] = df['LTQ'].expanding().quantile(0.10)
    df['LTQ_25th_Percentile'] = df['LTQ'].expanding().quantile(0.25)
    df['LTQ_50th_Percentile'] = df['LTQ'].expanding().quantile(0.50)
    df['LTQ_75th_Percentile'] = df['LTQ'].expanding().quantile(0.75)
    df['LTQ_90th_Percentile'] = df['LTQ'].expanding().quantile(0.90)
    df['LTQIQR'] = (df['LTQ_75th_Percentile'] - df['LTQ_25th_Percentile']).round(2)
    df['PrevAutoCorLTQNumerator'] = ((df['LTQ'] - df['MeanLTQ']) * (df['LTQ'].shift(1) - df['MeanLTQ'])).cumsum()
    df['PrevAutoCorLTQDenominator'] = np.square(df['LTQ']- df['MeanLTQ']).cumsum()
    df['AutoCorrelationLTQ'] = (df['PrevAutoCorLTQNumerator']/df['PrevAutoCorLTQDenominator']).round(2)
    df['DeviationFlag'] = np.where(df['LTQ'] > (df['MeanLTQ'] + df['StdLTQ']), 1, 0)
    df['AbsoluteDeviationLTQ'] = df['DeviationFlag'].cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['LTQ'].where(df['AbsoluteDeviationLTQ'] != df['AbsoluteDeviationLTQ'].shift(1)).cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['SumAbsoluteDeviationLTQ'].ffill()
    df['MeanAbsoluteDeviationLTQ'] = np.where(df['AbsoluteDeviationLTQ'] != 0, (df['SumAbsoluteDeviationLTQ']/df['AbsoluteDeviationLTQ']).round(2), np.nan)
    
    volume_dict = df['Volume'].to_dict()
    df['PrevPrevVolume'] = df['PrevPrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevVolume'] = df['PrevVolume'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevVolume'] = df['SumPrevVolume'].ffill()
    df['PrevMeanVolume'] = (df['SumPrevVolume']/(df['GroupNum']-1)).round(2)
    df['MeanVolume'] = ((df['SumPrevVolume'] + df['Volume'])/df['GroupNum']).round(2)
    df['ResidualsVolume'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsVolume'] = df['ResidualsVolume'].ffill()
    df['PrevStdVolume'] = np.sqrt(df['ResidualsVolume']/df['GroupNum']-1)
    df['ResidualsVolume'] = df['ResidualsVolume'] + np.square(df['Volume'] - df['MeanVolume'])
    df['StdVolume'] = (np.sqrt(df['ResidualsVolume']/df['GroupNum'])).round(2)
    df['PrevZScoreVolume'] = ((df['PrevVolume']-df['PrevMeanVolume'])/df['PrevStdVolume']).round(2)
    df['ZScoreVolume'] = ((df['Volume']-df['MeanVolume'])/df['StdVolume']).round(2)
    df['SkewNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewVolume'] = df['PrevSkewVariance']/(df['PrevStdVolume']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Volume'] - df['MeanVolume'])**3)/df['GroupNum']
    df['SkewVolume'] = (df['SkewVariance']/(df['StdVolume']**3)).round(2)
    df['CVVolume'] = (df['StdVolume']/df['MeanVolume']).round(2)
    df['KurtosisNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisVolume'] = df['PrevKurtosisVariance']/(df['PrevStdVolume']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Volume'] - df['MeanVolume'])**4)/df['GroupNum']
    df['KurtosisVolume'] = (df['KurtosisVariance']/(df['StdVolume']**4)-3).round(2)
    df['Volume_10th_Percentile'] = df['Volume'].expanding().quantile(0.10)
    df['Volume_25th_Percentile'] = df['Volume'].expanding().quantile(0.25)
    df['Volume_50th_Percentile'] = df['Volume'].expanding().quantile(0.50)
    df['Volume_75th_Percentile'] = df['Volume'].expanding().quantile(0.75)
    df['Volume_90th_Percentile'] = df['Volume'].expanding().quantile(0.90)
    df['VolumeIQR'] = (df['Volume_75th_Percentile'] - df['Volume_25th_Percentile']).round(2)
    df['PrevAutoCorVolumeNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume']) * (df['PrevPrevVolume'] - df['PrevMeanVolume'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorVolumeDenominator'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationVolume'] = (df['PrevAutoCorVolumeNumerator']/df['PrevAutoCorVolumeDenominator']).round(2)
    df['AutoCorrelationVolume'] = df['AutoCorrelationVolume'].ffill()
    mask = df['GroupNum'] != df['GroupNum'].shift(1)
    df['TimeSumPrevVolume'] = (df.groupby('GroupNumTime')['PrevVolume'].apply(lambda group: group.where(mask).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevVolume'] = df['TimeSumPrevVolume'].ffill()
    df['TimeSumGroupNumTime'] = (df.groupby('GroupNumTime').apply(lambda group: group['GroupNum'].ne(group['GroupNum'].shift()).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevVolume'] = np.where(df['GroupNumTime'] == 1, 0, df['TimeSumPrevVolume'])
    df['TimeMeanVolume'] = (df['TimeSumPrevVolume']/df['TimeSumGroupNumTime']).round(2)
    df['VolumeOutlierSum1'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 1)).cumsum()
    df['VolumeOutlierSum1'] = df['VolumeOutlierSum1'].ffill()
    df['VolumeOutlierSum2'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 2)).cumsum()
    df['VolumeOutlierSum2'] = df['VolumeOutlierSum2'].ffill()
    df['VolumeOutlierSum3'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 3)).cumsum()
    df['VolumeOutlierSum3'] = df['VolumeOutlierSum3'].ffill()
    df['VolumeOutlierImpact1'] = (df['VolumeOutlierSum1']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact2'] = (df['VolumeOutlierSum2']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact3'] = (df['VolumeOutlierSum3']/df['SumPrevVolume']).round(2)
    df['ForwardVolume'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Volume'].shift(1), np.nan)
    df['ForwardVolume'] = df['ForwardVolume'].shift(-1)
    df['ForwardVolume'] = df['ForwardVolume'].bfill()
    df['ProportionVolume'] = (df['Volume']/df['ForwardVolume']).round(2)
    df['SpeedVolume'] = ((df['Volume']*15)/df['TimePassed']).round(2)
    df['SumIntervalMeanVolume'] = (df.groupby('SecondInterval')['ProportionVolume'].apply(lambda group: group.where(df['SecondInterval'] != df['SecondInterval'].shift(1)).cumsum()).reset_index(level=0, drop=True))
    df['SumIntervalMeanVolume'] = df['SumIntervalMeanVolume'].ffill()
    df['IntervalMeanVolume'] = (df['SumIntervalMeanVolume']/df['GroupNum']).round(2)

    df['Range'] = df['High'] - df['Low']
    df['ShiftedRange'] = df['Range'].shift(1).where(df['GroupNum'] != df['GroupNum'].shift(1))
    df['MinRange'] = df['ShiftedRange'].cummin()
    df['MinRange'] = df['MinRange'].ffill()
    range_dict = df['Range'].to_dict()
    df['PrevRange'] = df['PrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevRange'] = df['PrevPrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevRange'] = df['PrevRange'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevRange'] = df['SumPrevRange'].ffill()
    df['PrevMeanRange'] = (df['SumPrevRange']/(df['GroupNum']-1)).round(2)
    df['MeanRange'] = ((df['SumPrevRange'] + df['Range'])/df['GroupNum']).round(2)
    df['ResidualsRange'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsRange'] = df['ResidualsRange'].ffill()
    df['PrevStdRange'] = np.sqrt(df['ResidualsRange']/df['GroupNum']-1)
    df['ResidualsRange'] = df['ResidualsRange'] + np.square(df['Range'] - df['MeanRange'])
    df['StdRange'] = (np.sqrt(df['ResidualsRange']/df['GroupNum'])).round(2)
    df['PrevZScoreRange'] = ((df['PrevRange']-df['PrevMeanRange'])/df['PrevStdRange']).round(2)
    df['ZScoreRange'] = ((df['Range']-df['MeanRange'])/df['StdRange']).round(2)
    df['MaxRange'] = df['Range'].where(df['ZScoreRange'] <= 3).cummax()
    df['MaxRange'] = df['MaxRange'].ffill()
    df['SkewNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewRange'] = df['PrevSkewVariance']/(df['PrevStdRange']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Range'] - df['MeanRange'])**3)/df['GroupNum']
    df['SkewRange'] = (df['SkewVariance']/(df['StdRange']**3)).round(2)
    df['CVRange'] = (df['StdRange']/df['MeanRange']).round(2)
    df['KurtosisNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisRange'] = df['PrevKurtosisVariance']/(df['PrevStdRange']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Range'] - df['MeanRange'])**4)/df['GroupNum']
    df['KurtosisRange'] = (df['KurtosisVariance']/(df['StdRange']**4)-3).round(2)
    df['Range_10th_Percentile'] = df['Range'].expanding().quantile(0.10)
    df['Range_25th_Percentile'] = df['Range'].expanding().quantile(0.25)
    df['Range_50th_Percentile'] = df['Range'].expanding().quantile(0.50)
    df['Range_75th_Percentile'] = df['Range'].expanding().quantile(0.75)
    df['Range_90th_Percentile'] = df['Range'].expanding().quantile(0.90)
    df['RangeIQR'] = (df['Range_75th_Percentile'] - df['Range_25th_Percentile']).round(2)
    df['PrevAutoCorRangeNumerator'] = ((df['PrevRange'] - df['PrevMeanRange']) * (df['PrevPrevRange'] - df['PrevMeanRange'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorRangeDenominator'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationRange'] = (df['PrevAutoCorRangeNumerator']/df['PrevAutoCorRangeDenominator']).round(2)
    df['AutoCorrelationRange'] = df['AutoCorrelationRange'].ffill()
    df['NormalizedRange'] = np.minimum(((df['Range'] - df['MinRange'])/(df['MaxRange'] - df['MinRange'])).round(2), 1)
    mask = df['GroupNum'] != df['GroupNum'].shift(1)
    df['TimeSumPrevRange'] = (df.groupby('GroupNumTime')['PrevRange'].apply(lambda group: group.where(mask).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevRange'] = df['TimeSumPrevRange'].ffill()
    df['TimeSumGroupNumTime'] = (df.groupby('GroupNumTime').apply(lambda group: group['GroupNum'].ne(group['GroupNum'].shift()).cumsum()).reset_index(level=0, drop=True))
    df['TimeSumPrevRange'] = np.where(df['GroupNumTime'] == 1, 0, df['TimeSumPrevRange'])
    df['TimeMeanRange'] = (df['TimeSumPrevRange']/df['TimeSumGroupNumTime']).round(2)
    df['RangeOutlierSum1'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 1)).cumsum()
    df['RangeOutlierSum1'] = df['RangeOutlierSum1'].ffill()
    df['RangeOutlierSum2'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 2)).cumsum()
    df['RangeOutlierSum2'] = df['RangeOutlierSum2'].ffill()
    df['RangeOutlierSum3'] = df['PrevRange'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreRange'].shift(1) > 3)).cumsum()
    df['RangeOutlierSum3'] = df['RangeOutlierSum3'].ffill()
    df['RangeOutlierImpact1'] = (df['RangeOutlierSum1']/df['SumPrevRange']).round(2)
    df['RangeOutlierImpact2'] = (df['RangeOutlierSum2']/df['SumPrevRange']).round(2)
    df['RangeOutlierImpact3'] = (df['RangeOutlierSum3']/df['SumPrevRange']).round(2)
    df['ForwardRange'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Range'].shift(1), np.nan)
    df['ForwardRange'] = df['ForwardRange'].shift(-1)
    df['ForwardRange'] = df['ForwardRange'].bfill()
    df['ProportionRange'] = (df['Range']/df['ForwardRange']).round(2)
    df['SpeedRange'] = ((df['Range']*15)/df['TimePassed']).round(2)
    df['SumIntervalMeanRange'] = (df.groupby('SecondInterval')['ProportionRange'].apply(lambda group: group.where(df['SecondInterval'] != df['SecondInterval'].shift(1)).cumsum()).reset_index(level=0, drop=True))
    df['SumIntervalMeanRange'] = df['SumIntervalMeanRange'].ffill()
    df['IntervalMeanRange'] = (df['SumIntervalMeanRange']/df['GroupNum']).round(2)

    df['Lows'] = (df['Low'] - df['PrevLow']).round(2)
    df['Lows'] = np.where(df['SessionBar'] == True, 0, df['Lows'])
    df['Lows'] = np.where(df['Lows'] < 0, 0, df['Lows'])
    df['LastLows'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Lows'].shift(1), np.nan)
    df['LastLows'] = df['LastLows'].shift(1)
    df['LastLows'] = df['LastLows'].ffill()
    df['LastLows'] = np.where(df['LastLows'] < 0, 0, df['LastLows'])
    lows_dict = df['Lows'].to_dict() 
    df['PrevLows'] = df['PrevBar'].map(lambda x: lows_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfLows'] = ((df['PrevLows'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfLows'] = np.where(df['Lows'] > 0, df['NumberOfLows']+1, df['NumberOfLows'])
    df['SumPrevLows'] = df['PrevLows'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLows'] = df['SumPrevLows'].ffill()
    df['PrevMeanLows'] = (df['SumPrevLows']/df['NumberOfLows']).round(2)
    df['MeanLows'] = ((df['SumPrevLows'] + df['Lows'])/df['CurrentNoOfLows']).round(2)
    df['ResidualsLows'] = (np.square(df['PrevLows'] - df['PrevMeanLows']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()).round(2)
    df['ResidualsLows'] = df['ResidualsLows'].ffill()
    df['PrevStdLows'] = np.sqrt(df['ResidualsLows']/df['NumberOfLows'])
    df['ResidualsLows'] = df['ResidualsLows'] + np.square(df['Lows'] - df['MeanLows'])
    df['StdLows'] = (np.sqrt(df['ResidualsLows']/df['CurrentNoOfLows'])).round(2)
    df['PrevZScoreLows'] = ((df['PrevLows']-df['PrevMeanLows'])/df['PrevStdLows']).round(2)
    df['ZScoreLows'] = ((df['Lows']-df['MeanLows'])/df['StdLows']).round(2)
    df['SkewLowsNumerator'] = ((df['PrevLows'] - df['PrevMeanLows'])**3).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()
    df['SkewLowsNumerator'] = df['SkewLowsNumerator'].ffill()
    df['PrevSkewLowsVariance'] = df['SkewLowsNumerator']/(df['NumberOfLows'])
    df['PrevSkewLows'] = df['PrevSkewLowsVariance']/(df['PrevStdLows']**3)
    df['SkewLowsVariance'] = (df['SkewLowsNumerator'] + (df['Lows'] - df['MeanLows'])**3)/df['CurrentNoOfLows']
    df['SkewLows'] = (df['SkewLowsVariance']/(df['StdLows']**3)).round(2)
    df['CVLows'] = (df['StdLows']/df['MeanLows']).round(2)
    df['KurtosisLowsNumerator'] = ((df['PrevLows'] - df['PrevMeanLows'])**4).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()
    df['KurtosisLowsNumerator'] = df['KurtosisLowsNumerator'].ffill()
    df['PrevKurtosisLowsVariance'] = df['KurtosisLowsNumerator']/(df['NumberOfLows'])
    df['PrevKurtosisLows'] = df['PrevKurtosisLowsVariance']/(df['PrevStdLows']**4)-3
    df['KurtosisLowsVariance'] = (df['KurtosisLowsNumerator'] + (df['Lows'] - df['MeanLows'])**4)/df['CurrentNoOfLows']
    df['KurtosisLows'] = (df['KurtosisLowsVariance']/(df['StdLows']**4)-3).round(2)
    df['Lows_10th_Percentile'] = df['Lows'].expanding().quantile(0.10)
    df['Lows_25th_Percentile'] = df['Lows'].expanding().quantile(0.25)
    df['Lows_50th_Percentile'] = df['Lows'].expanding().quantile(0.50)
    df['Lows_75th_Percentile'] = df['Lows'].expanding().quantile(0.75)
    df['Lows_90th_Percentile'] = df['Lows'].expanding().quantile(0.90)
    df['LowsIQR'] = (df['Lows_75th_Percentile'] - df['Lows_25th_Percentile']).round(2)

    df['Highs'] = (df['PrevHigh'] - df['High']).round(2)
    df['Highs'] = np.where(df['SessionBar'] == True, 0, df['Highs'])
    df['Highs'] = np.where(df['Highs'] < 0, 0, df['Highs'])
    df['LastHighs'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Highs'].shift(1), np.nan)
    df['LastHighs'] = df['LastHighs'].shift(1)
    df['LastHighs'] = df['LastHighs'].ffill()
    df['LastHighs'] = np.where(df['LastHighs'] < 0, 0, df['LastHighs'])
    highs_dict = df['Highs'].to_dict()
    df['PrevHighs'] = df['PrevBar'].map(lambda x: highs_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfHighs'] = ((df['PrevHighs'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfHighs'] = np.where(df['Lows'] > 0, df['NumberOfHighs']+1, df['NumberOfHighs'])
    df['SumPrevHighs'] = df['PrevHighs'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevHighs'] = df['SumPrevHighs'].ffill()
    df['PrevMeanHighs'] = (df['SumPrevHighs']/df['NumberOfHighs']).round(2)
    df['MeanHighs'] = ((df['SumPrevHighs'] + df['Highs'])/df['CurrentNoOfHighs']).round(2)
    df['ResidualsHighs'] = (np.square(df['PrevHighs'] - df['PrevMeanHighs']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()).round(2)
    df['ResidualsHighs'] = df['ResidualsHighs'].ffill()
    df['PrevStdHighs'] = np.sqrt(df['ResidualsHighs']/df['NumberOfHighs'])
    df['ResidualsHighs'] = df['ResidualsHighs'] + np.square(df['Highs'] - df['MeanHighs'])
    df['StdHighs'] = (np.sqrt(df['ResidualsHighs']/df['CurrentNoOfHighs'])).round(2)
    df['PrevZScoreHighs'] = ((df['PrevHighs']-df['PrevMeanHighs'])/df['PrevStdHighs']).round(2)
    df['ZScoreHighs'] = ((df['Highs']-df['MeanHighs'])/df['StdHighs']).round(2)
    df['SkewHighsNumerator'] = ((df['PrevHighs'] - df['PrevMeanHighs'])**3).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()
    df['SkewHighsNumerator'] = df['SkewHighsNumerator'].ffill()
    df['PrevSkewHighsVariance'] = df['SkewHighsNumerator']/(df['NumberOfHighs'])
    df['PrevSkewHighs'] = df['PrevSkewHighsVariance']/(df['PrevStdHighs']**3)
    df['SkewHighsVariance'] = (df['SkewHighsNumerator'] + (df['Highs'] - df['MeanHighs'])**3)/df['CurrentNoOfHighs']
    df['SkewHighs'] = (df['SkewHighsVariance']/(df['StdHighs']**3)).round(2)
    df['CVHighs'] = (df['StdHighs']/df['MeanHighs']).round(2)
    df['KurtosisHighsNumerator'] = ((df['PrevHighs'] - df['PrevMeanHighs'])**4).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()
    df['KurtosisHighsNumerator'] = df['KurtosisHighsNumerator'].ffill()
    df['PrevKurtosisHighsVariance'] = df['KurtosisHighsNumerator']/(df['NumberOfHighs'])
    df['PrevKurtosisHighs'] = df['PrevKurtosisHighsVariance']/(df['PrevStdHighs']**4)-3
    df['KurtosisHighsVariance'] = (df['KurtosisHighsNumerator'] + (df['Highs'] - df['MeanHighs'])**4)/df['CurrentNoOfHighs']
    df['KurtosisHighs'] = (df['KurtosisHighsVariance']/(df['StdHighs']**4)-3).round(2)
    df['Highs_10th_Percentile'] = df['Highs'].expanding().quantile(0.10)
    df['Highs_25th_Percentile'] = df['Highs'].expanding().quantile(0.25)
    df['Highs_50th_Percentile'] = df['Highs'].expanding().quantile(0.50)
    df['Highs_75th_Percentile'] = df['Highs'].expanding().quantile(0.75)
    df['Highs_90th_Percentile'] = df['Highs'].expanding().quantile(0.90)
    df['HighsIQR'] = (df['Highs_75th_Percentile'] - df['Highs_25th_Percentile']).round(2)

    df['le_next_low_pred'] = 2 * df['Low'] - df['Low'].shift(1)
    df['le_next_high_pred'] = 2 * df['High'] - df['High'].shift(1) 
    df['atr'] = df['Range'].rolling(window=14, min_periods=1).mean()  
    df['atr_next_low_pred'] = df['Low'] - 0.5 * df['atr']
    df['atr_next_high_pred'] = df['High'] + 0.5 * df['atr'] 
    df['reg_next_low_pred'] = df['Low'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['reg_next_high_pred'] = df['High'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['ema_low'] = df['Low'].ewm(span=10, adjust=False).mean()
    df['ema_high'] = df['High'].ewm(span=10, adjust=False).mean()
    df['low_std'] = df['Low'].rolling(window=10, min_periods=1).std()
    df['high_std'] = df['High'].rolling(window=10, min_periods=1).std()
    df['ema_next_low_pred'] = df['ema_low'] - df['low_std']
    df['ema_next_high_pred'] = df['ema_high'] + df['high_std']
    #ensemble of em all
    df['predicted_next_low'] = (df['le_next_low_pred'] + df['atr_next_low_pred'] + df['reg_next_low_pred']) / 3
    df['predicted_next_high'] = (df['le_next_high_pred'] + df['atr_next_high_pred'] + df['reg_next_high_pred']) / 3
    df['feature_imbalance'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    df['realtive_spread'] = df['Spread'] / ((df['BestBid'] + df['BestAsk']) / 2)
    #rapdi changed in bid ask prices - quote stuffing
    df['quote_stuffing'] = df['BestBid'].diff().abs() + df['BestAsk'].diff().abs()
    #liq based    
    df['depth_slope'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    # Hidden Liquidity Detection (large orders appearing and disappearing)
    df['hidden_liquidity'] = ((df['BidSize'] > df['BidSize'].shift(1) * 2) & (df['BidSize'].shift(-1) < df['BidSize'] / 2)).astype(int)
    ## 4. Price Action & Trend Features
    df['price_momentum'] = df['Close'].diff()
    df['price_momentum_ratio'] = df['Close'] / df['Close'].shift(1)
    
    # Hilbert Transform (Measures Market Cycles)
    df['hilbert_phase'] = df['Close'].rolling(50).apply(rolling_hilbert, raw=True).shift(1)
    
    # Fraction of Large Orders (Relative to Volume)
    df['large_order_fraction'] = (df['BidSize'] + df['AskSize']) / (df['Volume'] + 1e-9)
    
    #micorstucture volatlity
    df['mvr'] = df['Close'].pct_change().rolling(5).std().shift(1) / df['Close'].pct_change().rolling(30).std().shift(1)

    df = df.drop(columns = ['PrevRange', 'GroupNumTime', 'PrevMeanRange', 'PrevStdRange', 'PrevZScoreRange', 'PrevSkewRange', 'PrevKurtosisVariance', 
                            'PrevKurtosisRange', 'PrevMeanLows', 'PrevLows', 'PrevMeanHighs', 'PrevLows', 'PrevHighs', 'PrevStdLows', 
                            'PrevStdHighs', 'PrevZScoreLows', 'PrevZScoreHighs', 'PrevSkewLows', 'PrevKurtosisLows', 'PrevZScoreHighs',
                            'PrevSkewHighs', 'PrevKurtosisHighs', 'MinRange', 'MaxRange'])
    df = df.drop(columns = ['Group', 'Session', 'SessionBar', 'PrevBar', 'PrevPrevBar', 'PrevOpen', 'PrevHigh', 'PrevLow', 'PrevClose', 
                            'ShiftedRange', 'PrevPrevRange', 'SumPrevRange', 'ResidualsRange', 'SkewNumerator', 'PrevSkewVariance', 
                            'SkewVariance', 'KurtosisNumerator', 'KurtosisVariance', 'PrevAutoCorRangeNumerator', 'Range_10th_Percentile', 
                            'Range_25th_Percentile', 'Range_50th_Percentile', 'Range_75th_Percentile', 'Range_90th_Percentile', 'RangeOutlierSum2',
                            'PrevAutoCorRangeDenominator', 'SumPrevLows', 'SumPrevHighs', 'NumberOfLows', 'NumberOfHighs', 'CurrentNoOfLows', 
                            'CurrentNoOfHighs', 'ResidualsLows', 'Lows_10th_Percentile', 'Lows_25th_Percentile', 'Lows_50th_Percentile',
                            'Lows_75th_Percentile', 'Lows_90th_Percentile', 'KurtosisLowsVariance', 'PrevKurtosisLowsVariance', 'RangeOutlierSum3',
                            'KurtosisLowsNumerator', 'SkewLowsVariance', 'PrevSkewLowsVariance', 'SkewLowsNumerator', 'ResidualsHighs', 
                            'SkewHighsNumerator', 'PrevSkewHighsVariance', 'SkewHighsVariance', 'KurtosisHighsNumerator', 'RangeOutlierSum1',
                            'PrevKurtosisHighsVariance', 'KurtosisHighsVariance', 'Highs_10th_Percentile', 'Highs_25th_Percentile', 
                            'Highs_50th_Percentile', 'Highs_75th_Percentile', 'Highs_90th_Percentile', 'TimeSumPrevRange','TimeSumGroupNumTime',
                            'RangeOutlierImpact2', 'RangeOutlierImpact3', 'SumIntervalMeanRange', 'SkewLTQVariance', 'SumPrevLTQ',
                            'PrevLTQ', 'PrevMeanLTQ', 'PrevAutoCorLTQDenominator', 'PrevAutoCorLTQNumerator', 'PrevPrevLTQ', 'LTQ_10th_Percentile',
                            'LTQ_25th_Percentile', 'LTQ_50th_Percentile', 'LTQ_75th_Percentile', 'LTQ_90th_Percentile', 'DeviationFlag', 
                            'SumAbsoluteDeviationLTQ', 'AbsoluteDeviationLTQ', 'SumPrevVolume', 'ResidualsVolume', 'PrevStdVolume', 'PrevMeanVolume',
                            'PrevZScoreVolume', 'PrevSkewVolume', 'PrevKurtosisVolume', 'PrevAutoCorVolumeNumerator', 'PrevAutoCorVolumeDenominator', 
                            'TimeSumPrevVolume', 'TimeSumGroupNumTime', 'VolumeOutlierSum1', 'VolumeOutlierSum2', 'VolumeOutlierSum3', 'ForwardVolume', 
                            'SumIntervalMeanVolume', 'PrevPrevVolume'])
    df = df.drop(columns=[
                            'ForwardRange', 'ProportionVolume', 'ProportionRange', 'SpeedVolume', 'SpeedRange', 
                            'MeanLTQ', 'StdLTQ', 'MeanVolume', 'StdVolume', 'MeanRange', 'StdRange', 
                            'Volume_10th_Percentile', 'Volume_25th_Percentile', 'Volume_50th_Percentile', 
                            'Volume_75th_Percentile', 'Volume_90th_Percentile', 'TimeMeanVolume', 
                            'IntervalMeanVolume', 'TimeMeanRange', 'IntervalMeanRange', 'RangeOutlierImpact1'
                    ])

    return df

# ==========================================
# 4. RESAMPLING LOGIC (FIXED)
# ==========================================

def resampled_ohlc(df):
    df = df.copy()
    
    # --- FIX: Rename Columns if they are in raw format ---
    # This prevents the KeyError: 'BuyPrice'
    rename_map = {
        'BestBuyPrice': 'BuyPrice',
        'BestSellPrice': 'SellPrice',
        'BestBuyQty': 'BuyQty',
        'BestSellQty': 'SellQty',
        'BestBid': 'BuyPrice',
        'BestAsk': 'SellPrice',
        'BidSize': 'BuyQty',
        'AskSize': 'SellQty',
        'LastTradedQty': 'LTQ',
        'LastTradedPrice': 'LTP' # Ensure LTP is consistent
    }
    df.rename(columns=rename_map, inplace=True)
    # -----------------------------------------------------

    df['Time'] = df['Time'].astype(str)
    df['Time'] = df['Time'].apply(clean_time_format) 
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')

    df.set_index('datetime', inplace=True)

    intervals = ['15s']
    resampled_dfs = {}

    for interval in intervals:
        
        interval_start = df.index.floor(interval)
        df['group'] = interval_start
        df['GroupTime'] = interval_start.time
        df['TimePassed'] = ((pd.to_datetime(df['Time'], format='%H:%M:%S.%f') - pd.to_datetime(df['GroupTime'], format='%H:%M:%S')).dt.total_seconds())

        df['Open'] = df.groupby('group')['LTP'].transform('first')
        df['High'] = df.groupby('group')['LTP'].transform('cummax')
        df['Low'] = df.groupby('group')['LTP'].transform('cummin')
        df['Close'] = df['LTP'] 
        
        # Now these keys will exist because of the rename block above
        df['BestBid'] = df['BuyPrice']
        df['BestAsk'] = df['SellPrice']
        df['BidSize'] = df['BuyQty']
        df['AskSize'] = df['SellQty']
        
        df['Volume'] = df.groupby('group')['LTQ'].transform('cumsum')
                
        df['GroupNum'] = (df['group'] != df['group'].shift(1)).cumsum()

        df['GroupTimeOnly'] = df['group'].dt.time
        intervals_list_sorted = sorted(df['GroupTimeOnly'].unique())
        time_to_groupnum_mapping = {time: idx + 1 for idx, time in enumerate(intervals_list_sorted)}
        df['GroupNumTime'] = df['GroupTimeOnly'].map(time_to_groupnum_mapping)
        
        ohlc = df.copy()

        ohlc.reset_index(inplace=True)
        ohlc['Date'] = ohlc['datetime'].dt.strftime('%d/%m/%Y')
        ohlc.drop(columns=['datetime'], inplace=True)
        
        bar_change_mask = ohlc['GroupNum'] != ohlc['GroupNum'].shift(1)
        ohlc['PrevBar'] = np.nan
        ohlc['PrevPrevBar'] = np.nan

        ohlc['PrevBar'] = np.where(bar_change_mask, ohlc.index - 1, np.nan)
        ohlc['PrevBar'] = np.where(ohlc['PrevBar'] == -1, np.nan, ohlc['PrevBar'])
        ohlc['PrevBar'] = ohlc['PrevBar'].ffill()

        ohlc['PrevPrevBar'] = np.where(bar_change_mask, ohlc['PrevBar'].shift(1), np.nan)
        ohlc['PrevPrevBar'] = ohlc['PrevPrevBar'].ffill()

        ohlc.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'group': 'Group'}, inplace=True)
        ohlc.dropna(subset=['Open', 'High', 'Low', 'Close'], inplace=True)
        
        # Select required columns
        req_cols = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk', 'BidSize', 'AskSize', 'Volume', 'Group', 'GroupNumTime', 'TimePassed', 'GroupNum', 'PrevBar', 'PrevPrevBar']
        ohlc = ohlc[req_cols]
        
        ohlc = additional_columns(ohlc)
        ohlc['SessionBar'] = ohlc.groupby('Group')['Session'].transform(lambda x: (x == 0).any())
        
        ohlc = ohlc.reset_index(drop=True)
        
        resampled_dfs[interval] = [ohlc]

    return resampled_dfs

# ==========================================
# 5. MAIN EXECUTION LOOP
# ==========================================

if __name__ == "__main__":
    
    # ---------------------------------------------------------
    # PLACEHOLDER: Load your data here!
    # df_start = pd.read_csv('master_fut_df.csv') 
    # ---------------------------------------------------------
    
    if 'df_start' in locals():
        
        df_start['Date'] = pd.to_datetime(df_start['Date'], dayfirst=True, errors='coerce')
        df_start['Date'] = df_start['Date'].dt.floor('D')

        df_start['Time'] = pd.to_datetime(
            df_start['Time'],
            format="%H:%M:%S.%f",
            errors='coerce'
        ).fillna(pd.to_datetime(df_start['Time'], format="%H:%M:%S", errors='coerce')).dt.time

        unique_dates = sorted(df_start['Date'].unique())
        
        # --- MODIFIED: Use full range for "train on all data" request ---
        start_date = pd.Timestamp("2025-09-01")
        end_date = pd.Timestamp("2025-11-15")
        
        print(f'Starting bulk processing from {start_date.date()} to {end_date.date()}')

        start_time = pd.to_datetime("09:15:00").time()
        end_time = pd.to_datetime("15:30:00").time()

        # Filter the entire dataset at once
        df_current = df_start[
            (df_start['Date'] >= start_date) &
            (df_start['Date'] <= end_date) &
            (df_start['Time'] >= start_time) &
            (df_start['Time'] < end_time)
        ].copy()
        
        # 1. Resample
        Nifty_FUT_resampled = resampled_ohlc(df_current)
        df = Nifty_FUT_resampled['15s'][0]
        del Nifty_FUT_resampled
        del df_current
        gc.collect()

        # 2. Aggregation
        df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
        df.set_index('DateTime', inplace = True)

        resampled_df = df.resample('15s').agg({      
                        'Date': 'last',         
                        'Time': 'last',        
                        'Open': 'first',
                        'High':'max',
                        'Low':'min',
                        'Close':'last',
                        'BestBid':'last',
                        'BestAsk':'last',
                        'BidSize':'last',
                        'AskSize':'last',
                        'Volume':'sum',
                        'Group':'last',
                        'GroupNumTime':'last',
                        'TimePassed':'last', 
                        'GroupNum':'last', 
                        'PrevBar':'last',
                        'PrevPrevBar':'last', 
                        'Session':'last', 
                        'SessionBar':'last' 
            }).reset_index()
        resampled_df.drop(columns = 'DateTime', inplace = True , axis =1, errors = 'ignore')

        print('Agg Done')
        
        df = resampled_df.copy()
        gc.collect()

        # 3. Add Basic Variables
        df_after_adding_variables = generate_basic_variables(df)
        print('Variables Added')

        # 4. Add Complex Variables
        df_after_adding_variables['Date_Time'] = pd.to_datetime(df_after_adding_variables['Date'] + ' ' + df_after_adding_variables['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
        df_after_adding_variables.set_index('Date_Time', inplace=True)
        df_after_adding_variables['Close'] = pd.to_numeric(df_after_adding_variables['Close'], errors='coerce')
        df_after_adding_variables.dropna(subset=['Close'], inplace=True)
        
        window_sizes = [200, 500, 1000, 1500]
        
        for window in window_sizes:
            df_after_adding_variables[f'Lyapunov_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
                lambda x: lyapunov_exponent_numba(x), raw=True)
            df_after_adding_variables[f'Hurst_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
                lambda x: hurst_exponent_numba(x), raw=True)
            df_after_adding_variables[f'FDI_{window}'] = df_after_adding_variables['Close'].rolling(window=window).apply(
                lambda x: fractal_dimension_numba(x), raw=True)
        
        df_after_adding_variables.reset_index(drop=True, inplace=True)
        
        print('Feature Engineering Complete. Preparing Training Data...')

        df = df_after_adding_variables.copy()
        del df_after_adding_variables
        gc.collect()

        # 5. Training Prep
        df['Pct_Change'] = df['Close'].pct_change() * 100
        df['Target'] = df['Pct_Change'].shift(-1)
        df = df.dropna(subset=['Pct_Change', 'Target'])
        
        feature_columns = ['Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk',
                           'BidSize', 'AskSize', 'Volume', 'TimePassed', 'GroupNum', 'PrevVolume',
                           'SecondInterval', 'Spread', 'SpreadSize', 'LTQ', 'ZScoreLTQ', 'SkewLTQ',
                           'AutoCorrelationLTQ', 'MeanAbsoluteDeviationLTQ', 'ZScoreVolume',
                           'SkewVolume', 'CVVolume', 'KurtosisVolume', 'AutoCorrelationVolume',
                           'VolumeOutlierImpact1', 'VolumeOutlierImpact2', 'VolumeOutlierImpact3', 'Range',
                           'ZScoreRange', 'SkewRange', 'CVRange', 'KurtosisRange',
                           'AutoCorrelationRange', 'NormalizedRange', 'Lows', 'LastLows', 'MeanLows',
                           'StdLows', 'ZScoreLows', 'SkewLows', 'CVLows', 'KurtosisLows',
                           'Highs', 'LastHighs', 'MeanHighs', 'StdHighs', 'ZScoreHighs', 'SkewHighs',
                           'CVHighs', 'KurtosisHighs', 'le_next_low_pred', 'le_next_high_pred',
                           'atr', 'atr_next_low_pred', 'atr_next_high_pred', 'reg_next_low_pred',
                           'reg_next_high_pred', 'ema_low', 'ema_high', 'low_std', 'high_std',
                           'ema_next_low_pred', 'ema_next_high_pred', 'predicted_next_low',
                           'predicted_next_high', 'feature_imbalance', 'realtive_spread', 'quote_stuffing',
                           'depth_slope', 'price_momentum', 'price_momentum_ratio',
                           'hilbert_phase', 'large_order_fraction', 'mvr']
        
        X = df[feature_columns]
        y = df['Target']
        
        # Split for validation during tuning
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, shuffle=False
        )
        
        # Clean
        X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.median())
        X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.median())
        
        # Scale (Fit on Training portion first for tuning)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # 6. Hyperparameter Tuning
        print("Starting Optuna Optimization...")
        def objective(trial):
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "max_depth": trial.suggest_int("max_depth", 4, 16),
                "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.15),
                "subsample": trial.suggest_float("subsample", 0.7, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 0.8),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
                "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
                "alpha": trial.suggest_float("alpha", 0, 1),
                "objective": "reg:squarederror",
                "random_state": 42,
                "n_jobs": -1,
            }
        
            model = xgb.XGBRegressor(**params)
            tscv = TimeSeriesSplit(n_splits=5)
            scores = []
        
            for train_idx, valid_idx in tscv.split(X_train_scaled):
                X_t, X_v = X_train_scaled[train_idx], X_train_scaled[valid_idx]
                y_t, y_v = y_train.values[train_idx], y_train.values[valid_idx]
        
                model.fit(X_t, y_t,
                          eval_set=[(X_v, y_v)],
                          verbose=False)
                
                y_pred = model.predict(X_v)
                rmse = mean_squared_error(y_v, y_pred)
                scores.append(rmse)
        
            return np.mean(scores)
        
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=50)
        
        print("Best parameters:", study.best_trial.params)
        
        # 7. Final Training on ALL Data
        print("Retraining Final Model on ALL Data...")
        
        # Combine Train + Test for final production model
        X_full = pd.concat([X_train, X_test])
        y_full = pd.concat([y_train, y_test])
        
        # Refit Scaler on full data
        final_scaler = StandardScaler()
        X_full_scaled = final_scaler.fit_transform(X_full)
        
        best_params = study.best_trial.params
        final_model = xgb.XGBRegressor(**best_params)
        final_model.fit(X_full_scaled, y_full)
        
        # 8. Save Artifacts
        date_str = f"{start_date.date()}-{end_date.date()}"
        scaler_filename = f"{date_str}-scaler_tuned.pkl"
        model_filename = f"{date_str}-model-tuned.pkl"
        
        joblib.dump(final_scaler, scaler_filename)
        joblib.dump(final_model, model_filename)
        
        print(f"Success! Saved {model_filename} and {scaler_filename}")
        
        # Optional: Print final test metrics just for reference (using the split)
        y_pred_test = final_model.predict(X_test_scaled) # Using the old scaled test set roughly proxy
        r2 = r2_score(y_test, y_pred_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        print("Reference Test R² Score:", r2)
        print("Reference Test RMSE:", rmse)

    else:
        print("Please load your initial dataframe into 'df_start' before running.")

Starting bulk processing from 2025-09-01 to 2025-11-15
Agg Done
Variables Added
Feature Engineering Complete. Preparing Training Data...


[I 2025-12-01 19:31:56,628] A new study created in memory with name: no-name-13b82e32-b049-49a7-90ad-54e2fa093b1b


Starting Optuna Optimization...


[I 2025-12-01 19:31:58,160] Trial 0 finished with value: 0.0002862393694773392 and parameters: {'n_estimators': 140, 'max_depth': 10, 'learning_rate': 0.07618328265894635, 'subsample': 0.8615378056676061, 'colsample_bytree': 0.7654742358785116, 'gamma': 0.46180516603394245, 'min_child_weight': 6, 'reg_lambda': 0.05647248222355161, 'alpha': 0.6753793574611354}. Best is trial 0 with value: 0.0002862393694773392.
[I 2025-12-01 19:32:03,604] Trial 1 finished with value: 0.00028623964747464945 and parameters: {'n_estimators': 483, 'max_depth': 14, 'learning_rate': 0.11376144367166414, 'subsample': 0.7483036266828061, 'colsample_bytree': 0.9795396370703044, 'gamma': 0.44472251861744444, 'min_child_weight': 1, 'reg_lambda': 0.5447976944988219, 'alpha': 0.5513373901629304}. Best is trial 0 with value: 0.0002862393694773392.
[I 2025-12-01 19:32:06,946] Trial 2 finished with value: 0.00028617017569082905 and parameters: {'n_estimators': 290, 'max_depth': 12, 'learning_rate': 0.07447700465528206,

Best parameters: {'n_estimators': 496, 'max_depth': 12, 'learning_rate': 0.13068924506412566, 'subsample': 0.7549463365229563, 'colsample_bytree': 0.8213085003655075, 'gamma': 0.18829837469517047, 'min_child_weight': 4, 'reg_lambda': 0.2827800341689895, 'alpha': 0.1986980026635773}
Retraining Final Model on ALL Data...
Success! Saved 2025-09-01-2025-11-15-model-tuned.pkl and 2025-09-01-2025-11-15-scaler_tuned.pkl
Reference Test R² Score: -0.005075094790603751
Reference Test RMSE: 0.01686373381556758


In [32]:
df_start

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,2025-09-01,08:46:42.278000,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,2025-09-01,09:10:01.994000,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,2025-09-01,09:15:01.722000,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,2025-09-01,09:15:02.471000,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,2025-09-01,09:15:02.975000,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,2025-11-13,15:29:33.077000,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,2025-11-13,15:29:34.659000,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,2025-11-13,15:29:35.142000,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,2025-11-13,15:29:35.834000,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [ ]:
MODEL_PATH = "/Users/harshitgajjar/kinetic-bot/2025-09-01-2025-11-15-model-tuned.pkl"
SCALER_PATH = "/Users/harshitgajjar/kinetic-bot/2025-09-01-2025-11-15-model-tuned.pkl"

In [43]:
import pandas as pd
import numpy as np
import warnings
import joblib
import boto3
import os
import io
from datetime import datetime as dt
from collections import deque
from scipy.signal import hilbert
from numba import njit
from sklearn.metrics import accuracy_score, classification_report

# ==========================================
# CONFIGURATION
# ==========================================
# Model Artifacts (Ensure these match your saved files)
MODEL_PATH = "/Users/harshitgajjar/kinetic-bot/2025-09-01-2025-11-15-model-tuned.pkl"
SCALER_PATH = "/Users/harshitgajjar/kinetic-bot/2025-09-01-2025-11-15-scaler_tuned.pkl"

# Testing Date (Nov 17 is the Monday after Nov 15)
TEST_YEAR = 2025
TEST_MONTH = 11
TEST_DAY = 17 
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"
BUCKET = "live-market-data"

# Trading Params
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
COST_PER_TRADE = 1.0 # Points

warnings.filterwarnings('ignore')

# ==========================================
# 1. IDENTICAL FEATURE ENGINEERING
# ==========================================

@njit
def std_numba(arr):
    n = arr.shape[0]; s = 0.0; mean = 0.0
    for i in range(n): mean += arr[i]
    mean /= n
    for i in range(n): diff = arr[i] - mean; s += diff * diff
    return np.sqrt(s / n)

@njit
def hurst_exponent_numba(ts, max_lag=100):
    n = ts.shape[0]
    if n < max_lag: return np.nan
    m = max_lag - 2
    lags = np.empty(m); tau = np.empty(m)
    for i in range(m):
        lag = i + 2; lags[i] = lag
        diff_sum = 0.0; count = n - lag
        for j in range(count): diff_sum += ts[j + lag] - ts[j]
        mean_diff = diff_sum / count
        var = 0.0
        for j in range(count): d = (ts[j + lag] - ts[j]) - mean_diff; var += d * d
        tau[i] = np.sqrt(var / count)
    sum_log_lags = np.sum(np.log(lags)); sum_log_tau = np.sum(np.log(tau))
    mean_log_lags = sum_log_lags / m; mean_log_tau = sum_log_tau / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_ll = np.log(lags[i]) - mean_log_lags
        diff_tau = np.log(tau[i]) - mean_log_tau
        cov += diff_ll * diff_tau; var_ll += diff_ll * diff_ll
    return cov / var_ll
    
@njit
def fractal_dimension_numba(ts, max_scale=20):
    n = ts.shape[0]
    if n < max_scale: return np.nan
    m = max_scale - 2
    scales = np.empty(m); variances = np.empty(m)
    for i in range(m):
        scale = i + 2; scales[i] = scale; npart = n // scale; var_sum = 0.0
        for j in range(npart):
            seg_mean = 0.0
            for k in range(scale): seg_mean += ts[j * scale + k]
            seg_mean /= scale
            seg_var = 0.0
            for k in range(scale): diff = ts[j * scale + k] - seg_mean; seg_var += diff * diff
            var_sum += np.sqrt(seg_var / scale)
        variances[i] = var_sum / npart
    sum_log_scales = np.sum(np.log(scales)); sum_log_vars = np.sum(np.log(variances))
    mean_log_scales = sum_log_scales / m; mean_log_vars = sum_log_vars / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_sc = np.log(scales[i]) - mean_log_scales
        diff_v = np.log(variances[i]) - mean_log_vars
        cov += diff_sc * diff_v; var_ll += diff_sc * diff_sc
    return 2 - (cov / var_ll)

@njit
def lyapunov_exponent_numba(time_series, epsilon=1e-4, steps=100):
    N = time_series.shape[0]
    if N <= steps: return np.nan
    s = 0.0; count = N - steps
    diff0 = np.abs(time_series[steps] - time_series[0])
    if diff0 < epsilon: diff0 = epsilon
    for i in range(count):
        diff = np.abs(time_series[i + steps] - time_series[i])
        if diff < epsilon: diff = epsilon
        s += np.log(diff / diff0)
    return s / (count * steps)

def clean_time_format(time_str):
    if '.' not in time_str: return f"{time_str}.000000"
    time_parts = time_str.split('.')
    milliseconds = time_parts[1][:6].ljust(6, '0')
    return f"{time_parts[0]}.{milliseconds}"  

def additional_columns(df):
    df = df.copy()
    df['Session'] = np.where((df['Date'] != df['Date'].shift(1)) & (df['Time'] != pd.to_datetime('00:00:00').time()), 0, np.nan)
    df['Session'] = np.where((df['Session'].shift(-1) == 0), 1, df['Session'])  
    df.at[df.index[-1], 'Session'] = 1
    return df

def rolling_hilbert(series):
    if len(series) < 10: return np.nan
    return np.angle(hilbert(series)[-1])
    
def forecast_next(x):
    if len(x) < 2: return x[-1]
    X = np.arange(len(x)); coef = np.polyfit(X, x, 1)   
    return np.polyval(coef, len(x))  

def generate_basic_variables(df):
    df = df.copy()
    
    open_dict, high_dict, low_dict, close_dict, volume_dict = df['Open'].to_dict(), df['High'].to_dict(), df['Low'].to_dict(), df['Close'].to_dict(), df['Volume'].to_dict()
    df['PrevOpen'] = df['PrevBar'].map(lambda x: open_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevHigh'] = df['PrevBar'].map(lambda x: high_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevLow'] = df['PrevBar'].map(lambda x: low_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevClose'] = df['PrevBar'].map(lambda x: close_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevVolume'] = df['PrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SecondInterval'] = np.floor(df['TimePassed']).where(df['TimePassed'] < 15, np.nan) + 1

    df['Spread'] = (df['BestAsk'] - df['BestBid']).round(2)
    df['SpreadSize'] = (df['AskSize'] - df['BidSize']).round(2)

    df['LTQ'] = np.where(df['GroupNum'] == df['GroupNum'].shift(1), df['Volume'] - df['Volume'].shift(1), df['Volume'])
    LTQ_dict = df['LTQ'].to_dict()
    df['PrevLTQ'] = df['PrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevLTQ'] = df['PrevPrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['MeanLTQ'] = df['LTQ'].expanding().mean().round(2)
    df['StdLTQ'] = df['LTQ'].expanding().std().round(2)
    df['SumPrevLTQ'] = df['PrevLTQ'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLTQ'] = df['SumPrevLTQ'].ffill()
    df['PrevMeanLTQ'] = (df['SumPrevLTQ']/(df['GroupNum']-1)).round(2)
    df['ZScoreLTQ'] = ((df['LTQ'] - df['MeanLTQ'])/df['StdLTQ']).round(2)
    df['SkewLTQVariance'] = ((df['LTQ'] - df['MeanLTQ'])**3).cumsum()
    df['SkewLTQ'] = (df['SkewLTQVariance']/((df['StdLTQ']**3)*(df.index))).round(2)
    
    df['LTQ_10th_Percentile'] = df['LTQ'].expanding().quantile(0.10)
    df['LTQ_25th_Percentile'] = df['LTQ'].expanding().quantile(0.25)
    df['LTQ_50th_Percentile'] = df['LTQ'].expanding().quantile(0.50)
    df['LTQ_75th_Percentile'] = df['LTQ'].expanding().quantile(0.75)
    df['LTQ_90th_Percentile'] = df['LTQ'].expanding().quantile(0.90)
    df['LTQIQR'] = (df['LTQ_75th_Percentile'] - df['LTQ_25th_Percentile']).round(2)
    
    df['PrevAutoCorLTQNumerator'] = ((df['LTQ'] - df['MeanLTQ']) * (df['LTQ'].shift(1) - df['MeanLTQ'])).cumsum()
    df['PrevAutoCorLTQDenominator'] = np.square(df['LTQ']- df['MeanLTQ']).cumsum()
    df['AutoCorrelationLTQ'] = (df['PrevAutoCorLTQNumerator']/df['PrevAutoCorLTQDenominator']).round(2)
    df['DeviationFlag'] = np.where(df['LTQ'] > (df['MeanLTQ'] + df['StdLTQ']), 1, 0)
    df['AbsoluteDeviationLTQ'] = df['DeviationFlag'].cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['LTQ'].where(df['AbsoluteDeviationLTQ'] != df['AbsoluteDeviationLTQ'].shift(1)).cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['SumAbsoluteDeviationLTQ'].ffill()
    df['MeanAbsoluteDeviationLTQ'] = np.where(df['AbsoluteDeviationLTQ'] != 0, (df['SumAbsoluteDeviationLTQ']/df['AbsoluteDeviationLTQ']).round(2), np.nan)
    
    volume_dict = df['Volume'].to_dict()
    df['PrevPrevVolume'] = df['PrevPrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevVolume'] = df['PrevVolume'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevVolume'] = df['SumPrevVolume'].ffill()
    df['PrevMeanVolume'] = (df['SumPrevVolume']/(df['GroupNum']-1)).round(2)
    df['MeanVolume'] = ((df['SumPrevVolume'] + df['Volume'])/df['GroupNum']).round(2)
    df['ResidualsVolume'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsVolume'] = df['ResidualsVolume'].ffill()
    df['PrevStdVolume'] = np.sqrt(df['ResidualsVolume']/df['GroupNum']-1)
    df['ResidualsVolume'] = df['ResidualsVolume'] + np.square(df['Volume'] - df['MeanVolume'])
    df['StdVolume'] = (np.sqrt(df['ResidualsVolume']/df['GroupNum'])).round(2)
    df['PrevZScoreVolume'] = ((df['PrevVolume']-df['PrevMeanVolume'])/df['PrevStdVolume']).round(2)
    df['ZScoreVolume'] = ((df['Volume']-df['MeanVolume'])/df['StdVolume']).round(2)
    
    df['SkewNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewVolume'] = df['PrevSkewVariance']/(df['PrevStdVolume']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Volume'] - df['MeanVolume'])**3)/df['GroupNum']
    df['SkewVolume'] = (df['SkewVariance']/(df['StdVolume']**3)).round(2)
    df['CVVolume'] = (df['StdVolume']/df['MeanVolume']).round(2)
    
    df['KurtosisNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisVolume'] = df['PrevKurtosisVariance']/(df['PrevStdVolume']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Volume'] - df['MeanVolume'])**4)/df['GroupNum']
    df['KurtosisVolume'] = (df['KurtosisVariance']/(df['StdVolume']**4)-3).round(2)
    
    df['PrevAutoCorVolumeNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume']) * (df['PrevPrevVolume'] - df['PrevMeanVolume'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorVolumeDenominator'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationVolume'] = (df['PrevAutoCorVolumeNumerator']/df['PrevAutoCorVolumeDenominator']).round(2)
    df['AutoCorrelationVolume'] = df['AutoCorrelationVolume'].ffill()
    
    df['VolumeOutlierSum1'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 1)).cumsum()
    df['VolumeOutlierSum1'] = df['VolumeOutlierSum1'].ffill()
    df['VolumeOutlierSum2'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 2)).cumsum()
    df['VolumeOutlierSum2'] = df['VolumeOutlierSum2'].ffill()
    df['VolumeOutlierSum3'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 3)).cumsum()
    df['VolumeOutlierSum3'] = df['VolumeOutlierSum3'].ffill()
    df['VolumeOutlierImpact1'] = (df['VolumeOutlierSum1']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact2'] = (df['VolumeOutlierSum2']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact3'] = (df['VolumeOutlierSum3']/df['SumPrevVolume']).round(2)

    df['Range'] = df['High'] - df['Low']
    range_dict = df['Range'].to_dict()
    df['PrevRange'] = df['PrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevRange'] = df['PrevRange'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevRange'] = df['SumPrevRange'].ffill()
    df['PrevMeanRange'] = (df['SumPrevRange']/(df['GroupNum']-1)).round(2)
    df['MeanRange'] = ((df['SumPrevRange'] + df['Range'])/df['GroupNum']).round(2)
    df['ResidualsRange'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsRange'] = df['ResidualsRange'].ffill()
    df['PrevStdRange'] = np.sqrt(df['ResidualsRange']/df['GroupNum']-1)
    df['ResidualsRange'] = df['ResidualsRange'] + np.square(df['Range'] - df['MeanRange'])
    df['StdRange'] = (np.sqrt(df['ResidualsRange']/df['GroupNum'])).round(2)
    df['PrevZScoreRange'] = ((df['PrevRange']-df['PrevMeanRange'])/df['PrevStdRange']).round(2)
    df['ZScoreRange'] = ((df['Range']-df['MeanRange'])/df['StdRange']).round(2)
    
    df['SkewNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewRange'] = df['PrevSkewVariance']/(df['PrevStdRange']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Range'] - df['MeanRange'])**3)/df['GroupNum']
    df['SkewRange'] = (df['SkewVariance']/(df['StdRange']**3)).round(2)
    df['CVRange'] = (df['StdRange']/df['MeanRange']).round(2)
    
    df['KurtosisNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisRange'] = df['PrevKurtosisVariance']/(df['PrevStdRange']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Range'] - df['MeanRange'])**4)/df['GroupNum']
    df['KurtosisRange'] = (df['KurtosisVariance']/(df['StdRange']**4)-3).round(2)
    
    df['PrevPrevRange'] = df['PrevPrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevAutoCorRangeNumerator'] = ((df['PrevRange'] - df['PrevMeanRange']) * (df['PrevPrevRange'] - df['PrevMeanRange'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorRangeDenominator'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationRange'] = (df['PrevAutoCorRangeNumerator']/df['PrevAutoCorRangeDenominator']).round(2)
    
    df['ShiftedRange'] = df['Range'].shift(1).where(df['GroupNum'] != df['GroupNum'].shift(1))
    df['MinRange'] = df['ShiftedRange'].cummin().ffill()
    df['MaxRange'] = df['Range'].where(df['ZScoreRange'] <= 3).cummax().ffill()
    df['NormalizedRange'] = np.minimum(((df['Range'] - df['MinRange'])/(df['MaxRange'] - df['MinRange'])).round(2), 1)

    df['Lows'] = (df['Low'] - df['PrevLow']).round(2)
    df['Lows'] = np.where(df['SessionBar'] == True, 0, df['Lows'])
    df['Lows'] = np.where(df['Lows'] < 0, 0, df['Lows'])
    df['LastLows'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Lows'].shift(1), np.nan)
    df['LastLows'] = df['LastLows'].shift(1).ffill()
    df['LastLows'] = np.where(df['LastLows'] < 0, 0, df['LastLows'])
    lows_dict = df['Lows'].to_dict() 
    df['PrevLows'] = df['PrevBar'].map(lambda x: lows_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfLows'] = ((df['PrevLows'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfLows'] = np.where(df['Lows'] > 0, df['NumberOfLows']+1, df['NumberOfLows'])
    df['SumPrevLows'] = df['PrevLows'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum().ffill()
    df['PrevMeanLows'] = (df['SumPrevLows']/df['NumberOfLows']).round(2)
    df['MeanLows'] = ((df['SumPrevLows'] + df['Lows'])/df['CurrentNoOfLows']).round(2)
    df['ResidualsLows'] = (np.square(df['PrevLows'] - df['PrevMeanLows']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()).round(2).ffill()
    df['PrevStdLows'] = np.sqrt(df['ResidualsLows']/df['NumberOfLows'])
    df['ResidualsLows'] = df['ResidualsLows'] + np.square(df['Lows'] - df['MeanLows'])
    df['StdLows'] = (np.sqrt(df['ResidualsLows']/df['CurrentNoOfLows'])).round(2)
    df['PrevZScoreLows'] = ((df['PrevLows']-df['PrevMeanLows'])/df['PrevStdLows']).round(2)
    df['ZScoreLows'] = ((df['Lows']-df['MeanLows'])/df['StdLows']).round(2)
    
    # Simplified Skew/Kurtosis for Lows/Highs to save space - Core stats above are critical
    df['SkewLows'] = 0.0 # Placeholder if deep calc fails/skips
    df['CVLows'] = (df['StdLows']/df['MeanLows']).round(2)
    df['KurtosisLows'] = 0.0

    df['Highs'] = (df['PrevHigh'] - df['High']).round(2)
    df['Highs'] = np.where(df['SessionBar'] == True, 0, df['Highs'])
    df['Highs'] = np.where(df['Highs'] < 0, 0, df['Highs'])
    df['LastHighs'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Highs'].shift(1), np.nan)
    df['LastHighs'] = df['LastHighs'].shift(1).ffill()
    df['LastHighs'] = np.where(df['LastHighs'] < 0, 0, df['LastHighs'])
    highs_dict = df['Highs'].to_dict()
    df['PrevHighs'] = df['PrevBar'].map(lambda x: highs_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfHighs'] = ((df['PrevHighs'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfHighs'] = np.where(df['Lows'] > 0, df['NumberOfHighs']+1, df['NumberOfHighs'])
    df['SumPrevHighs'] = df['PrevHighs'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum().ffill()
    df['PrevMeanHighs'] = (df['SumPrevHighs']/df['NumberOfHighs']).round(2)
    df['MeanHighs'] = ((df['SumPrevHighs'] + df['Highs'])/df['CurrentNoOfHighs']).round(2)
    df['ResidualsHighs'] = (np.square(df['PrevHighs'] - df['PrevMeanHighs']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()).round(2).ffill()
    df['PrevStdHighs'] = np.sqrt(df['ResidualsHighs']/df['NumberOfHighs'])
    df['ResidualsHighs'] = df['ResidualsHighs'] + np.square(df['Highs'] - df['MeanHighs'])
    df['StdHighs'] = (np.sqrt(df['ResidualsHighs']/df['CurrentNoOfHighs'])).round(2)
    df['ZScoreHighs'] = ((df['Highs']-df['MeanHighs'])/df['StdHighs']).round(2)
    
    df['SkewHighs'] = 0.0
    df['CVHighs'] = (df['StdHighs']/df['MeanHighs']).round(2)
    df['KurtosisHighs'] = 0.0

    # Predictions & Technicals
    df['le_next_low_pred'] = 2 * df['Low'] - df['Low'].shift(1)
    df['le_next_high_pred'] = 2 * df['High'] - df['High'].shift(1) 
    df['atr'] = df['Range'].rolling(window=14, min_periods=1).mean()  
    df['atr_next_low_pred'] = df['Low'] - 0.5 * df['atr']
    df['atr_next_high_pred'] = df['High'] + 0.5 * df['atr'] 
    
    # Reg Forecast
    df['reg_next_low_pred'] = df['Low'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['reg_next_high_pred'] = df['High'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    
    df['ema_low'] = df['Low'].ewm(span=10, adjust=False).mean()
    df['ema_high'] = df['High'].ewm(span=10, adjust=False).mean()
    df['low_std'] = df['Low'].rolling(window=10, min_periods=1).std()
    df['high_std'] = df['High'].rolling(window=10, min_periods=1).std()
    df['ema_next_low_pred'] = df['ema_low'] - df['low_std']
    df['ema_next_high_pred'] = df['ema_high'] + df['high_std']
    
    df['predicted_next_low'] = (df['le_next_low_pred'] + df['atr_next_low_pred'] + df['reg_next_low_pred']) / 3
    df['predicted_next_high'] = (df['le_next_high_pred'] + df['atr_next_high_pred'] + df['reg_next_high_pred']) / 3
    
    df['feature_imbalance'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    df['realtive_spread'] = df['Spread'] / ((df['BestBid'] + df['BestAsk']) / 2)
    df['quote_stuffing'] = df['BestBid'].diff().abs() + df['BestAsk'].diff().abs()
    df['depth_slope'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    
    df['price_momentum'] = df['Close'].diff()
    df['price_momentum_ratio'] = df['Close'] / df['Close'].shift(1)
    
    # Hilbert & MVR
    df['hilbert_phase'] = df['Close'].rolling(50).apply(rolling_hilbert, raw=True).shift(1)
    df['large_order_fraction'] = (df['BidSize'] + df['AskSize']) / (df['Volume'] + 1e-9)
    df['mvr'] = df['Close'].pct_change().rolling(5).std().shift(1) / df['Close'].pct_change().rolling(30).std().shift(1)

    return df

def resampled_ohlc(df):
    df = df.copy()
    
    # --- FIXED COLUMN MAPPING ---
    rename_map = {
        'BestBuyPrice': 'BuyPrice', 'BestSellPrice': 'SellPrice',
        'BestBuyQty': 'BuyQty', 'BestSellQty': 'SellQty',
        'BestBid': 'BuyPrice', 'BestAsk': 'SellPrice', # Added aliases
        'BidSize': 'BuyQty', 'AskSize': 'SellQty',
        'LastTradedQty': 'LTQ', 'LastTradedPrice': 'LTP'
    }
    df.rename(columns=rename_map, inplace=True)
    
    df['Time'] = df['Time'].astype(str).apply(clean_time_format)
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')
    df.set_index('datetime', inplace=True)

    # 15s Resampling
    interval_start = df.index.floor('15s')
    df['group'] = interval_start
    df['GroupTime'] = interval_start.time
    df['TimePassed'] = ((pd.to_datetime(df['Time'], format='%H:%M:%S.%f') - pd.to_datetime(df['GroupTime'], format='%H:%M:%S')).dt.total_seconds())

    df['Open'] = df.groupby('group')['LTP'].transform('first')
    df['High'] = df.groupby('group')['LTP'].transform('cummax')
    df['Low'] = df.groupby('group')['LTP'].transform('cummin')
    df['Close'] = df['LTP']
    
    # Safe access after rename
    df['BestBid'] = df['BuyPrice'] if 'BuyPrice' in df else np.nan
    df['BestAsk'] = df['SellPrice'] if 'SellPrice' in df else np.nan
    df['BidSize'] = df['BuyQty'] if 'BuyQty' in df else np.nan
    df['AskSize'] = df['SellQty'] if 'SellQty' in df else np.nan
    
    df['Volume'] = df.groupby('group')['LTQ'].transform('cumsum')
    df['GroupNum'] = (df['group'] != df['group'].shift(1)).cumsum()
    
    # Helper for Time features
    df['GroupTimeOnly'] = df['group'].dt.time
    intervals_list_sorted = sorted(df['GroupTimeOnly'].unique())
    time_map = {t: i + 1 for i, t in enumerate(intervals_list_sorted)}
    df['GroupNumTime'] = df['GroupTimeOnly'].map(time_map)

    ohlc = df.copy()
    ohlc.reset_index(inplace=True)
    ohlc['Date'] = ohlc['datetime'].dt.strftime('%d/%m/%Y')
    ohlc.drop(columns=['datetime'], inplace=True)
    
    mask = ohlc['GroupNum'] != ohlc['GroupNum'].shift(1)
    ohlc['PrevBar'] = np.where(mask, ohlc.index - 1, np.nan)
    ohlc['PrevBar'] = ohlc['PrevBar'].ffill()
    ohlc['PrevPrevBar'] = np.where(mask, ohlc['PrevBar'].shift(1), np.nan)
    ohlc['PrevPrevBar'] = ohlc['PrevPrevBar'].ffill()

    ohlc.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'group': 'Group'}, inplace=True)
    ohlc.dropna(subset=['Open'], inplace=True)
    ohlc = additional_columns(ohlc)
    ohlc['SessionBar'] = ohlc.groupby('Group')['Session'].transform(lambda x: (x == 0).any())
    
    return ohlc.reset_index(drop=True)

# ==========================================
# 2. PIPELINE: DATA & PREP
# ==========================================

def get_prepared_data_for_day(year, month, day):
    # Download
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{FUT_TS}.parquet"
    local_path = "test_data.parquet"
    
    s3 = boto3.client("s3")
    try:
        s3.download_file(BUCKET, key, local_path)
    except Exception as e:
        print(f"Failed to download {key}: {e}")
        return None, None

    # Load Raw Tick Data
    df_tick = pd.read_parquet(local_path)
    df_tick["DateTime"] = pd.to_datetime(
        df_tick["Date"].astype(str) + " " + df_tick["Time"].astype(str), 
        dayfirst=True, errors="coerce"
    )
    df_tick["LTP"] = pd.to_numeric(df_tick["LastTradedPrice"] if "LastTradedPrice" in df_tick else df_tick["LTP"], errors='coerce')
    df_tick = df_tick.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Process 15s Bars
    print("Generating Features (Resampling 15s)...")
    df_prep = df_tick.copy()
    df_bars = resampled_ohlc(df_prep)
    
    # Aggregation
    df_bars['DateTime'] = pd.to_datetime(df_bars['Date'] + ' ' + df_bars['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    df_bars.set_index('DateTime', inplace=True)
    
    # --- FIX 1: Removed 'PrevVolume' (doesn't exist yet) ---
    # --- FIX 2: Added ALL structural columns required by feature gen ---
    df_agg = df_bars.resample('15s').agg({
        'Date': 'last', 'Time': 'last', 'Open': 'first', 'High':'max', 'Low':'min', 'Close':'last',
        'BestBid':'last', 'BestAsk':'last', 'BidSize':'last', 'AskSize':'last', 'Volume':'sum',
        'GroupNum':'last',
        'Group':'last', 'GroupNumTime':'last', 'TimePassed':'last', 
        'PrevBar':'last', 'PrevPrevBar':'last', 'Session':'last', 'SessionBar':'last' 
    }).reset_index()
    
    # Generate Vars
    df_feats = generate_basic_variables(df_agg)
    
    # Add Complex Vars (Numba)
    df_feats['Close'] = pd.to_numeric(df_feats['Close'], errors='coerce')
    for w in [200, 500, 1000, 1500]:
        df_feats[f'Lyapunov_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: lyapunov_exponent_numba(x), raw=True)
        df_feats[f'Hurst_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: hurst_exponent_numba(x), raw=True)
        df_feats[f'FDI_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: fractal_dimension_numba(x), raw=True)
        
    return df_tick, df_feats

# ==========================================
# 3. TEST 1: DIRECTIONAL ACCURACY
# ==========================================

def test_model_direction(df_bars, model, scaler):
    print("\n--- TEST 1: MODEL DIRECTIONAL ACCURACY ---")
    
    # Features List (Must match training exactly)
    feats = ['Open', 'High', 'Low', 'Close', 'BestBid', 'BestAsk',
             'BidSize', 'AskSize', 'Volume', 'TimePassed', 'GroupNum', 'PrevVolume',
             'SecondInterval', 'Spread', 'SpreadSize', 'LTQ', 'ZScoreLTQ', 'SkewLTQ',
             'AutoCorrelationLTQ', 'MeanAbsoluteDeviationLTQ', 'ZScoreVolume',
             'SkewVolume', 'CVVolume', 'KurtosisVolume', 'AutoCorrelationVolume',
             'VolumeOutlierImpact1', 'VolumeOutlierImpact2', 'VolumeOutlierImpact3', 'Range',
             'ZScoreRange', 'SkewRange', 'CVRange', 'KurtosisRange',
             'AutoCorrelationRange', 'NormalizedRange', 'Lows', 'LastLows', 'MeanLows',
             'StdLows', 'ZScoreLows', 'SkewLows', 'CVLows', 'KurtosisLows',
             'Highs', 'LastHighs', 'MeanHighs', 'StdHighs', 'ZScoreHighs', 'SkewHighs',
             'CVHighs', 'KurtosisHighs', 'le_next_low_pred', 'le_next_high_pred',
             'atr', 'atr_next_low_pred', 'atr_next_high_pred', 'reg_next_low_pred',
             'reg_next_high_pred', 'ema_low', 'ema_high', 'low_std', 'high_std',
             'ema_next_low_pred', 'ema_next_high_pred', 'predicted_next_low',
             'predicted_next_high', 'feature_imbalance', 'realtive_spread', 'quote_stuffing',
             'depth_slope', 'price_momentum', 'price_momentum_ratio',
             'hilbert_phase', 'large_order_fraction', 'mvr']
             
    # Prepare Data
    # REMOVED STRICT DROPNA: We fill NaNs to allow test to run on short data
    df_eval = df_bars.copy()
    
    # Calculate Actual Target (Next Bar Return)
    df_eval['Actual_Ret'] = df_eval['Close'].pct_change().shift(-1) * 100
    df_eval.dropna(subset=['Actual_Ret'], inplace=True)
    
    # Predict
    X = df_eval[feats]
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0) # IMPUTATION
    X_scaled = scaler.transform(X)
    
    preds = model.predict(X_scaled)
    df_eval['Pred_Ret'] = preds
    
    # Binary Classification (Up/Down)
    df_eval['Actual_Dir'] = np.where(df_eval['Actual_Ret'] > 0, 1, 0)
    df_eval['Pred_Dir'] = np.where(df_eval['Pred_Ret'] > 0, 1, 0)
    
    acc = accuracy_score(df_eval['Actual_Dir'], df_eval['Pred_Dir'])
    print(f"Model Directional Accuracy (15s Horizon): {acc*100:.2f}%")
    print(classification_report(df_eval['Actual_Dir'], df_eval['Pred_Dir']))
    
    return df_eval

# ==========================================
# 4. TEST 2: KINETIC INTEGRATION
# ==========================================

class KineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.buf = deque(maxlen=50)
        self.last_score = 0
    
    def tick(self, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0
        
        arr = np.array(self.buf)
        p, v = arr[:,0], arr[:,1]
        v_diff = np.diff(v)
        v_diff = np.where(v_diff < 0, 0, v_diff)
        
        score = np.sum(v_diff) / (abs(p[-1]-p[0]) + 0.05)
        self.last_score = score
        return score

def run_kinetic_trading_test(df_tick, df_bars):
    print("\n--- TEST 2: KINETIC + MODEL TRADING ---")
    
    # Index bars by time for fast lookup
    # df_bars indexed by DateTime already? No, reset_index happened.
    df_bars_lookup = df_bars.set_index('DateTime').sort_index()
    
    brain = KineticBrain(threshold=KINETIC_THRESHOLD)
    trades = []
    in_trade = False
    
    # Iteration
    for i, row in df_tick.iterrows():
        ts = row['DateTime']
        ltp = row['LTP']
        vol = row['Volume'] if 'Volume' in row else 0 # Or OpenInterest/LTQ cumsum
        
        k_score = brain.tick(ltp, vol)
        
        if not in_trade and k_score > KINETIC_THRESHOLD:
            # TRIGGER FIRED: CHECK MODEL
            # Find the latest CLOSED bar prediction
            # We use 'asof' to find the nearest bar BEFORE current tick
            try:
                # We need the prediction made at the end of the previous 15s interval
                # which predicts the current interval
                latest_bar_idx = df_bars_lookup.index.asof(ts)
                
                if pd.isna(latest_bar_idx): continue
                
                bar_data = df_bars_lookup.loc[latest_bar_idx]
                
                # Check if we have a prediction
                if 'Pred_Ret' not in bar_data: continue
                
                model_pred = bar_data['Pred_Ret']
                
                # LOGIC: KINETIC + MODEL
                if model_pred > 0.05: # Strong Buy signal (>0.05% expected return)
                    # ENTRY LONG
                    trades.append({
                        'Entry_Time': ts, 'Type': 'LONG', 'Entry_Price': ltp, 'Kinetic': k_score, 'Model_Pred': model_pred
                    })
                    in_trade = True # Simple one trade at a time logic
                    
                elif model_pred < -0.05: # Strong Sell signal
                    # ENTRY SHORT
                    trades.append({
                        'Entry_Time': ts, 'Type': 'SHORT', 'Entry_Price': ltp, 'Kinetic': k_score, 'Model_Pred': model_pred
                    })
                    in_trade = True
                    
            except Exception as e:
                pass
                
        # Simple Exit Logic (e.g., 5 mins or fixed TP/SL - just for simulation)
        # For this test, let's just log entries to see alignment
        if in_trade:
            # Artificial exit after 5 mins or if day ends
            last_entry = trades[-1]
            if (ts - last_entry['Entry_Time']).total_seconds() > 300:
                trades[-1]['Exit_Time'] = ts
                trades[-1]['Exit_Price'] = ltp
                in_trade = False

    # PnL Calc
    res = pd.DataFrame(trades)
    if not res.empty:
        res['PnL_Pts'] = np.where(res['Type']=='LONG', res['Exit_Price']-res['Entry_Price'], res['Entry_Price']-res['Exit_Price'])
        res['PnL_Pts'] -= COST_PER_TRADE
        print(f"Total Trades: {len(res)}")
        print(f"Total PnL: {res['PnL_Pts'].sum():.2f} pts")
        print(f"Win Rate: {(res['PnL_Pts']>0).mean():.2f}")
    else:
        print("No trades generated.")

# ==========================================
# MAIN
# ==========================================
def main():
    # 1. Load Model
    if not os.path.exists(MODEL_PATH):
        print("Model file not found. Run training first.")
        return
        
    # --- SMART LOAD FIX: Check which object is which ---
    obj1 = joblib.load(MODEL_PATH)
    obj2 = joblib.load(SCALER_PATH)
    
    model = None
    scaler = None
    
    # Identify Model (Has predict method)
    if hasattr(obj1, 'predict'): model = obj1
    elif hasattr(obj2, 'predict'): model = obj2
        
    # Identify Scaler (Has transform method)
    if hasattr(obj1, 'transform'): scaler = obj1
    elif hasattr(obj2, 'transform'): scaler = obj2
        
    if model is None or scaler is None:
        print("Error: Could not identify Model and Scaler correctly.")
        return

    print("Artifacts loaded and identified.")
    
    # 2. Get Data
    df_tick, df_bars = get_prepared_data_for_day(TEST_YEAR, TEST_MONTH, TEST_DAY)
    if df_tick is None: return
    
    # 3. Test Direction
    df_scored = test_model_direction(df_bars, model, scaler)
    
    # 4. Test Integration
    run_kinetic_trading_test(df_tick, df_scored)

if __name__ == "__main__":
    main()

Artifacts loaded and identified.
Generating Features (Resampling 15s)...

--- TEST 1: MODEL DIRECTIONAL ACCURACY ---
Model Directional Accuracy (15s Horizon): 40.98%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       952
           1       0.41      1.00      0.58       661

    accuracy                           0.41      1613
   macro avg       0.20      0.50      0.29      1613
weighted avg       0.17      0.41      0.24      1613


--- TEST 2: KINETIC + MODEL TRADING ---
No trades generated.


In [45]:
import pandas as pd
import numpy as np
import warnings
import joblib
import boto3
import os
import io
from datetime import datetime as dt
from collections import deque
from scipy.signal import hilbert
from numba import njit
from sklearn.metrics import accuracy_score, classification_report

# ==========================================
# CONFIGURATION
# ==========================================
# Testing Date
TEST_YEAR = 2025
TEST_MONTH = 11
TEST_DAY = 18 
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"
BUCKET = "live-market-data"

# Trading Params
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
COST_PER_TRADE = 1.0 # Points

# QUANT THRESHOLDS (The "Essence")
IMBALANCE_THRESH = 0.3  # > 0.3 means Buyers are 30% stronger
ZSCORE_THRESH = 1.5     # Volume must be 1.5 sigma above mean
HURST_THRESH = 0.45     # Only trade if market isn't purely mean-reverting

warnings.filterwarnings('ignore')

# ==========================================
# 1. IDENTICAL FEATURE ENGINEERING
# ==========================================

@njit
def std_numba(arr):
    n = arr.shape[0]; s = 0.0; mean = 0.0
    for i in range(n): mean += arr[i]
    mean /= n
    for i in range(n): diff = arr[i] - mean; s += diff * diff
    return np.sqrt(s / n)

@njit
def hurst_exponent_numba(ts, max_lag=100):
    n = ts.shape[0]
    if n < max_lag: return np.nan
    m = max_lag - 2
    lags = np.empty(m); tau = np.empty(m)
    for i in range(m):
        lag = i + 2; lags[i] = lag
        diff_sum = 0.0; count = n - lag
        for j in range(count): diff_sum += ts[j + lag] - ts[j]
        mean_diff = diff_sum / count
        var = 0.0
        for j in range(count): d = (ts[j + lag] - ts[j]) - mean_diff; var += d * d
        tau[i] = np.sqrt(var / count)
    sum_log_lags = np.sum(np.log(lags)); sum_log_tau = np.sum(np.log(tau))
    mean_log_lags = sum_log_lags / m; mean_log_tau = sum_log_tau / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_ll = np.log(lags[i]) - mean_log_lags
        diff_tau = np.log(tau[i]) - mean_log_tau
        cov += diff_ll * diff_tau; var_ll += diff_ll * diff_ll
    return cov / var_ll
    
@njit
def fractal_dimension_numba(ts, max_scale=20):
    n = ts.shape[0]
    if n < max_scale: return np.nan
    m = max_scale - 2
    scales = np.empty(m); variances = np.empty(m)
    for i in range(m):
        scale = i + 2; scales[i] = scale; npart = n // scale; var_sum = 0.0
        for j in range(npart):
            seg_mean = 0.0
            for k in range(scale): seg_mean += ts[j * scale + k]
            seg_mean /= scale
            seg_var = 0.0
            for k in range(scale): diff = ts[j * scale + k] - seg_mean; seg_var += diff * diff
            var_sum += np.sqrt(seg_var / scale)
        variances[i] = var_sum / npart
    sum_log_scales = np.sum(np.log(scales)); sum_log_vars = np.sum(np.log(variances))
    mean_log_scales = sum_log_scales / m; mean_log_vars = sum_log_vars / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_sc = np.log(scales[i]) - mean_log_scales
        diff_v = np.log(variances[i]) - mean_log_vars
        cov += diff_sc * diff_v; var_ll += diff_sc * diff_sc
    return 2 - (cov / var_ll)

@njit
def lyapunov_exponent_numba(time_series, epsilon=1e-4, steps=100):
    N = time_series.shape[0]
    if N <= steps: return np.nan
    s = 0.0; count = N - steps
    diff0 = np.abs(time_series[steps] - time_series[0])
    if diff0 < epsilon: diff0 = epsilon
    for i in range(count):
        diff = np.abs(time_series[i + steps] - time_series[i])
        if diff < epsilon: diff = epsilon
        s += np.log(diff / diff0)
    return s / (count * steps)

def clean_time_format(time_str):
    if '.' not in time_str: return f"{time_str}.000000"
    time_parts = time_str.split('.')
    milliseconds = time_parts[1][:6].ljust(6, '0')
    return f"{time_parts[0]}.{milliseconds}"  

def additional_columns(df):
    df = df.copy()
    df['Session'] = np.where((df['Date'] != df['Date'].shift(1)) & (df['Time'] != pd.to_datetime('00:00:00').time()), 0, np.nan)
    df['Session'] = np.where((df['Session'].shift(-1) == 0), 1, df['Session'])  
    df.at[df.index[-1], 'Session'] = 1
    return df

def rolling_hilbert(series):
    if len(series) < 10: return np.nan
    return np.angle(hilbert(series)[-1])
    
def forecast_next(x):
    if len(x) < 2: return x[-1]
    X = np.arange(len(x)); coef = np.polyfit(X, x, 1)   
    return np.polyval(coef, len(x))  

def generate_basic_variables(df):
    df = df.copy()
    
    open_dict, high_dict, low_dict, close_dict, volume_dict = df['Open'].to_dict(), df['High'].to_dict(), df['Low'].to_dict(), df['Close'].to_dict(), df['Volume'].to_dict()
    df['PrevOpen'] = df['PrevBar'].map(lambda x: open_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevHigh'] = df['PrevBar'].map(lambda x: high_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevLow'] = df['PrevBar'].map(lambda x: low_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevClose'] = df['PrevBar'].map(lambda x: close_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevVolume'] = df['PrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SecondInterval'] = np.floor(df['TimePassed']).where(df['TimePassed'] < 15, np.nan) + 1

    df['Spread'] = (df['BestAsk'] - df['BestBid']).round(2)
    df['SpreadSize'] = (df['AskSize'] - df['BidSize']).round(2)

    df['LTQ'] = np.where(df['GroupNum'] == df['GroupNum'].shift(1), df['Volume'] - df['Volume'].shift(1), df['Volume'])
    LTQ_dict = df['LTQ'].to_dict()
    df['PrevLTQ'] = df['PrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevLTQ'] = df['PrevPrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['MeanLTQ'] = df['LTQ'].expanding().mean().round(2)
    df['StdLTQ'] = df['LTQ'].expanding().std().round(2)
    df['SumPrevLTQ'] = df['PrevLTQ'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLTQ'] = df['SumPrevLTQ'].ffill()
    df['PrevMeanLTQ'] = (df['SumPrevLTQ']/(df['GroupNum']-1)).round(2)
    df['ZScoreLTQ'] = ((df['LTQ'] - df['MeanLTQ'])/df['StdLTQ']).round(2)
    df['SkewLTQVariance'] = ((df['LTQ'] - df['MeanLTQ'])**3).cumsum()
    df['SkewLTQ'] = (df['SkewLTQVariance']/((df['StdLTQ']**3)*(df.index))).round(2)
    
    # ... [Assuming full feature logic exists, using simplified for robustness in this snippet]
    # Critical Features for Logic:
    df['feature_imbalance'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    df['realtive_spread'] = df['Spread'] / ((df['BestBid'] + df['BestAsk']) / 2)
    
    return df

def resampled_ohlc(df):
    df = df.copy()
    
    # --- FIXED COLUMN MAPPING ---
    rename_map = {
        'BestBuyPrice': 'BuyPrice', 'BestSellPrice': 'SellPrice',
        'BestBuyQty': 'BuyQty', 'BestSellQty': 'SellQty',
        'BestBid': 'BuyPrice', 'BestAsk': 'SellPrice', # Added aliases
        'BidSize': 'BuyQty', 'AskSize': 'SellQty',
        'LastTradedQty': 'LTQ', 'LastTradedPrice': 'LTP'
    }
    df.rename(columns=rename_map, inplace=True)
    
    df['Time'] = df['Time'].astype(str).apply(clean_time_format)
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')
    df.set_index('datetime', inplace=True)

    # 15s Resampling
    interval_start = df.index.floor('15s')
    df['group'] = interval_start
    df['GroupTime'] = interval_start.time
    df['TimePassed'] = ((pd.to_datetime(df['Time'], format='%H:%M:%S.%f') - pd.to_datetime(df['GroupTime'], format='%H:%M:%S')).dt.total_seconds())

    df['Open'] = df.groupby('group')['LTP'].transform('first')
    df['High'] = df.groupby('group')['LTP'].transform('cummax')
    df['Low'] = df.groupby('group')['LTP'].transform('cummin')
    df['Close'] = df['LTP']
    
    # Safe access after rename
    df['BestBid'] = df['BuyPrice'] if 'BuyPrice' in df else np.nan
    df['BestAsk'] = df['SellPrice'] if 'SellPrice' in df else np.nan
    df['BidSize'] = df['BuyQty'] if 'BuyQty' in df else np.nan
    df['AskSize'] = df['SellQty'] if 'SellQty' in df else np.nan
    
    df['Volume'] = df.groupby('group')['LTQ'].transform('cumsum')
    df['GroupNum'] = (df['group'] != df['group'].shift(1)).cumsum()
    
    # Helper for Time features
    df['GroupTimeOnly'] = df['group'].dt.time
    intervals_list_sorted = sorted(df['GroupTimeOnly'].unique())
    time_map = {t: i + 1 for i, t in enumerate(intervals_list_sorted)}
    df['GroupNumTime'] = df['GroupTimeOnly'].map(time_map)

    ohlc = df.copy()
    ohlc.reset_index(inplace=True)
    ohlc['Date'] = ohlc['datetime'].dt.strftime('%d/%m/%Y')
    ohlc.drop(columns=['datetime'], inplace=True)
    
    mask = ohlc['GroupNum'] != ohlc['GroupNum'].shift(1)
    ohlc['PrevBar'] = np.where(mask, ohlc.index - 1, np.nan)
    ohlc['PrevBar'] = ohlc['PrevBar'].ffill()
    ohlc['PrevPrevBar'] = np.where(mask, ohlc['PrevBar'].shift(1), np.nan)
    ohlc['PrevPrevBar'] = ohlc['PrevPrevBar'].ffill()

    ohlc.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'group': 'Group'}, inplace=True)
    ohlc.dropna(subset=['Open'], inplace=True)
    ohlc = additional_columns(ohlc)
    ohlc['SessionBar'] = ohlc.groupby('Group')['Session'].transform(lambda x: (x == 0).any())
    
    return ohlc.reset_index(drop=True)

# ==========================================
# 2. PIPELINE: DATA & PREP
# ==========================================

def get_prepared_data_for_day(year, month, day):
    # Download
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{FUT_TS}.parquet"
    local_path = "test_data.parquet"
    
    s3 = boto3.client("s3")
    try:
        s3.download_file(BUCKET, key, local_path)
    except Exception as e:
        print(f"Failed to download {key}: {e}")
        return None, None

    # Load Raw Tick Data
    df_tick = pd.read_parquet(local_path)
    df_tick["DateTime"] = pd.to_datetime(
        df_tick["Date"].astype(str) + " " + df_tick["Time"].astype(str), 
        dayfirst=True, errors="coerce"
    )
    df_tick["LTP"] = pd.to_numeric(df_tick["LastTradedPrice"] if "LastTradedPrice" in df_tick else df_tick["LTP"], errors='coerce')
    df_tick = df_tick.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Process 15s Bars
    print("Generating Features (Resampling 15s)...")
    df_prep = df_tick.copy()
    df_bars = resampled_ohlc(df_prep)
    
    # Aggregation
    df_bars['DateTime'] = pd.to_datetime(df_bars['Date'] + ' ' + df_bars['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    df_bars.set_index('DateTime', inplace=True)
    
    # --- FIX 1: Removed 'PrevVolume' (doesn't exist yet) ---
    # --- FIX 2: Added ALL structural columns required by feature gen ---
    df_agg = df_bars.resample('15s').agg({
        'Date': 'last', 'Time': 'last', 'Open': 'first', 'High':'max', 'Low':'min', 'Close':'last',
        'BestBid':'last', 'BestAsk':'last', 'BidSize':'last', 'AskSize':'last', 'Volume':'sum',
        'GroupNum':'last',
        'Group':'last', 'GroupNumTime':'last', 'TimePassed':'last', 
        'PrevBar':'last', 'PrevPrevBar':'last', 'Session':'last', 'SessionBar':'last' 
    }).reset_index()
    
    # Generate Vars
    df_feats = generate_basic_variables(df_agg)
    
    # Add Complex Vars (Numba)
    df_feats['Close'] = pd.to_numeric(df_feats['Close'], errors='coerce')
    for w in [200]: # 500, 1000, 1500 omitted for speed in test
        df_feats[f'Lyapunov_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: lyapunov_exponent_numba(x), raw=True)
        df_feats[f'Hurst_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: hurst_exponent_numba(x), raw=True)
        
    return df_tick, df_feats

# ==========================================
# 3. TEST: QUANT KINETIC LOGIC
# ==========================================

class KineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.buf = deque(maxlen=50)
        self.last_score = 0
    
    def tick(self, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0
        
        arr = np.array(self.buf)
        p, v = arr[:,0], arr[:,1]
        v_diff = np.diff(v)
        v_diff = np.where(v_diff < 0, 0, v_diff)
        
        score = np.sum(v_diff) / (abs(p[-1]-p[0]) + 0.05)
        self.last_score = score
        return score

def run_kinetic_trading_test(df_tick, df_bars):
    print("\n--- TEST: QUANT KINETIC TRADING (The Essence) ---")
    
    # Index bars by time for fast lookup
    df_bars_lookup = df_bars.set_index('DateTime').sort_index()
    
    brain = KineticBrain(threshold=KINETIC_THRESHOLD)
    trades = []
    in_trade = False
    
    # Iteration
    for i, row in df_tick.iterrows():
        ts = row['DateTime']
        ltp = row['LTP']
        vol = row['Volume'] if 'Volume' in row else 0 
        
        k_score = brain.tick(ltp, vol)
        
        if not in_trade and k_score > KINETIC_THRESHOLD:
            # TRIGGER FIRED: CHECK ESSENCE FEATURES
            try:
                # Find the latest CLOSED bar feature data
                latest_bar_idx = df_bars_lookup.index.asof(ts)
                if pd.isna(latest_bar_idx): continue
                
                bar_data = df_bars_lookup.loc[latest_bar_idx]
                
                # EXTRACT ESSENCE
                imbalance = bar_data.get('feature_imbalance', 0)
                zscore = bar_data.get('ZScoreLTQ', 0)
                hurst = bar_data.get('Hurst_200', 0.5)
                
                # --- QUANT LOGIC ---
                # 1. Feature Imbalance: Positive = Buyers, Negative = Sellers
                # 2. ZScore: Must be > 1.5 (Significant Volume)
                # 3. Hurst: If < 0.45 (Mean Reverting), maybe fade? Let's stick to Trend for now.
                
                is_valid_setup = (zscore > ZSCORE_THRESH) or (hurst > HURST_THRESH)
                
                if is_valid_setup:
                    if imbalance > IMBALANCE_THRESH: 
                        # Strong Buyers + Kinetic -> LONG
                        trades.append({
                            'Entry_Time': ts, 'Type': 'LONG', 'Entry_Price': ltp, 
                            'Kinetic': k_score, 'Imbalance': imbalance, 'ZScore': zscore
                        })
                        in_trade = True 
                        
                    elif imbalance < -IMBALANCE_THRESH: 
                        # Strong Sellers + Kinetic -> SHORT
                        trades.append({
                            'Entry_Time': ts, 'Type': 'SHORT', 'Entry_Price': ltp, 
                            'Kinetic': k_score, 'Imbalance': imbalance, 'ZScore': zscore
                        })
                        in_trade = True
                    
            except Exception as e:
                pass
                
        # Simple Simulation Exit
        if in_trade:
            last_entry = trades[-1]
            # Exit after 5 mins
            if (ts - last_entry['Entry_Time']).total_seconds() > 300:
                trades[-1]['Exit_Time'] = ts
                trades[-1]['Exit_Price'] = ltp
                in_trade = False

    # PnL Calc
    res = pd.DataFrame(trades)
    if not res.empty:
        res['PnL_Pts'] = np.where(res['Type']=='LONG', res['Exit_Price']-res['Entry_Price'], res['Entry_Price']-res['Exit_Price'])
        res['PnL_Pts'] -= COST_PER_TRADE
        print(f"Total Trades: {len(res)}")
        print(f"Total PnL: {res['PnL_Pts'].sum():.2f} pts")
        print(f"Win Rate: {(res['PnL_Pts']>0).mean()*100:.2f}%")
        
        # Diagnostics
        print("\n--- Trade Diagnostics ---")
        print(res[['Entry_Time', 'Type', 'Imbalance', 'ZScore', 'PnL_Pts']].head(10))
    else:
        print("No trades generated. (Tighten Thresholds?)")

# ==========================================
# MAIN
# ==========================================
def main():
    # 2. Get Data
    print(f"Loading Data for {TEST_DAY}-{TEST_MONTH}-{TEST_YEAR}...")
    df_tick, df_bars = get_prepared_data_for_day(TEST_YEAR, TEST_MONTH, TEST_DAY)
    if df_tick is None: return
    
    # 3. Test Integration (No Model, Just Quant)
    run_kinetic_trading_test(df_tick, df_bars)

if __name__ == "__main__":
    main()

Loading Data for 18-11-2025...
Generating Features (Resampling 15s)...

--- TEST: QUANT KINETIC TRADING (The Essence) ---
Total Trades: 30
Total PnL: -51.10 pts
Win Rate: 40.00%

--- Trade Diagnostics ---
               Entry_Time   Type  Imbalance  ZScore  PnL_Pts
0 2025-11-18 09:16:13.305   LONG   0.907692    5.01     14.7
1 2025-11-18 09:21:50.408  SHORT  -0.750000    2.60    -19.7
2 2025-11-18 09:28:42.958   LONG   0.333333    2.13      5.1
3 2025-11-18 09:34:37.162   LONG   0.333333    2.44    -11.6
4 2025-11-18 09:41:03.915  SHORT  -0.800000    4.12     -8.0
5 2025-11-18 09:50:26.218  SHORT  -0.833333    1.86      4.1
6 2025-11-18 10:00:28.318  SHORT  -0.600000    3.31    -33.4
7 2025-11-18 10:19:58.677   LONG   0.333333    1.65      7.3
8 2025-11-18 10:30:02.821   LONG   0.333333    1.71    -11.0
9 2025-11-18 10:41:04.208  SHORT  -0.833333    2.82      4.3


In [46]:
import pandas as pd
import numpy as np
import warnings
import joblib
import boto3
import os
import io
from datetime import datetime as dt
from collections import deque
from scipy.signal import hilbert
from numba import njit
from sklearn.metrics import accuracy_score, classification_report

# ==========================================
# CONFIGURATION
# ==========================================
# Model Artifacts
MODEL_PATH = "2025-09-01-2025-11-15-model-tuned.pkl"
SCALER_PATH = "2025-09-01-2025-11-15-scaler-tuned.pkl"

# Testing Date (Nov 18 is a Tuesday)
TEST_YEAR = 2025
TEST_MONTH = 11
TEST_DAY = 18 
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"
BUCKET = "live-market-data"

# Trading Params
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
COST_PER_TRADE = 1.0 

# QUANT LOGIC SETTINGS
IMBALANCE_THRESH = 0.4  # Increased from 0.3 for stronger conviction
ZSCORE_THRESH = 1.5     
HURST_THRESH = 0.45     

# TRADE MANAGEMENT (The "Shield")
STOP_LOSS = 15.0        # Max tolerance for pain
TAKE_PROFIT = 30.0      # 1:2 Risk Reward
MAX_HOLD_SEC = 600      # 10 Minutes max

warnings.filterwarnings('ignore')

# ==========================================
# 1. IDENTICAL FEATURE ENGINEERING
# ==========================================

@njit
def std_numba(arr):
    n = arr.shape[0]; s = 0.0; mean = 0.0
    for i in range(n): mean += arr[i]
    mean /= n
    for i in range(n): diff = arr[i] - mean; s += diff * diff
    return np.sqrt(s / n)

@njit
def hurst_exponent_numba(ts, max_lag=100):
    n = ts.shape[0]
    if n < max_lag: return np.nan
    m = max_lag - 2
    lags = np.empty(m); tau = np.empty(m)
    for i in range(m):
        lag = i + 2; lags[i] = lag
        diff_sum = 0.0; count = n - lag
        for j in range(count): diff_sum += ts[j + lag] - ts[j]
        mean_diff = diff_sum / count
        var = 0.0
        for j in range(count): d = (ts[j + lag] - ts[j]) - mean_diff; var += d * d
        tau[i] = np.sqrt(var / count)
    sum_log_lags = np.sum(np.log(lags)); sum_log_tau = np.sum(np.log(tau))
    mean_log_lags = sum_log_lags / m; mean_log_tau = sum_log_tau / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_ll = np.log(lags[i]) - mean_log_lags
        diff_tau = np.log(tau[i]) - mean_log_tau
        cov += diff_ll * diff_tau; var_ll += diff_ll * diff_ll
    return cov / var_ll
    
@njit
def fractal_dimension_numba(ts, max_scale=20):
    n = ts.shape[0]
    if n < max_scale: return np.nan
    m = max_scale - 2
    scales = np.empty(m); variances = np.empty(m)
    for i in range(m):
        scale = i + 2; scales[i] = scale; npart = n // scale; var_sum = 0.0
        for j in range(npart):
            seg_mean = 0.0
            for k in range(scale): seg_mean += ts[j * scale + k]
            seg_mean /= scale
            seg_var = 0.0
            for k in range(scale): diff = ts[j * scale + k] - seg_mean; seg_var += diff * diff
            var_sum += np.sqrt(seg_var / scale)
        variances[i] = var_sum / npart
    sum_log_scales = np.sum(np.log(scales)); sum_log_vars = np.sum(np.log(variances))
    mean_log_scales = sum_log_scales / m; mean_log_vars = sum_log_vars / m
    cov = 0.0; var_ll = 0.0
    for i in range(m):
        diff_sc = np.log(scales[i]) - mean_log_scales
        diff_v = np.log(variances[i]) - mean_log_vars
        cov += diff_sc * diff_v; var_ll += diff_sc * diff_sc
    return 2 - (cov / var_ll)

@njit
def lyapunov_exponent_numba(time_series, epsilon=1e-4, steps=100):
    N = time_series.shape[0]
    if N <= steps: return np.nan
    s = 0.0; count = N - steps
    diff0 = np.abs(time_series[steps] - time_series[0])
    if diff0 < epsilon: diff0 = epsilon
    for i in range(count):
        diff = np.abs(time_series[i + steps] - time_series[i])
        if diff < epsilon: diff = epsilon
        s += np.log(diff / diff0)
    return s / (count * steps)

def clean_time_format(time_str):
    if '.' not in time_str: return f"{time_str}.000000"
    time_parts = time_str.split('.')
    milliseconds = time_parts[1][:6].ljust(6, '0')
    return f"{time_parts[0]}.{milliseconds}"  

def additional_columns(df):
    df = df.copy()
    df['Session'] = np.where((df['Date'] != df['Date'].shift(1)) & (df['Time'] != pd.to_datetime('00:00:00').time()), 0, np.nan)
    df['Session'] = np.where((df['Session'].shift(-1) == 0), 1, df['Session'])  
    df.at[df.index[-1], 'Session'] = 1
    return df

def rolling_hilbert(series):
    if len(series) < 10: return np.nan
    return np.angle(hilbert(series)[-1])
    
def forecast_next(x):
    if len(x) < 2: return x[-1]
    X = np.arange(len(x)); coef = np.polyfit(X, x, 1)   
    return np.polyval(coef, len(x))  

def generate_basic_variables(df):
    df = df.copy()
    
    open_dict, high_dict, low_dict, close_dict, volume_dict = df['Open'].to_dict(), df['High'].to_dict(), df['Low'].to_dict(), df['Close'].to_dict(), df['Volume'].to_dict()
    df['PrevOpen'] = df['PrevBar'].map(lambda x: open_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevHigh'] = df['PrevBar'].map(lambda x: high_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevLow'] = df['PrevBar'].map(lambda x: low_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevClose'] = df['PrevBar'].map(lambda x: close_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevVolume'] = df['PrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SecondInterval'] = np.floor(df['TimePassed']).where(df['TimePassed'] < 15, np.nan) + 1

    df['Spread'] = (df['BestAsk'] - df['BestBid']).round(2)
    df['SpreadSize'] = (df['AskSize'] - df['BidSize']).round(2)

    df['LTQ'] = np.where(df['GroupNum'] == df['GroupNum'].shift(1), df['Volume'] - df['Volume'].shift(1), df['Volume'])
    LTQ_dict = df['LTQ'].to_dict()
    df['PrevLTQ'] = df['PrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevPrevLTQ'] = df['PrevPrevBar'].map(lambda x: LTQ_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['MeanLTQ'] = df['LTQ'].expanding().mean().round(2)
    df['StdLTQ'] = df['LTQ'].expanding().std().round(2)
    df['SumPrevLTQ'] = df['PrevLTQ'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevLTQ'] = df['SumPrevLTQ'].ffill()
    df['PrevMeanLTQ'] = (df['SumPrevLTQ']/(df['GroupNum']-1)).round(2)
    df['ZScoreLTQ'] = ((df['LTQ'] - df['MeanLTQ'])/df['StdLTQ']).round(2)
    df['SkewLTQVariance'] = ((df['LTQ'] - df['MeanLTQ'])**3).cumsum()
    df['SkewLTQ'] = (df['SkewLTQVariance']/((df['StdLTQ']**3)*(df.index))).round(2)
    
    # Simplified Percentiles for speed in test mode (expanding is slow)
    # Using rolling approximation or full expanding if critical
    df['LTQ_10th_Percentile'] = df['LTQ'].expanding().quantile(0.10)
    df['LTQ_25th_Percentile'] = df['LTQ'].expanding().quantile(0.25)
    df['LTQ_50th_Percentile'] = df['LTQ'].expanding().quantile(0.50)
    df['LTQ_75th_Percentile'] = df['LTQ'].expanding().quantile(0.75)
    df['LTQ_90th_Percentile'] = df['LTQ'].expanding().quantile(0.90)
    
    df['PrevAutoCorLTQNumerator'] = ((df['LTQ'] - df['MeanLTQ']) * (df['LTQ'].shift(1) - df['MeanLTQ'])).cumsum()
    df['PrevAutoCorLTQDenominator'] = np.square(df['LTQ']- df['MeanLTQ']).cumsum()
    df['AutoCorrelationLTQ'] = (df['PrevAutoCorLTQNumerator']/df['PrevAutoCorLTQDenominator']).round(2)
    df['DeviationFlag'] = np.where(df['LTQ'] > (df['MeanLTQ'] + df['StdLTQ']), 1, 0)
    df['AbsoluteDeviationLTQ'] = df['DeviationFlag'].cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['LTQ'].where(df['AbsoluteDeviationLTQ'] != df['AbsoluteDeviationLTQ'].shift(1)).cumsum()
    df['SumAbsoluteDeviationLTQ'] = df['SumAbsoluteDeviationLTQ'].ffill()
    df['MeanAbsoluteDeviationLTQ'] = np.where(df['AbsoluteDeviationLTQ'] != 0, (df['SumAbsoluteDeviationLTQ']/df['AbsoluteDeviationLTQ']).round(2), np.nan)
    
    volume_dict = df['Volume'].to_dict()
    df['PrevPrevVolume'] = df['PrevPrevBar'].map(lambda x: volume_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevVolume'] = df['PrevVolume'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevVolume'] = df['SumPrevVolume'].ffill()
    df['PrevMeanVolume'] = (df['SumPrevVolume']/(df['GroupNum']-1)).round(2)
    df['MeanVolume'] = ((df['SumPrevVolume'] + df['Volume'])/df['GroupNum']).round(2)
    df['ResidualsVolume'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsVolume'] = df['ResidualsVolume'].ffill()
    df['PrevStdVolume'] = np.sqrt(df['ResidualsVolume']/df['GroupNum']-1)
    df['ResidualsVolume'] = df['ResidualsVolume'] + np.square(df['Volume'] - df['MeanVolume'])
    df['StdVolume'] = (np.sqrt(df['ResidualsVolume']/df['GroupNum'])).round(2)
    df['PrevZScoreVolume'] = ((df['PrevVolume']-df['PrevMeanVolume'])/df['PrevStdVolume']).round(2)
    df['ZScoreVolume'] = ((df['Volume']-df['MeanVolume'])/df['StdVolume']).round(2)
    
    df['SkewNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewVolume'] = df['PrevSkewVariance']/(df['PrevStdVolume']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Volume'] - df['MeanVolume'])**3)/df['GroupNum']
    df['SkewVolume'] = (df['SkewVariance']/(df['StdVolume']**3)).round(2)
    df['CVVolume'] = (df['StdVolume']/df['MeanVolume']).round(2)
    
    df['KurtosisNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisVolume'] = df['PrevKurtosisVariance']/(df['PrevStdVolume']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Volume'] - df['MeanVolume'])**4)/df['GroupNum']
    df['KurtosisVolume'] = (df['KurtosisVariance']/(df['StdVolume']**4)-3).round(2)
    
    df['PrevAutoCorVolumeNumerator'] = ((df['PrevVolume'] - df['PrevMeanVolume']) * (df['PrevPrevVolume'] - df['PrevMeanVolume'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorVolumeDenominator'] = np.square(df['PrevVolume'] - df['PrevMeanVolume']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationVolume'] = (df['PrevAutoCorVolumeNumerator']/df['PrevAutoCorVolumeDenominator']).round(2)
    df['AutoCorrelationVolume'] = df['AutoCorrelationVolume'].ffill()
    
    df['VolumeOutlierSum1'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 1)).cumsum()
    df['VolumeOutlierSum1'] = df['VolumeOutlierSum1'].ffill()
    df['VolumeOutlierSum2'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 2)).cumsum()
    df['VolumeOutlierSum2'] = df['VolumeOutlierSum2'].ffill()
    df['VolumeOutlierSum3'] = df['PrevVolume'].where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['ZScoreVolume'].shift(1) > 3)).cumsum()
    df['VolumeOutlierSum3'] = df['VolumeOutlierSum3'].ffill()
    df['VolumeOutlierImpact1'] = (df['VolumeOutlierSum1']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact2'] = (df['VolumeOutlierSum2']/df['SumPrevVolume']).round(2)
    df['VolumeOutlierImpact3'] = (df['VolumeOutlierSum3']/df['SumPrevVolume']).round(2)

    df['Range'] = df['High'] - df['Low']
    range_dict = df['Range'].to_dict()
    df['PrevRange'] = df['PrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['SumPrevRange'] = df['PrevRange'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SumPrevRange'] = df['SumPrevRange'].ffill()
    df['PrevMeanRange'] = (df['SumPrevRange']/(df['GroupNum']-1)).round(2)
    df['MeanRange'] = ((df['SumPrevRange'] + df['Range'])/df['GroupNum']).round(2)
    df['ResidualsRange'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['ResidualsRange'] = df['ResidualsRange'].ffill()
    df['PrevStdRange'] = np.sqrt(df['ResidualsRange']/df['GroupNum']-1)
    df['ResidualsRange'] = df['ResidualsRange'] + np.square(df['Range'] - df['MeanRange'])
    df['StdRange'] = (np.sqrt(df['ResidualsRange']/df['GroupNum'])).round(2)
    df['PrevZScoreRange'] = ((df['PrevRange']-df['PrevMeanRange'])/df['PrevStdRange']).round(2)
    df['ZScoreRange'] = ((df['Range']-df['MeanRange'])/df['StdRange']).round(2)
    
    df['SkewNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**3).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['SkewNumerator'] = df['SkewNumerator'].ffill()
    df['PrevSkewVariance'] = df['SkewNumerator']/(df['GroupNum']-1)
    df['PrevSkewRange'] = df['PrevSkewVariance']/(df['PrevStdRange']**3)
    df['SkewVariance'] = (df['SkewNumerator'] + (df['Range'] - df['MeanRange'])**3)/df['GroupNum']
    df['SkewRange'] = (df['SkewVariance']/(df['StdRange']**3)).round(2)
    df['CVRange'] = (df['StdRange']/df['MeanRange']).round(2)
    
    df['KurtosisNumerator'] = ((df['PrevRange'] - df['PrevMeanRange'])**4).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['KurtosisNumerator'] = df['KurtosisNumerator'].ffill()
    df['PrevKurtosisVariance'] = df['KurtosisNumerator']/(df['GroupNum']-1)
    df['PrevKurtosisRange'] = df['PrevKurtosisVariance']/(df['PrevStdRange']**4)-3
    df['KurtosisVariance'] = (df['KurtosisNumerator'] + (df['Range'] - df['MeanRange'])**4)/df['GroupNum']
    df['KurtosisRange'] = (df['KurtosisVariance']/(df['StdRange']**4)-3).round(2)
    
    df['PrevPrevRange'] = df['PrevPrevBar'].map(lambda x: range_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['PrevAutoCorRangeNumerator'] = ((df['PrevRange'] - df['PrevMeanRange']) * (df['PrevPrevRange'] - df['PrevMeanRange'])).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['PrevAutoCorRangeDenominator'] = np.square(df['PrevRange'] - df['PrevMeanRange']).where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum()
    df['AutoCorrelationRange'] = (df['PrevAutoCorRangeNumerator']/df['PrevAutoCorRangeDenominator']).round(2)
    
    df['ShiftedRange'] = df['Range'].shift(1).where(df['GroupNum'] != df['GroupNum'].shift(1))
    df['MinRange'] = df['ShiftedRange'].cummin().ffill()
    df['MaxRange'] = df['Range'].where(df['ZScoreRange'] <= 3).cummax().ffill()
    df['NormalizedRange'] = np.minimum(((df['Range'] - df['MinRange'])/(df['MaxRange'] - df['MinRange'])).round(2), 1)

    df['Lows'] = (df['Low'] - df['PrevLow']).round(2)
    df['Lows'] = np.where(df['SessionBar'] == True, 0, df['Lows'])
    df['Lows'] = np.where(df['Lows'] < 0, 0, df['Lows'])
    df['LastLows'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Lows'].shift(1), np.nan)
    df['LastLows'] = df['LastLows'].shift(1).ffill()
    df['LastLows'] = np.where(df['LastLows'] < 0, 0, df['LastLows'])
    lows_dict = df['Lows'].to_dict() 
    df['PrevLows'] = df['PrevBar'].map(lambda x: lows_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfLows'] = ((df['PrevLows'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfLows'] = np.where(df['Lows'] > 0, df['NumberOfLows']+1, df['NumberOfLows'])
    df['SumPrevLows'] = df['PrevLows'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum().ffill()
    df['PrevMeanLows'] = (df['SumPrevLows']/df['NumberOfLows']).round(2)
    df['MeanLows'] = ((df['SumPrevLows'] + df['Lows'])/df['CurrentNoOfLows']).round(2)
    df['ResidualsLows'] = (np.square(df['PrevLows'] - df['PrevMeanLows']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevLows'] > 0)).cumsum()).round(2).ffill()
    df['PrevStdLows'] = np.sqrt(df['ResidualsLows']/df['NumberOfLows'])
    df['ResidualsLows'] = df['ResidualsLows'] + np.square(df['Lows'] - df['MeanLows'])
    df['StdLows'] = (np.sqrt(df['ResidualsLows']/df['CurrentNoOfLows'])).round(2)
    df['PrevZScoreLows'] = ((df['PrevLows']-df['PrevMeanLows'])/df['PrevStdLows']).round(2)
    df['ZScoreLows'] = ((df['Lows']-df['MeanLows'])/df['StdLows']).round(2)
    
    # Simplified Skew/Kurtosis for Lows/Highs to save space - Core stats above are critical
    df['SkewLows'] = 0.0 # Placeholder if deep calc fails/skips
    df['CVLows'] = (df['StdLows']/df['MeanLows']).round(2)
    df['KurtosisLows'] = 0.0

    df['Highs'] = (df['PrevHigh'] - df['High']).round(2)
    df['Highs'] = np.where(df['SessionBar'] == True, 0, df['Highs'])
    df['Highs'] = np.where(df['Highs'] < 0, 0, df['Highs'])
    df['LastHighs'] = np.where(df['GroupNum'] != df['GroupNum'].shift(1), df['Highs'].shift(1), np.nan)
    df['LastHighs'] = df['LastHighs'].shift(1).ffill()
    df['LastHighs'] = np.where(df['LastHighs'] < 0, 0, df['LastHighs'])
    highs_dict = df['Highs'].to_dict()
    df['PrevHighs'] = df['PrevBar'].map(lambda x: highs_dict.get(int(x), np.nan) if not np.isnan(x) else np.nan)
    df['NumberOfHighs'] = ((df['PrevHighs'] > 0) & (df['GroupNum'] != df['GroupNum'].shift(1))).cumsum()
    df['CurrentNoOfHighs'] = np.where(df['Lows'] > 0, df['NumberOfHighs']+1, df['NumberOfHighs'])
    df['SumPrevHighs'] = df['PrevHighs'].where(df['GroupNum'] != df['GroupNum'].shift(1)).cumsum().ffill()
    df['PrevMeanHighs'] = (df['SumPrevHighs']/df['NumberOfHighs']).round(2)
    df['MeanHighs'] = ((df['SumPrevHighs'] + df['Highs'])/df['CurrentNoOfHighs']).round(2)
    df['ResidualsHighs'] = (np.square(df['PrevHighs'] - df['PrevMeanHighs']).where((df['GroupNum'] != df['GroupNum'].shift(1)) & (df['PrevHighs'] > 0)).cumsum()).round(2).ffill()
    df['PrevStdHighs'] = np.sqrt(df['ResidualsHighs']/df['NumberOfHighs'])
    df['ResidualsHighs'] = df['ResidualsHighs'] + np.square(df['Highs'] - df['MeanHighs'])
    df['StdHighs'] = (np.sqrt(df['ResidualsHighs']/df['CurrentNoOfHighs'])).round(2)
    df['ZScoreHighs'] = ((df['Highs']-df['MeanHighs'])/df['StdHighs']).round(2)
    
    df['SkewHighs'] = 0.0
    df['CVHighs'] = (df['StdHighs']/df['MeanHighs']).round(2)
    df['KurtosisHighs'] = 0.0

    # Predictions & Technicals
    df['le_next_low_pred'] = 2 * df['Low'] - df['Low'].shift(1)
    df['le_next_high_pred'] = 2 * df['High'] - df['High'].shift(1) 
    df['atr'] = df['Range'].rolling(window=14, min_periods=1).mean()  
    df['atr_next_low_pred'] = df['Low'] - 0.5 * df['atr']
    df['atr_next_high_pred'] = df['High'] + 0.5 * df['atr'] 
    
    # Reg Forecast
    df['reg_next_low_pred'] = df['Low'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    df['reg_next_high_pred'] = df['High'].rolling(7, min_periods=2).apply(forecast_next, raw=False)
    
    df['ema_low'] = df['Low'].ewm(span=10, adjust=False).mean()
    df['ema_high'] = df['High'].ewm(span=10, adjust=False).mean()
    df['low_std'] = df['Low'].rolling(window=10, min_periods=1).std()
    df['high_std'] = df['High'].rolling(window=10, min_periods=1).std()
    df['ema_next_low_pred'] = df['ema_low'] - df['low_std']
    df['ema_next_high_pred'] = df['ema_high'] + df['high_std']
    
    df['predicted_next_low'] = (df['le_next_low_pred'] + df['atr_next_low_pred'] + df['reg_next_low_pred']) / 3
    df['predicted_next_high'] = (df['le_next_high_pred'] + df['atr_next_high_pred'] + df['reg_next_high_pred']) / 3
    
    df['feature_imbalance'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    df['realtive_spread'] = df['Spread'] / ((df['BestBid'] + df['BestAsk']) / 2)
    df['quote_stuffing'] = df['BestBid'].diff().abs() + df['BestAsk'].diff().abs()
    df['depth_slope'] = (df['BidSize'] - df['AskSize']) / (df['BidSize'] + df['AskSize'] + 1e-9)
    
    df['price_momentum'] = df['Close'].diff()
    df['price_momentum_ratio'] = df['Close'] / df['Close'].shift(1)
    
    # Hilbert & MVR
    df['hilbert_phase'] = df['Close'].rolling(50).apply(rolling_hilbert, raw=True).shift(1)
    df['large_order_fraction'] = (df['BidSize'] + df['AskSize']) / (df['Volume'] + 1e-9)
    df['mvr'] = df['Close'].pct_change().rolling(5).std().shift(1) / df['Close'].pct_change().rolling(30).std().shift(1)

    return df

def resampled_ohlc(df):
    df = df.copy()
    
    # --- FIXED COLUMN MAPPING ---
    rename_map = {
        'BestBuyPrice': 'BuyPrice', 'BestSellPrice': 'SellPrice',
        'BestBuyQty': 'BuyQty', 'BestSellQty': 'SellQty',
        'BestBid': 'BuyPrice', 'BestAsk': 'SellPrice', # Added aliases
        'BidSize': 'BuyQty', 'AskSize': 'SellQty',
        'LastTradedQty': 'LTQ', 'LastTradedPrice': 'LTP'
    }
    df.rename(columns=rename_map, inplace=True)
    
    df['Time'] = df['Time'].astype(str).apply(clean_time_format)
    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), errors='coerce')
    df.set_index('datetime', inplace=True)

    # 15s Resampling
    interval_start = df.index.floor('15s')
    df['group'] = interval_start
    df['GroupTime'] = interval_start.time
    df['TimePassed'] = ((pd.to_datetime(df['Time'], format='%H:%M:%S.%f') - pd.to_datetime(df['GroupTime'], format='%H:%M:%S')).dt.total_seconds())

    df['Open'] = df.groupby('group')['LTP'].transform('first')
    df['High'] = df.groupby('group')['LTP'].transform('cummax')
    df['Low'] = df.groupby('group')['LTP'].transform('cummin')
    df['Close'] = df['LTP']
    
    # Safe access after rename
    df['BestBid'] = df['BuyPrice'] if 'BuyPrice' in df else np.nan
    df['BestAsk'] = df['SellPrice'] if 'SellPrice' in df else np.nan
    df['BidSize'] = df['BuyQty'] if 'BuyQty' in df else np.nan
    df['AskSize'] = df['SellQty'] if 'SellQty' in df else np.nan
    
    df['Volume'] = df.groupby('group')['LTQ'].transform('cumsum')
    df['GroupNum'] = (df['group'] != df['group'].shift(1)).cumsum()
    
    # Helper for Time features
    df['GroupTimeOnly'] = df['group'].dt.time
    intervals_list_sorted = sorted(df['GroupTimeOnly'].unique())
    time_map = {t: i + 1 for i, t in enumerate(intervals_list_sorted)}
    df['GroupNumTime'] = df['GroupTimeOnly'].map(time_map)

    ohlc = df.copy()
    ohlc.reset_index(inplace=True)
    ohlc['Date'] = ohlc['datetime'].dt.strftime('%d/%m/%Y')
    ohlc.drop(columns=['datetime'], inplace=True)
    
    mask = ohlc['GroupNum'] != ohlc['GroupNum'].shift(1)
    ohlc['PrevBar'] = np.where(mask, ohlc.index - 1, np.nan)
    ohlc['PrevBar'] = ohlc['PrevBar'].ffill()
    ohlc['PrevPrevBar'] = np.where(mask, ohlc['PrevBar'].shift(1), np.nan)
    ohlc['PrevPrevBar'] = ohlc['PrevPrevBar'].ffill()

    ohlc.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'group': 'Group'}, inplace=True)
    ohlc.dropna(subset=['Open'], inplace=True)
    ohlc = additional_columns(ohlc)
    ohlc['SessionBar'] = ohlc.groupby('Group')['Session'].transform(lambda x: (x == 0).any())
    
    return ohlc.reset_index(drop=True)

# ==========================================
# 2. PIPELINE: DATA & PREP
# ==========================================

def get_prepared_data_for_day(year, month, day):
    # Download
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{FUT_TS}.parquet"
    local_path = "test_data.parquet"
    
    s3 = boto3.client("s3")
    try:
        s3.download_file(BUCKET, key, local_path)
    except Exception as e:
        print(f"Failed to download {key}: {e}")
        return None, None

    # Load Raw Tick Data
    df_tick = pd.read_parquet(local_path)
    df_tick["DateTime"] = pd.to_datetime(
        df_tick["Date"].astype(str) + " " + df_tick["Time"].astype(str), 
        dayfirst=True, errors="coerce"
    )
    df_tick["LTP"] = pd.to_numeric(df_tick["LastTradedPrice"] if "LastTradedPrice" in df_tick else df_tick["LTP"], errors='coerce')
    df_tick = df_tick.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Process 15s Bars
    print("Generating Features (Resampling 15s)...")
    df_prep = df_tick.copy()
    df_bars = resampled_ohlc(df_prep)
    
    # Aggregation
    df_bars['DateTime'] = pd.to_datetime(df_bars['Date'] + ' ' + df_bars['Time'], format="%d/%m/%Y %H:%M:%S.%f", errors='coerce')
    df_bars.set_index('DateTime', inplace=True)
    
    # --- FIX 1: Removed 'PrevVolume' (doesn't exist yet) ---
    # --- FIX 2: Added ALL structural columns required by feature gen ---
    df_agg = df_bars.resample('15s').agg({
        'Date': 'last', 'Time': 'last', 'Open': 'first', 'High':'max', 'Low':'min', 'Close':'last',
        'BestBid':'last', 'BestAsk':'last', 'BidSize':'last', 'AskSize':'last', 'Volume':'sum',
        'GroupNum':'last',
        'Group':'last', 'GroupNumTime':'last', 'TimePassed':'last', 
        'PrevBar':'last', 'PrevPrevBar':'last', 'Session':'last', 'SessionBar':'last' 
    }).reset_index()
    
    # Generate Vars
    df_feats = generate_basic_variables(df_agg)
    
    # Add Complex Vars (Numba)
    df_feats['Close'] = pd.to_numeric(df_feats['Close'], errors='coerce')
    for w in [200, 500, 1000, 1500]:
        df_feats[f'Lyapunov_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: lyapunov_exponent_numba(x), raw=True)
        df_feats[f'Hurst_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: hurst_exponent_numba(x), raw=True)
        df_feats[f'FDI_{w}'] = df_feats['Close'].rolling(w).apply(lambda x: fractal_dimension_numba(x), raw=True)
        
    return df_tick, df_feats

# ==========================================
# 3. TEST: QUANT KINETIC LOGIC WITH TRADE MANAGEMENT
# ==========================================

class KineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.buf = deque(maxlen=50)
        self.last_score = 0
        self.last_vwap = 0.0
    
    def tick(self, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0
        
        arr = np.array(self.buf)
        p, v = arr[:,0], arr[:,1]
        v_diff = np.diff(v)
        v_diff = np.where(v_diff < 0, 0, v_diff)
        
        score = np.sum(v_diff) / (abs(p[-1]-p[0]) + 0.05)
        self.last_score = score
        
        # Calculate Tick VWAP (Micro-Trend)
        self.last_vwap = np.average(p, weights=v) if np.sum(v) > 0 else np.mean(p)
        
        return score

def run_kinetic_trading_test(df_tick, df_bars):
    print("\n--- TEST: QUANT KINETIC TRADING (Managed) ---")
    
    df_bars_lookup = df_bars.set_index('DateTime').sort_index()
    brain = KineticBrain(threshold=KINETIC_THRESHOLD)
    trades = []
    
    # Trade State
    in_trade = False
    entry_time = None
    entry_price = 0.0
    trade_type = None
    
    for i, row in df_tick.iterrows():
        ts = row['DateTime']
        ltp = row['LTP']
        vol = row['Volume'] if 'Volume' in row else 0 
        
        # --- TRADE MANAGEMENT (Check Every Tick) ---
        if in_trade:
            duration = (ts - entry_time).total_seconds()
            pnl_pts = 0
            
            if trade_type == 'LONG':
                pnl_pts = ltp - entry_price
            else:
                pnl_pts = entry_price - ltp
                
            # Exit Logic
            exit_reason = None
            if pnl_pts <= -STOP_LOSS: exit_reason = 'SL'
            elif pnl_pts >= TAKE_PROFIT: exit_reason = 'TP'
            elif duration >= MAX_HOLD_SEC: exit_reason = 'TIME'
            
            if exit_reason:
                trades.append({
                    'Entry_Time': entry_time, 'Exit_Time': ts, 'Type': trade_type, 
                    'Entry_Price': entry_price, 'Exit_Price': ltp, 
                    'PnL_Pts': pnl_pts - COST_PER_TRADE, 'Reason': exit_reason
                })
                in_trade = False
                continue

        # --- SIGNAL LOGIC ---
        k_score = brain.tick(ltp, vol)
        
        if not in_trade and k_score > KINETIC_THRESHOLD:
            try:
                latest_bar_idx = df_bars_lookup.index.asof(ts)
                if pd.isna(latest_bar_idx): continue
                
                bar_data = df_bars_lookup.loc[latest_bar_idx]
                imbalance = bar_data.get('feature_imbalance', 0)
                zscore = bar_data.get('ZScoreLTQ', 0)
                
                # Check 1: Statistical Significance
                if zscore < ZSCORE_THRESH: continue
                
                # Check 2: Imbalance + Micro-Trend Alignment (VWAP)
                micro_trend_up = ltp > brain.last_vwap
                micro_trend_down = ltp < brain.last_vwap
                
                if imbalance > IMBALANCE_THRESH and micro_trend_up:
                    trade_type = 'LONG'
                    in_trade = True
                    entry_time = ts; entry_price = ltp
                    
                elif imbalance < -IMBALANCE_THRESH and micro_trend_down:
                    trade_type = 'SHORT'
                    in_trade = True
                    entry_time = ts; entry_price = ltp
                    
            except Exception as e:
                pass

    # Results
    res = pd.DataFrame(trades)
    if not res.empty:
        print(f"Total Trades: {len(res)}")
        print(f"Total PnL: {res['PnL_Pts'].sum():.2f} pts")
        print(f"Win Rate: {(res['PnL_Pts']>0).mean()*100:.2f}%")
        print("\n--- Trade Diagnostics ---")
        print(res[['Entry_Time', 'Type', 'Reason', 'PnL_Pts']].head(10))
    else:
        print("No trades generated.")

# ==========================================
# MAIN
# ==========================================
def main():
    print(f"Loading Data for {TEST_DAY}-{TEST_MONTH}-{TEST_YEAR}...")
    df_tick, df_bars = get_prepared_data_for_day(TEST_YEAR, TEST_MONTH, TEST_DAY)
    if df_tick is None: return
    
    run_kinetic_trading_test(df_tick, df_bars)

if __name__ == "__main__":
    main()

Loading Data for 18-11-2025...
Generating Features (Resampling 15s)...

--- TEST: QUANT KINETIC TRADING (Managed) ---
Total Trades: 15
Total PnL: -23.60 pts
Win Rate: 46.67%

--- Trade Diagnostics ---
               Entry_Time   Type Reason  PnL_Pts
0 2025-11-18 09:16:35.251   LONG     TP     29.0
1 2025-11-18 09:41:03.915  SHORT     SL    -16.1
2 2025-11-18 09:50:26.218  SHORT     TP     29.0
3 2025-11-18 10:00:28.318  SHORT     SL    -19.9
4 2025-11-18 10:01:19.025  SHORT     SL    -16.0
5 2025-11-18 10:24:51.980   LONG     SL    -16.5
6 2025-11-18 11:00:26.243   LONG   TIME      5.8
7 2025-11-18 11:41:45.296  SHORT     SL    -17.7
8 2025-11-18 13:13:45.330   LONG   TIME      9.5
9 2025-11-18 13:48:12.270  SHORT   TIME      3.4


In [47]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Expiry Dates 2025 (Thursdays)
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params
KINETIC_THRESHOLD = 37500
ORB_TIME = datetime.time(10, 0) # The Trap Closes at 10:00 AM
MAX_HOLD_SECONDS = 1800         # 30 Mins Max Hold
STOP_LOSS_POINTS = 30.0
TAKE_PROFIT_POINTS = 90.0
COST_PER_TRADE = 1.0            # Slippage + Comm per trade

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        # Standardize Columns
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        # Handle LTP aliases
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        # Handle Volume aliases (we need incremental volume if possible, or OpenInterest)
        # Assuming Volume/OpenInterest column exists. 
        # If 'Volume' is cumulative, we handle diffing in the brain.
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] # Fallback
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. KINETIC HUNTER BRAIN
# ==========================================
class KineticHunterBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.tick_buffer = deque(maxlen=50)
        
    def get_signal(self, ltp, volume, orb_high, orb_low, current_time):
        self.tick_buffer.append((ltp, volume))
        
        # 1. Check Time (Must be AFTER ORB)
        if current_time <= ORB_TIME:
            return 0 # Still building the trap
            
        if len(self.tick_buffer) < 50:
            return 0
            
        # 2. Calculate Kinetic Energy
        arr = np.array(self.tick_buffer)
        prices = arr[:, 0]
        vols = arr[:, 1]
        
        # We need volume flow. If input 'volume' is cumulative, take diff.
        # If input is tick quantity (LTQ), take sum.
        # Let's assume input stream sends what we have. 
        # Robust logic:
        vol_diff = np.diff(vols)
        # If vol_diff is mostly negative (resets), assume input was cumulative.
        # If vol_diff is mostly positive/zero, it works.
        # Safe approach: Just use positive deltas
        vol_flow = np.where(vol_diff > 0, vol_diff, 0)
        
        kinetic_score = np.sum(vol_flow) / (abs(prices[-1] - prices[0]) + 0.05)
        
        # 3. Check Breakout + Kinetic
        if kinetic_score > self.threshold:
            if ltp > orb_high:
                return 1 # Breakout Long
            elif ltp < orb_low:
                return -1 # Breakout Short
        
        return 0

# ==========================================
# 3. DAILY BACKTEST ENGINE
# ==========================================
def run_day(df):
    brain = KineticHunterBrain(threshold=KINETIC_THRESHOLD)
    
    # ORB State
    orb_high = -999999
    orb_low = 999999
    orb_locked = False
    
    # Trade State
    in_trade = False
    entry_price = 0.0
    entry_time = None
    direction = 0 # 1 Long, -1 Short
    
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        curr_time = ts.time()
        
        # --- 1. ORB LOGIC ---
        if curr_time <= ORB_TIME:
            if ltp > orb_high: orb_high = ltp
            if ltp < orb_low: orb_low = ltp
            orb_locked = True # Mark that we are observing
            brain.tick_buffer.append((ltp, vol)) # Keep buffer warm
            continue
            
        # --- 2. TRADE MANAGEMENT ---
        if in_trade:
            # Check Exits
            elapsed = (ts - entry_time).total_seconds()
            pnl = 0
            
            if direction == 1: # Long
                pnl = ltp - entry_price
            else: # Short
                pnl = entry_price - ltp
                
            exit_reason = None
            if pnl <= -STOP_LOSS_POINTS: exit_reason = "SL"
            elif pnl >= TAKE_PROFIT_POINTS: exit_reason = "TP"
            elif elapsed >= MAX_HOLD_SECONDS: exit_reason = "TIME"
            
            if exit_reason:
                trades.append(pnl - COST_PER_TRADE)
                in_trade = False
                direction = 0
            continue # Don't re-enter on same tick
            
        # --- 3. SIGNAL GENERATION ---
        # Only if ORB is valid
        if orb_locked and orb_high > 0:
            sig = brain.get_signal(ltp, vol, orb_high, orb_low, curr_time)
            
            if sig != 0:
                in_trade = True
                entry_price = ltp
                entry_time = ts
                direction = sig
                
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running KINETIC HUNTER from {START_DATE} to {END_DATE}")
    print(f"Trap: 09:15 - {ORB_TIME} | Trigger: Kinetic > {KINETIC_THRESHOLD}")
    print("-" * 65)
    print(f"{'Date':<12} | {'Trades':<6} | {'Daily PnL (Pts)':<15} | {'Status'}")
    print("-" * 65)
    
    current_date = START_DATE
    total_points = 0.0
    total_trades = 0
    
    date_pnl_map = []
    
    while current_date <= END_DATE:
        # Skip weekends efficiently
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            daily_pts = sum(daily_trades)
            count = len(daily_trades)
            
            total_points += daily_pts
            total_trades += count
            
            status = "PROFIT" if daily_pts > 0 else "LOSS" if daily_pts < 0 else "FLAT"
            print(f"{current_date} | {count:<6} | {daily_pts:>10.2f} pts   | {status}")
            
            date_pnl_map.append({'Date': current_date, 'PnL': daily_pts, 'Trades': count})
        else:
            # print(f"{current_date} | NO DATA")
            pass
            
        current_date += datetime.timedelta(days=1)
        
    print("-" * 65)
    print(f"TOTAL RESULTS:")
    print(f"Gross Points: {total_points:.2f}")
    print(f"Total Trades: {total_trades}")
    print(f"Avg Pts/Day:  {total_points / len(date_pnl_map) if date_pnl_map else 0:.2f}")
    
    # Convert to rupees for context
    print(f"Total INR:    ₹{total_points * 75:,.2f}")

if __name__ == "__main__":
    main()

Running KINETIC HUNTER from 2025-09-01 to 2025-11-26
Trap: 09:15 - 10:00:00 | Trigger: Kinetic > 37500
-----------------------------------------------------------------
Date         | Trades | Daily PnL (Pts) | Status
-----------------------------------------------------------------
2025-09-01 | 7      |       4.90 pts   | PROFIT
2025-09-02 | 10     |       1.20 pts   | PROFIT
2025-09-03 | 5      |      32.70 pts   | PROFIT
2025-09-04 | 1      |     -31.20 pts   | LOSS
2025-09-05 | 12     |     -35.40 pts   | LOSS
2025-09-08 | 5      |     -57.30 pts   | LOSS
2025-09-09 | 6      |    -115.70 pts   | LOSS
2025-09-10 | 2      |     -41.00 pts   | LOSS
2025-09-11 | 5      |     -23.00 pts   | LOSS
2025-09-12 | 12     |      30.30 pts   | PROFIT
2025-09-18 | 2      |     -11.00 pts   | LOSS
2025-09-22 | 1      |      29.00 pts   | PROFIT
2025-09-23 | 5      |    -131.20 pts   | LOSS
2025-09-25 | 8      |      59.70 pts   | PROFIT
2025-10-03 | 0      |       0.00 pts   | FLAT
2025-10-06 | 1

In [51]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Expiry Dates 2025 (Thursdays)
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params: "IMBALANCE GATED SNIPER"
KINETIC_THRESHOLD = 150000   
IMBALANCE_THRESHOLD = 0.25   # Only trap if Order Book is skewed 25%
TRAP_BUFFER = 5.0            
PENDING_TIMEOUT = 60         # Faster timeout (1 min)
COOLDOWN_SECONDS = 600       # 10 Mins

# Exit Logic (Widened SL to breathe)
STOP_LOSS_POINTS = 20.0      
TAKE_PROFIT_POINTS = 40.0    # 1:2 Risk Reward

# Costs
COST_PER_TRADE = 1.0     
LOT_SIZE = 75

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {
            'LastTradedPrice': 'LTP', 'Close': 'LTP',
            'BestBuyQty': 'BidSize', 'BestSellQty': 'AskSize', # Map for Imbalance
            'BuyQty': 'BidSize', 'SellQty': 'AskSize' # Alt names
        }
        df.rename(columns=col_map, inplace=True)
        
        # Ensure we have Volume
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        # Ensure we have Bid/Ask Size
        if 'BidSize' not in df.columns: df['BidSize'] = 0
        if 'AskSize' not in df.columns: df['AskSize'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. KINETIC BRAIN + IMBALANCE
# ==========================================
class KineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.tick_buffer = deque(maxlen=50)
        
    def get_signal_setup(self, ltp, volume, bid_size, ask_size):
        self.tick_buffer.append((ltp, volume))
        
        if len(self.tick_buffer) < 50:
            return None # Not ready
            
        arr = np.array(self.tick_buffer)
        prices = arr[:, 0]
        vols = arr[:, 1]
        
        vol_diff = np.diff(vols)
        vol_flow = np.where(vol_diff > 0, vol_diff, 0)
        kinetic_score = np.sum(vol_flow) / (abs(prices[-1] - prices[0]) + 0.05)
        
        if kinetic_score > self.threshold:
            # Check Imbalance
            # (Bids - Asks) / (Bids + Asks)
            # Positive = Buying Pressure, Negative = Selling Pressure
            total_size = bid_size + ask_size + 1e-9
            imbalance = (bid_size - ask_size) / total_size
            
            return imbalance
            
        return None

# ==========================================
# 3. DAILY BACKTEST ENGINE
# ==========================================
def run_day(df):
    brain = KineticBrain(threshold=KINETIC_THRESHOLD)
    
    state = 'SEARCHING'
    
    # Pending Order
    trigger_price = 0.0
    pending_direction = 0 # 1 Long, -1 Short
    pending_start_time = None
    
    # Active Trade
    entry_price = 0.0
    target_price = 0.0
    stop_price = 0.0
    active_direction = 0
    trade_start_time = None
    
    cooldown_start = None
    
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        bid = row.BidSize
        ask = row.AskSize
        
        # --- STATE 4: COOLDOWN ---
        if state == 'COOLDOWN':
            if (ts - cooldown_start).total_seconds() > COOLDOWN_SECONDS:
                state = 'SEARCHING'
                brain.tick_buffer.clear()
            else:
                continue

        # --- STATE 1: SEARCHING ---
        if state == 'SEARCHING':
            imbalance = brain.get_signal_setup(ltp, vol, bid, ask)
            
            if imbalance is not None:
                # ONE-SIDED TRAP ONLY
                if imbalance > IMBALANCE_THRESHOLD:
                    # Buyers Dominant -> Trap Highs Only
                    state = 'PENDING'
                    trigger_price = ltp + TRAP_BUFFER
                    pending_direction = 1 # Long
                    pending_start_time = ts
                    
                elif imbalance < -IMBALANCE_THRESHOLD:
                    # Sellers Dominant -> Trap Lows Only
                    state = 'PENDING'
                    trigger_price = ltp - TRAP_BUFFER
                    pending_direction = -1 # Short
                    pending_start_time = ts
                
        # --- STATE 2: PENDING ---
        elif state == 'PENDING':
            if (ts - pending_start_time).total_seconds() > PENDING_TIMEOUT:
                state = 'SEARCHING'
                continue
                
            # Check Trigger
            triggered = False
            if pending_direction == 1 and ltp >= trigger_price:
                triggered = True
            elif pending_direction == -1 and ltp <= trigger_price:
                triggered = True
                
            if triggered:
                state = 'IN_TRADE'
                active_direction = pending_direction
                entry_price = ltp # Slippage applied at exit cost
                trade_start_time = ts
                
                if active_direction == 1:
                    target_price = entry_price + TAKE_PROFIT_POINTS
                    stop_price = entry_price - STOP_LOSS_POINTS
                else:
                    target_price = entry_price - TAKE_PROFIT_POINTS
                    stop_price = entry_price + STOP_LOSS_POINTS
                
        # --- STATE 3: IN TRADE ---
        elif state == 'IN_TRADE':
            exit_pnl = 0
            exited = False
            
            # 1. Price Targets
            if active_direction == 1: # Long
                if ltp >= target_price:
                    exit_pnl = ltp - entry_price; exited = True
                elif ltp <= stop_price:
                    exit_pnl = ltp - entry_price; exited = True
            else: # Short
                if ltp <= target_price:
                    exit_pnl = entry_price - ltp; exited = True
                elif ltp >= stop_price:
                    exit_pnl = entry_price - ltp; exited = True
            
            # 2. Time Stop (Safety Valve) No trade should last > 30 mins
            if not exited and (ts - trade_start_time).total_seconds() > 1800:
                if active_direction == 1: exit_pnl = ltp - entry_price
                else: exit_pnl = entry_price - ltp
                exited = True
            
            if exited:
                trades.append(exit_pnl - COST_PER_TRADE)
                state = 'COOLDOWN'
                cooldown_start = ts
                
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running IMBALANCE GATED SNIPER from {START_DATE} to {END_DATE}")
    print(f"Kinetic > {KINETIC_THRESHOLD} | Imbalance > {IMBALANCE_THRESHOLD}")
    print(f"Stop: {STOP_LOSS_POINTS} | Target: {TAKE_PROFIT_POINTS} | Cooldown: {COOLDOWN_SECONDS}s")
    print("-" * 65)
    print(f"{'Date':<12} | {'Trades':<6} | {'Daily PnL (Pts)':<15} | {'Status'}")
    print("-" * 65)
    
    current_date = START_DATE
    total_points = 0.0
    total_trades = 0
    date_pnl_map = []
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            daily_pts = sum(daily_trades)
            count = len(daily_trades)
            
            total_points += daily_pts
            total_trades += count
            
            status = "PROFIT" if daily_pts > 0 else "LOSS" if daily_pts < 0 else "FLAT"
            print(f"{current_date} | {count:<6} | {daily_pts:>10.2f} pts   | {status}")
            date_pnl_map.append({'Date': current_date, 'PnL': daily_pts})
        
        current_date += datetime.timedelta(days=1)
        
    print("-" * 65)
    print(f"TOTAL RESULTS:")
    print(f"Gross Points: {total_points:.2f}")
    print(f"Total Trades: {total_trades}")
    print(f"Avg Pts/Day:  {total_points / len(date_pnl_map) if date_pnl_map else 0:.2f}")
    print(f"Total INR:    ₹{total_points * LOT_SIZE:,.2f}")

if __name__ == "__main__":
    main()

Running IMBALANCE GATED SNIPER from 2025-09-01 to 2025-11-26
Kinetic > 150000 | Imbalance > 0.25
Stop: 20.0 | Target: 40.0 | Cooldown: 600s
-----------------------------------------------------------------
Date         | Trades | Daily PnL (Pts) | Status
-----------------------------------------------------------------
2025-09-01 | 5      |      63.40 pts   | PROFIT
2025-09-02 | 8      |      22.10 pts   | PROFIT
2025-09-03 | 3      |      28.00 pts   | PROFIT
2025-09-04 | 11     |      80.70 pts   | PROFIT
2025-09-05 | 9      |     -12.90 pts   | LOSS
2025-09-08 | 8      |     -74.00 pts   | LOSS
2025-09-09 | 4      |      -4.40 pts   | LOSS
2025-09-10 | 5      |     -45.50 pts   | LOSS
2025-09-11 | 3      |      23.00 pts   | PROFIT
2025-09-12 | 4      |     -44.80 pts   | LOSS
2025-09-18 | 6      |      57.00 pts   | PROFIT
2025-09-22 | 6      |     -32.80 pts   | LOSS
2025-09-23 | 8      |     -78.20 pts   | LOSS
2025-09-25 | 9      |     148.70 pts   | PROFIT
2025-10-03 | 2      |

In [50]:
"""
AWS KINETIC IRON CONDOR - COMPLETE SYSTEM
==========================================
Integrates with S3 data pipeline
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
import math
from scipy.stats import norm
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"

# Strategy
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
HOLD_SECONDS = 1800  # 30 minutes

# IC Parameters
EXPECTED_MOVE = 20
WING_WIDTH = 50
IC_DISTANCE = 25

# Costs
OPTION_ENTRY_COST = 2.0
OPTION_EXIT_COST = 2.0
IC_TOTAL_COST = 4 * (OPTION_ENTRY_COST + OPTION_EXIT_COST)  # 16 pts

print(f"\n{'='*60}")
print(f"AWS KINETIC IRON CONDOR - {MONTH}/{YEAR}")
print(f"{'='*60}")
print(f"IC Setup Cost: {IC_TOTAL_COST} points")
print(f"Expected Move: {EXPECTED_MOVE} points")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES (Same as before)
# ==========================================
def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except:
        return False


# ==========================================
# BLACK-SCHOLES
# ==========================================
def bs_call_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0.0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def bs_put_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0:
        return max(K - S, 0.0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)


# ==========================================
# KINETIC BRAIN
# ==========================================
class KineticBrainMagnitude:
    def __init__(self, threshold=37500, hold_seconds=1800):
        self.threshold = threshold
        self.hold_seconds = hold_seconds
        self.buffer = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.last_score = 0
    
    def tick(self, ts, ltp, volume):
        self.buffer.append((ltp, volume))
        
        if len(self.buffer) < 50:
            return 0
        
        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1
            return 0
        
        arr = np.array(self.buffer)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        displacement = abs(prices[-1] - prices[0])
        
        score = traded_vol / (displacement + 0.05)
        self.last_score = score
        
        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            return 1
        
        return 0


# ==========================================
# IRON CONDOR POSITION
# ==========================================
class IronCondorPosition:
    def __init__(self, entry_time, spot, T, sigma):
        self.entry_time = entry_time
        self.entry_spot = spot
        
        self.atm = round(spot / 50) * 50
        
        self.short_call_strike = self.atm + IC_DISTANCE
        self.long_call_strike = self.atm + IC_DISTANCE + WING_WIDTH
        self.short_put_strike = self.atm - IC_DISTANCE
        self.long_put_strike = self.atm - IC_DISTANCE - WING_WIDTH
        
        self.entry_premium = self._calculate_ic_value(spot, T, sigma)
        self.net_entry = self.entry_premium - IC_TOTAL_COST
    
    def _calculate_ic_value(self, S, T, sigma):
        call_spread = (bs_call_price(S, self.short_call_strike, T, sigma) -
                      bs_call_price(S, self.long_call_strike, T, sigma))
        put_spread = (bs_put_price(S, self.short_put_strike, T, sigma) -
                     bs_put_price(S, self.long_put_strike, T, sigma))
        return call_spread + put_spread
    
    def calculate_exit_pnl(self, exit_time, exit_spot, T_exit, sigma_exit):
        exit_value = self._calculate_ic_value(exit_spot, T_exit, sigma_exit)
        option_pnl = self.net_entry - (exit_value + IC_TOTAL_COST)
        
        move_from_atm = exit_spot - self.atm
        
        if abs(move_from_atm) < IC_DISTANCE:
            outcome = "WIN"
        elif abs(move_from_atm) > IC_DISTANCE + WING_WIDTH:
            outcome = "MAX_LOSS"
            option_pnl = -(WING_WIDTH + IC_TOTAL_COST)
        else:
            outcome = "PARTIAL"
        
        return {
            'Entry_Time': self.entry_time,
            'Exit_Time': exit_time,
            'Entry_Spot': self.entry_spot,
            'Exit_Spot': exit_spot,
            'Move': move_from_atm,
            'Entry_Premium': self.entry_premium,
            'Exit_Value': exit_value,
            'Option_PnL_Pts': option_pnl,
            'Option_PnL_INR': option_pnl * LOT_SIZE,
            'Outcome': outcome,
            'ATM': self.atm
        }


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{FUT_TS}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    
    if not download_parquet_to_path(key, local_path):
        return None
    
    try:
        df = pd.read_parquet(local_path)
        df["DateTime"] = pd.to_datetime(
            df["Date"].astype(str) + " " + df["Time"].astype(str),
            dayfirst=True,
            errors="coerce"
        )
        df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
        df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime")
        
        if "Volume" in df.columns:
            df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
        else:
            df["Volume"] = 0.0
        
        csv_path = os.path.join(day_folder, "FUT.csv")
        df.to_csv(csv_path, index=False)
        return csv_path
    except:
        return None


# ==========================================
# BACKTEST FOR ONE DAY
# ==========================================
def run_daily_ic_backtest(fut_csv, expiry_date, base_iv=0.15):
    df = pd.read_csv(fut_csv)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    
    brain = KineticBrainMagnitude(threshold=KINETIC_THRESHOLD, hold_seconds=HOLD_SECONDS)
    
    trades = []
    active_ic = None
    
    for _, row in df.iterrows():
        ts = row['DateTime']
        ltp = float(row['LTP'])
        vol = float(row['Volume'])
        
        signal = brain.tick(ts, ltp, vol)
        
        if signal == 1:
            T = (expiry_date - ts).total_seconds() / (365 * 24 * 3600)
            T = max(T, 0.01)
            active_ic = IronCondorPosition(ts, ltp, T, base_iv)
        
        elif signal == -1 and active_ic:
            T_exit = (expiry_date - ts).total_seconds() / (365 * 24 * 3600)
            T_exit = max(T_exit, 0.001)
            
            # IV crush after move
            exit_iv = base_iv * 0.90
            
            result = active_ic.calculate_exit_pnl(ts, ltp, T_exit, exit_iv)
            trades.append(result)
            active_ic = None
    
    return pd.DataFrame(trades)


# ==========================================
# MONTHLY EXECUTION
# ==========================================
def main():
    all_trades = []
    expiry_date = pd.Timestamp(f"{YEAR}-{MONTH}-27")  # Assume last Thursday
    
    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)
        
        print(f"{day:02d}/{MONTH}/{YEAR}...", end=" ")
        
        fut_csv = download_futures_for_day(YEAR, MONTH, day, day_folder)
        
        if not fut_csv:
            print("No data")
            continue
        
        trades_df = run_daily_ic_backtest(fut_csv, expiry_date, base_iv=0.15)
        
        if not trades_df.empty:
            daily_pnl = trades_df['Option_PnL_INR'].sum()
            n_trades = len(trades_df)
            wins = len(trades_df[trades_df['Option_PnL_Pts'] > 0])
            
            print(f"Trades: {n_trades} ({wins}W/{n_trades-wins}L) | PnL: ₹{daily_pnl:,.0f}")
            
            daily_csv = os.path.join(day_folder, "ic_results.csv")
            trades_df.to_csv(daily_csv, index=False)
            
            all_trades.append(trades_df)
        else:
            print("No signals")
    
    # Monthly summary
    if all_trades:
        full_df = pd.concat(all_trades, ignore_index=True)
        
        total_pnl = full_df['Option_PnL_INR'].sum()
        avg_pnl = full_df['Option_PnL_Pts'].mean()
        win_rate = (full_df['Option_PnL_Pts'] > 0).mean() * 100
        
        winners = full_df[full_df['Option_PnL_Pts'] > 0]
        losers = full_df[full_df['Option_PnL_Pts'] <= 0]
        
        print("\n" + "="*60)
        print(f"MONTHLY IRON CONDOR RESULTS - {MONTH}/{YEAR}")
        print("="*60)
        print(f"Total Trades:       {len(full_df)}")
        print(f"Winners:            {len(winners)} ({win_rate:.1f}%)")
        print(f"Losers:             {len(losers)} ({100-win_rate:.1f}%)")
        
        if len(winners) > 0:
            print(f"\nAvg Win:            {winners['Option_PnL_Pts'].mean():.2f} pts (₹{winners['Option_PnL_INR'].mean():,.0f})")
        if len(losers) > 0:
            print(f"Avg Loss:           {losers['Option_PnL_Pts'].mean():.2f} pts (₹{losers['Option_PnL_INR'].mean():,.0f})")
        
        print(f"\nTotal PnL:          {total_pnl/LOT_SIZE:.2f} pts (₹{total_pnl:,.2f})")
        print(f"Avg/Trade:          {avg_pnl:.2f} pts (₹{avg_pnl*LOT_SIZE:,.2f})")
        
        # Expectancy
        if len(winners) > 0 and len(losers) > 0:
            expectancy = (win_rate/100 * winners['Option_PnL_Pts'].mean() +
                         (100-win_rate)/100 * losers['Option_PnL_Pts'].mean())
            print(f"Expectancy:         {expectancy:.2f} pts/trade")
        
        print("\nOutcome Distribution:")
        print(full_df['Outcome'].value_counts())
        
        final_csv = f"data/nov_{YEAR}_iron_condor_results.csv"
        full_df.to_csv(final_csv, index=False)
        print(f"\nResults saved: {final_csv}")
        print("="*60)
        
    else:
        print("\nNo trades executed.")


if __name__ == "__main__":
    main()


AWS KINETIC IRON CONDOR - 11/2025
IC Setup Cost: 16.0 points
Expected Move: 20 points

01/11/2025... No data
02/11/2025... No data
03/11/2025... No data
04/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,594
05/11/2025... No data
06/11/2025... Trades: 12 (0W/12L) | PnL: ₹-31,150
07/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,576
08/11/2025... No data
09/11/2025... No data
10/11/2025... No data
11/11/2025... Trades: 12 (0W/12L) | PnL: ₹-31,120
12/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,543
13/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,535
14/11/2025... No data
15/11/2025... No data
16/11/2025... No data
17/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,483
18/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,467
19/11/2025... Trades: 13 (0W/13L) | PnL: ₹-30,815
20/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,421
21/11/2025... Trades: 12 (0W/12L) | PnL: ₹-28,392
22/11/2025... No data
23/11/2025... No data
24/11/2025... Trades: 12 (0W/12L) | PnL: ₹-27,277
25/11/2025... Trades: 5 (0W/5L) | PnL: ₹-10,584


In [53]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Expiry Dates 2025
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params: "ANCHORED KINETIC"
KINETIC_THRESHOLD = 37500    
ANCHOR_PERIOD = 5000         # 5000 Ticks (~2 Hours Trend)
MAX_HOLD_SECONDS = 1200      # 20 Mins

# Exit Logic
STOP_LOSS_POINTS = 30.0      
TAKE_PROFIT_POINTS = 60.0    # 1:2 Risk Reward

# Costs
COST_PER_TRADE = 1.0     
LOT_SIZE = 75

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. ANCHORED KINETIC BRAIN
# ==========================================
class AnchoredKineticBrain:
    def __init__(self, threshold=37500, anchor_period=5000):
        self.threshold = threshold
        
        # Micro Buffer (Kinetic Detection)
        self.micro_buffer = deque(maxlen=50)
        
        # Macro Buffer (The Anchor Trend)
        self.anchor_prices = deque(maxlen=anchor_period)
        self.anchor_vols = deque(maxlen=anchor_period)
        
    def get_signal(self, ltp, volume):
        # Update Buffers
        self.micro_buffer.append((ltp, volume))
        self.anchor_prices.append(ltp)
        self.anchor_vols.append(volume)
        
        if len(self.micro_buffer) < 50 or len(self.anchor_prices) < 100:
            return 0
            
        # 1. Calculate Kinetic Energy (The Trigger)
        arr = np.array(self.micro_buffer)
        prices = arr[:, 0]
        vols = arr[:, 1]
        
        vol_diff = np.diff(vols)
        vol_flow = np.where(vol_diff > 0, vol_diff, 0)
        
        kinetic_score = np.sum(vol_flow) / (abs(prices[-1] - prices[0]) + 0.05)
        
        if kinetic_score > self.threshold:
            
            # 2. Calculate Anchor VWAP (The Trend)
            # Use rolling window or simplified mean if volumes are noisy
            a_prices = np.array(self.anchor_prices)
            a_vols = np.array(self.anchor_vols)
            
            # Safe VWAP calc
            total_vol = np.sum(a_vols)
            if total_vol > 0:
                anchor_vwap = np.average(a_prices, weights=a_vols)
            else:
                anchor_vwap = np.mean(a_prices)
            
            # 3. Calculate Micro Direction (Fast VWAP)
            micro_vwap = np.average(prices[1:], weights=vol_flow) if np.sum(vol_flow) > 0 else np.mean(prices)
            
            # 4. ALIGNMENT CHECK
            
            # CASE A: BULLISH ALIGNMENT
            # Price > Anchor (Uptrend) AND Micro > VWAP (Buying Pressure)
            if ltp > anchor_vwap and ltp > micro_vwap:
                return 1 # LONG
                
            # CASE B: BEARISH ALIGNMENT
            # Price < Anchor (Downtrend) AND Micro < VWAP (Selling Pressure)
            elif ltp < anchor_vwap and ltp < micro_vwap:
                return -1 # SHORT
                
        return 0

# ==========================================
# 3. DAILY BACKTEST ENGINE
# ==========================================
def run_day(df):
    brain = AnchoredKineticBrain(threshold=KINETIC_THRESHOLD, anchor_period=ANCHOR_PERIOD)
    
    # Trade State
    in_trade = False
    entry_price = 0.0
    entry_time = None
    direction = 0 # 1 Long, -1 Short
    
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        # --- TRADE MANAGEMENT ---
        if in_trade:
            elapsed = (ts - entry_time).total_seconds()
            pnl = 0
            
            if direction == 1: # Long
                pnl = ltp - entry_price
            else: # Short
                pnl = entry_price - ltp
                
            exit_reason = None
            if pnl <= -STOP_LOSS_POINTS: exit_reason = "SL"
            elif pnl >= TAKE_PROFIT_POINTS: exit_reason = "TP"
            elif elapsed >= MAX_HOLD_SECONDS: exit_reason = "TIME"
            
            if exit_reason:
                trades.append(pnl - COST_PER_TRADE)
                in_trade = False
                direction = 0
            
            # Update Brain even while in trade
            brain.get_signal(ltp, vol) 
            continue 
            
        # --- SIGNAL GENERATION ---
        sig = brain.get_signal(ltp, vol)
        
        if sig != 0:
            in_trade = True
            entry_price = ltp
            entry_time = ts
            direction = sig
                
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running ANCHORED KINETIC from {START_DATE} to {END_DATE}")
    print(f"Trigger: Kinetic > {KINETIC_THRESHOLD} | Anchor: {ANCHOR_PERIOD} ticks")
    print(f"Stop: {STOP_LOSS_POINTS} | Target: {TAKE_PROFIT_POINTS}")
    print("-" * 65)
    print(f"{'Date':<12} | {'Trades':<6} | {'Daily PnL (Pts)':<15} | {'Status'}")
    print("-" * 65)
    
    current_date = START_DATE
    total_points = 0.0
    total_trades = 0
    
    date_pnl_map = []
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            daily_pts = sum(daily_trades)
            count = len(daily_trades)
            
            total_points += daily_pts
            total_trades += count
            
            status = "PROFIT" if daily_pts > 0 else "LOSS" if daily_pts < 0 else "FLAT"
            print(f"{current_date} | {count:<6} | {daily_pts:>10.2f} pts   | {status}")
            
            date_pnl_map.append({'Date': current_date, 'PnL': daily_pts, 'Trades': count})
        
        current_date += datetime.timedelta(days=1)
        
    print("-" * 65)
    print(f"TOTAL RESULTS:")
    print(f"Gross Points: {total_points:.2f}")
    print(f"Total Trades: {total_trades}")
    print(f"Avg Pts/Day:  {total_points / len(date_pnl_map) if date_pnl_map else 0:.2f}")
    print(f"Total INR:    ₹{total_points * LOT_SIZE:,.2f}")

if __name__ == "__main__":
    main()

Running ANCHORED KINETIC from 2025-09-01 to 2025-11-26
Trigger: Kinetic > 37500 | Anchor: 5000 ticks
Stop: 30.0 | Target: 60.0
-----------------------------------------------------------------
Date         | Trades | Daily PnL (Pts) | Status
-----------------------------------------------------------------
2025-09-01 | 17     |     -12.20 pts   | LOSS
2025-09-02 | 18     |     -22.90 pts   | LOSS
2025-09-03 | 13     |     -22.30 pts   | LOSS
2025-09-04 | 18     |      21.50 pts   | PROFIT
2025-09-05 | 21     |      40.80 pts   | PROFIT
2025-09-08 | 17     |     103.90 pts   | PROFIT
2025-09-09 | 17     |    -246.30 pts   | LOSS
2025-09-10 | 17     |     -74.70 pts   | LOSS
2025-09-11 | 14     |     -79.50 pts   | LOSS
2025-09-12 | 18     |     -77.40 pts   | LOSS
2025-09-18 | 16     |     -25.70 pts   | LOSS
2025-09-22 | 15     |      90.80 pts   | PROFIT
2025-09-23 | 19     |     213.70 pts   | PROFIT
2025-09-25 | 17     |     -30.20 pts   | LOSS
2025-10-03 | 9      |    -100.90 pts  